<a href="https://colab.research.google.com/github/SrdjanGrbic/intellij-community/blob/master/BQ_Best_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre activities needed for connection to BigQuery

**Install** Google components

In [ ]:
!pip3 install google-cloud

In [ ]:
!pip3 install google-cloud-bigquery

In [ ]:
!pip3 install google-cloud-storage

Authorization

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Basics is done

# 01 Customer Invoice Line

Create view

In [ ]:
%%bigquery --project cf-michelange-dev-sx
create or replace view metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_cust_invoice_line_qv_3 as
SELECT
f.invoice_id,
f.orig_invoice_id,
f.invoice_line_id,
f.date_of_day,
f.store_id,
f.month_id,
f.year_id,
f.art_no,
f.var_tu_key,
f.fnf_cd,
f.home_store_id,
f.cust_no,
f.license_type_cd,
f.gm_cond_type_cd,
f.advert_id,
f.prom_id,
f.disc_line_ind,
f.return_reason_cd,
f.vat_cd,
f.vat_rate,
f.vat_reverse_charge_ind,
f.lot_no,
f.lot_no_entry_method_cd,
f.sell_ean_no,
f.ean_error_cd,
f.ean_char_no,
f.vendor_no,
f.fulfillment_model_cd,
f.cod_price_type_cd,
f.sell_qty_colli,
f.sell_val_nsp,
f.sell_val_nnbp,
f.sell_val_invbp,
f.sell_val_gsp,
f.disc_val_nsp,
f.disc_charge_val_nsp,
f.disc_base_val_nsp,
f.suppl_no,
f.suppl_prod_no,
f.commercial_suppl_no,
f.private_label_cd,
f.high_vol_sales_ind,
f.cust_prof_promoter_group,
f.prof_bonus_pct,
f.prof_later_income_pct,
f.prof_shrinkage_store_pct,
f.prof_shrinkage_plat_pct,
f.prof_delvery_execution_val,
f.prof_dlvry_trnsp_drop_val,
f.share_nnbp_line_pct,
f.prof_delivery_hq_pct,
f.prof_delivery_oth_pct,
f.prof_sl_force_pct,
f.prof_kam_pct,
f.prof_sl_force_hq_pct,
f.prof_theoretical_nsp_val,
f.prof_disc_nsp_ind,
f.prof_disc_nsp_val,
f.prof_price_markup_nsp_ind,
f.prof_price_markup_nsp_val,
f.cust_domestic_ind,
f.prof_bonus_ind,
cio.dwh_insert_id,
f.conversion_rate_bc,
f.client_cd,
cust.branch_id,
cu.unique_home_store_id,
cu.unique_cust_no,    
cio.num_invoice_lines AS no_of_lines_per_invoice,
f.checkout_id AS till_no,
f.invoice_no AS invoice_no,
CAST(extract(HOUR from CAST(f.invoice_time as TIME)) * 100 + extract(MINUTE from CAST(f.invoice_time as TIME)) as NUMERIC) AS minute_id,
cast(substr(cast(f.orig_invoice_id as string), 19, 1) as NUMERIC) AS ind_invoice,
COALESCE (dlv.delivery_ind,0) AS Delivery_Flag,
COALESCE (blk.Bulk_Flag,0) AS Bulk_Flag,
COALESCE (pp.pre_picking_ind,0) AS Pre_Picking_Flag
FROM metro-bi-dl-srb-prod.mdw.dw_cust_invoice_line AS f
INNER JOIN metro-bi-dl-srb-prod.mdw.dw_cust_invoice AS i
ON f.invoice_id = i.invoice_id
AND f.home_store_id = i.home_store_id
AND f.date_of_day = i.date_of_day
INNER JOIN metro-bi-dl-srb-prod.mdw.dw_customer cust 
ON cust.cust_no = f.cust_no 
AND cust.home_store_id = f.home_store_id
LEFT OUTER JOIN metro-bi-dl-srb-prod.mdw.dw_customer_unique cu 
ON cu.unique_cust_no = cust.unique_cust_no 
AND cu.unique_home_store_id = cust.unique_home_store_id
LEFT JOIN metro-bi-dl-srb-prod.mdw.dw_cust_invoice_orig cio
ON cio.invoice_id = f.invoice_id 
AND cio.home_store_id = f.home_store_id 
AND cio.orig_invoice_id = f.orig_invoice_id 
AND cio.cust_no = f.cust_no 
AND cio.store_id = f.store_id
LEFT OUTER JOIN metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_delivery_flag dlv
ON cio.store_id = dlv.store_id
AND cio.checkout_id = dlv.checkout_id
AND cio.client_cd = dlv.client_cd
LEFT OUTER JOIN metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_pre_picking_flag pp
ON cio.store_id = pp.store_id
AND cio.checkout_id = pp.checkout_id
AND cio.client_cd = pp.client_cd
LEFT JOIN (
SELECT
a11.art_no art_no,
a11.date_of_day, 
a11.store_id,
cu.unique_home_store_id,
cu.unique_cust_no,
1 AS Bulk_Flag
FROM metro-bi-dl-srb-prod.mdw.dw_cust_invoice_line a11 
INNER JOIN metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_bulk_limit bl
ON bl.store_id = a11.store_id
AND bl.client_cd = a11.client_cd
INNER JOIN metro-bi-dl-srb-prod.mdw.dw_customer cust 
ON cust.cust_no = a11.cust_no 
AND cust.home_store_id = a11.home_store_id
LEFT OUTER JOIN metro-bi-dl-srb-prod.mdw.dw_customer_unique cu 
ON cu.unique_cust_no = cust.unique_cust_no 
AND cu.unique_home_store_id = cust.unique_home_store_id
GROUP BY 1,2,3,4,5,6
HAVING ( ABS(SUM(CASE WHEN fnf_cd = 'N' THEN a11.sell_qty_colli ELSE 0 END)) > 5 
OR ABS(SUM(CASE WHEN fnf_cd = 'F' THEN a11.sell_qty_colli ELSE 0 END)) > 0 ) 
AND ABS(SUM(a11.sell_val_nsp)) > MAX(bl.bulk_limit_val)
) AS blk
ON f.art_no = blk.art_no
AND cu.unique_home_store_id = blk.unique_home_store_id
AND cu.unique_cust_no = blk.unique_cust_no
AND f.store_id = blk.store_id
AND f.date_of_day = blk.date_of_day
WHERE cast(f.date_of_day as DATE) > cast(CONCAT(EXTRACT(YEAR FROM current_date) - 4,'-01','-01') as date) AND cast(f.date_of_day as DATE) < current_date;

Query is running:   0%|          |

""


Delete view previously created

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_cust_invoice_line_qv_3`

Query is running:   0%|          |

""


Views DW_V_QLIK_CUST_INVOICE_LINE_7

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_CUST_INVOICE_LINE_7
(
invoice_id,
orig_invoice_id,
invoice_line_id,
date_of_day,
store_id,
month_id,
year_id,
art_no,
var_tu_key,
fnf_cd,
home_store_id,
cust_no,
license_type_cd,
gm_cond_type_cd,
advert_id,
prom_id,
disc_line_ind,
return_reason_cd,
vat_cd,
vat_rate,
vat_reverse_charge_ind,
lot_no,
lot_no_entry_method_cd,
sell_ean_no,
ean_error_cd,
ean_char_no,
vendor_no,
fulfillment_model_cd,
cod_price_type_cd,
line_sell_qty_colli,
line_sell_val_nsp,
line_sell_val_nnbp,
line_sell_val_invbp,
line_sell_val_gsp,
line_disc_val_nsp,
line_disc_charge_val_nsp,
line_disc_base_val_nsp,
suppl_no,
suppl_prod_no,
commercial_suppl_no,
private_label_cd,
high_vol_sales_ind,
cust_prof_promoter_group,
prof_bonus_pct,
prof_later_income_pct,
prof_shrinkage_store_pct,
prof_shrinkage_plat_pct,
prof_delvery_execution_val,
prof_dlvry_trnsp_drop_val,
share_nnbp_line_pct,
prof_delivery_hq_pct,
prof_delivery_oth_pct,
prof_sl_force_pct,
prof_kam_pct,
prof_sl_force_hq_pct,
prof_theoretical_nsp_val,
prof_disc_nsp_ind,
prof_disc_nsp_val,
prof_price_markup_nsp_ind,
prof_price_markup_nsp_val,
cust_domestic_ind,
prof_bonus_ind,
dwh_insert_id,
conversion_rate_bc,
client_cd,
branch_id,
unique_home_store_id,
unique_cust_no,    
no_of_lines_per_invoice,
till_no,
invoice_no,
minute_id,
ind_invoice,
Delivery_Flag,
Bulk_Flag,
Pre_Picking_Flag,
MDSP_Flag
)
AS 
SELECT
f.invoice_id,
f.orig_invoice_id,
f.invoice_line_id,
f.date_of_day,
f.store_id,
f.month_id,
f.year_id,
f.art_no,
f.var_tu_key,
f.fnf_cd,
f.home_store_id,
f.cust_no,
f.license_type_cd,
f.gm_cond_type_cd,
f.advert_id,
f.prom_id,
f.disc_line_ind,
f.return_reason_cd,
f.vat_cd,
f.vat_rate,
f.vat_reverse_charge_ind,
f.lot_no,
f.lot_no_entry_method_cd,
f.sell_ean_no,
f.ean_error_cd,
f.ean_char_no,
f.vendor_no,
f.fulfillment_model_cd,
f.cod_price_type_cd,
f.sell_qty_colli,
f.sell_val_nsp,
f.sell_val_nnbp,
f.sell_val_invbp,
f.sell_val_gsp,
f.disc_val_nsp,
f.disc_charge_val_nsp,
f.disc_base_val_nsp,
f.suppl_no,
f.suppl_prod_no,
f.commercial_suppl_no,
f.private_label_cd,
f.high_vol_sales_ind,
f.cust_prof_promoter_group,
f.prof_bonus_pct,
f.prof_later_income_pct,
f.prof_shrinkage_store_pct,
f.prof_shrinkage_plat_pct,
f.prof_delvery_execution_val,
f.prof_dlvry_trnsp_drop_val,
f.share_nnbp_line_pct,
f.prof_delivery_hq_pct,
f.prof_delivery_oth_pct,
f.prof_sl_force_pct,
f.prof_kam_pct,
f.prof_sl_force_hq_pct,
f.prof_theoretical_nsp_val,
f.prof_disc_nsp_ind,
f.prof_disc_nsp_val,
f.prof_price_markup_nsp_ind,
f.prof_price_markup_nsp_val,
f.cust_domestic_ind,
f.prof_bonus_ind,
cio.dwh_insert_id,
f.conversion_rate_bc,
f.client_cd,
cust.branch_id,
cu.unique_home_store_id,
cu.unique_cust_no,    
cio.num_invoice_lines AS no_of_lines_per_invoice,
f.checkout_id AS till_no,
f.invoice_no AS invoice_no,
CAST(extract(HOUR from f.invoice_time) * 100 + extract(MINUTE from f.invoice_time) as NUMERIC) AS minute_id,
MOD(f.orig_invoice_id,10) AS ind_invoice,
COALESCE(case when d.flag=1 then 1 when cio.orig_cod_invoice_type_cd=2 then 1 else 0 end,0) as Delivery_Flag,  
COALESCE (blk.Bulk_Flag,0) AS Bulk_Flag,
COALESCE (pp.pre_picking_ind,0) AS Pre_Picking_Flag,
COALESCE (mdsp.MDSP_Flag,0) as MDSP_Flag
FROM `metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice_line` AS f
INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice` AS i
ON f.invoice_id = i.invoice_id
AND f.home_store_id = i.home_store_id
AND f.date_of_day = i.date_of_day
INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_customer` cust 
ON cust.cust_no = f.cust_no 
AND cust.home_store_id = f.home_store_id
LEFT OUTER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_customer_unique` cu 
ON cu.unique_cust_no = cust.unique_cust_no 
AND cu.unique_home_store_id = cust.unique_home_store_id
LEFT JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice_orig` cio
ON cio.invoice_id = f.invoice_id 
AND cio.home_store_id = f.home_store_id 
AND cio.orig_invoice_id = f.orig_invoice_id 
AND cio.cust_no = f.cust_no 
AND cio.store_id = f.store_id
LEFT OUTER JOIN `metro-bi-dl-srb-prod.cc_dwh_dialog.dw_udef_pre_picking_flag` pp
ON cio.store_id = pp.store_id
AND cio.checkout_id = pp.checkout_id
AND cio.client_cd = pp.client_cd
LEFT JOIN `metro-bi-wb-srb-im-s00.srb_qv_s00.del_flag_corr_42` d
on d.store_id=cio.store_id
and d.checkout_id=cio.checkout_id 
and d.invoice_no=cio.invoice_no 
and cast(d.cashier_no as numeric)=cast(cio.cashier_no as numeric)
LEFT JOIN (
						SELECT
						    a11.art_no AS art_no,
						    a11.date_of_day,
						    a11.store_id,
						    cu.unique_home_store_id,
						    cu.unique_cust_no,
						    1 AS bulk_flag
						  FROM
						    `metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice_line` AS a11
						    INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_customer` AS cust ON cust.cust_no = a11.cust_no
						     AND cust.home_store_id = a11.home_store_id
						    LEFT OUTER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_customer_unique` AS cu ON cu.unique_cust_no = cust.unique_cust_no
						     AND cu.unique_home_store_id = cust.unique_home_store_id
						  GROUP BY 1, 2, 3, 4, 5, 6
						  HAVING (abs(CAST(sum(CASE
						    WHEN upper(fnf_cd) = 'N' THEN a11.sell_qty_colli
						    ELSE 0
						  END) as FLOAT64)) > 5
						   OR abs(CAST(sum(CASE
						    WHEN upper(fnf_cd) = 'F' THEN a11.sell_qty_colli
						    ELSE 0
						  END) as FLOAT64)) > 0)
						   AND abs(sum(a11.sell_val_nsp)) > 180000
						) AS blk
ON f.art_no = blk.art_no
AND cu.unique_home_store_id = blk.unique_home_store_id
AND cu.unique_cust_no = blk.unique_cust_no
AND f.store_id = blk.store_id
AND f.date_of_day = blk.date_of_day
	left join 
	(
	SELECT 
			art_nr as art_no,
			var_tu_key,
			store_id,
			prod_lief_nr as suppl_prod_no,
			1 as MDSP_Flag
	 FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.mdsp_suppl` 
	) mdsp
	on mdsp.art_no=f.art_no
	and mdsp.var_tu_key=f.var_tu_key
	and mdsp.store_id=f.store_id
	and mdsp.suppl_prod_no=f.suppl_prod_no
WHERE f.date_of_day BETWEEN DATE(EXTRACT(YEAR FROM current_date())-4 , 1, 1) AND current_date()	
;

UDT dw_udef_bulk_limit

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_bulk_limit` LIMIT 10 ;

Query is running:   0%|          |

Downloading:   0%|          |

,store_id,bulk_limit_val,client_cd
0,21,180000,8910
1,43,180000,8910
2,25,180000,8910
3,42,180000,8910
4,44,180000,8910
5,60,180000,8910
6,20,180000,8910
7,41,180000,8910
8,22,180000,8910
9,24,180000,8910


UDT dw_udef_delivery_flag

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_delivery_flag` LIMIT 10 ;

Query is running:   0%|          |

Downloading:   0%|          |

,store_id,checkout_id,delivery_ind,client_cd
0,20,18,1,8910
1,20,41,1,8910
2,20,40,1,8910
3,21,40,1,8910
4,21,41,1,8910
5,22,41,1,8910
6,22,40,1,8910
7,23,40,1,8910
8,23,41,1,8910
9,24,41,1,8910


UDT dw_udef_pre_picking_flag

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_udef_pre_picking_flag` LIMIT 100 ;

Query is running:   0%|          |

Downloading:   0%|          |

,store_id,checkout_id,pre_picking_ind,client_cd
0,20,6,1,8910
1,21,7,1,8910
2,21,9,1,8910
3,22,3,1,8910
4,23,3,1,8910
5,24,3,1,8910
6,25,21,1,8910
7,26,4,1,8910


UDT del_flag_corr_2

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.del_flag_corr_2` LIMIT 10 ;

Query is running:   0%|          |

Downloading:   0%|          |

,store_id,checkout_id,invoice_no,cashier_no,flag
0,44,1,293016,103,1
1,23,1,291021,117,1
2,23,1,291014,117,1
3,23,1,291005,117,1
4,23,1,291008,117,1
5,23,1,291011,117,1
6,23,1,291020,117,1
7,23,1,291003,117,1
8,23,1,291009,117,1
9,23,1,291012,117,1


UDT MR_HS_Responsible

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.MR_HS_Responsible` LIMIT 10 ;

Query is running:   0%|          |

Downloading:   0%|          |

,LABEL,Home_responsible
0,BG,20
1,AD,20
2,BF,20
3,AA,20
4,AC,20
5,AB,20
6,AF,21
7,AG,21
8,BJ,21
9,BI,21


UDT kvi_art_final

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-srb-im-s00.srb_qv_s00.kvi_art_final` LIMIT 10 ;

Delete view DW_V_QLIK_CUST_INVOICE_LINE_7

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_CUST_INVOICE_LINE_7`

Query is running:   0%|          |

""


Create view for no of buying customers

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_cust_buying AS SELECT
    a11.home_store_id * 100000000 + a11.cust_no AS cust_hstore,
    1 AS cust_for_count
  FROM
    metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice_line AS a11
  GROUP BY 1
;


Query is running:   0%|          |

""


Drop view for no of buying customers

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_cust_buying`

Query is running:   0%|          |

""


Create view for no of customers visits

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_cust_visits_st_d AS SELECT
    a11.home_store_id * 100000000 + a11.cust_no AS cust_hstore,
    a11.date_of_day,
    a11.store_id,
    1 AS visit_for_count
  FROM metro-bi-dl-srb-prod.cc_dwh.dw_cust_invoice_line AS a11
  GROUP BY 1, 2, 3
;

Query is running:   0%|          |

""


Drop view for no of customers visits

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_cust_buying`

Create view old stock

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_gm_old_stock AS 
SELECT
    b.store_id,
    b.suppl_no,
    b.suppl_prod_no,
    b.art_no,
    date_sub(current_date(), interval 1 DAY) AS date_of_day,
    CASE
      WHEN date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY) BETWEEN 0 AND 30 THEN 1
      WHEN date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY) BETWEEN 31 AND 60 THEN 2
      WHEN date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY) BETWEEN 61 AND 90 THEN 3
      WHEN date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY) BETWEEN 91 AND 180 THEN 4
      WHEN date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY) BETWEEN 181 AND 360 THEN 5
      WHEN (extract(YEAR from date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY)) - 1900) * 10000 + extract(MONTH from date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY)) * 100 + extract(DAY from date_sub(date_sub(current_date(), interval 1 DAY), interval CAST(trunc(b.last_recpt_date) as INT64) DAY)) > 360 THEN 6
      ELSE 7
    END AS stock_age,
    CASE
      WHEN b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 32 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 32 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 32 DAY)) THEN 1
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 32 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 32 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 32 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 62 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 62 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 62 DAY)) THEN 2
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 62 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 62 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 62 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 92 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 92 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 92 DAY)) THEN 3
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 92 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 92 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 92 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 122 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 122 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 122 DAY)) THEN 4
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 122 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 122 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 122 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 152 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 152 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 152 DAY)) THEN 5
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 152 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 152 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 152 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 182 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 182 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 182 DAY)) THEN 6
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 182 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 182 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 182 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 212 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 212 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 212 DAY)) THEN 7
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 212 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 212 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 212 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 242 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 242 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 242 DAY)) THEN 8
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 242 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 242 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 242 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 272 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 272 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 272 DAY)) THEN 9
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 272 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 272 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 272 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 302 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 302 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 302 DAY)) THEN 10
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 302 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 302 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 302 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 332 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 332 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 332 DAY)) THEN 11
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 332 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 332 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 332 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 362 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 362 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 362 DAY)) THEN 12
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 362 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 362 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 362 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 722 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 722 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 722 DAY)) THEN 13
      WHEN b.last_recpt_date IS NULL THEN 14
      ELSE 15
    END AS m,
    CASE
      WHEN b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 92 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 92 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 92 DAY)) THEN 1
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 92 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 92 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 92 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 182 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 182 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 182 DAY)) THEN 2
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 182 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 182 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 182 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 272 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 272 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 272 DAY)) THEN 3
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 272 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 272 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 272 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 362 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 362 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 362 DAY)) THEN 4
      WHEN b.last_recpt_date <= (extract(YEAR from date_sub(current_date(), interval 362 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 362 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 362 DAY))
       AND b.last_recpt_date > (extract(YEAR from date_sub(current_date(), interval 722 DAY)) - 1900) * 10000 + extract(MONTH from date_sub(current_date(), interval 722 DAY)) * 100 + extract(DAY from date_sub(current_date(), interval 722 DAY)) THEN 5
      WHEN b.last_recpt_date IS NULL THEN 6
      ELSE 7
    END AS q,
    sum(b.stock_qty_colli) AS stock_qty_colli,
    sum(b.stock_nnbp) AS stock_nnbp
  FROM
    (
      SELECT
          CASE
             substr(concat(substr(a11.company_hier_id, 7, 2), '  '), 1, 2)
            WHEN '' THEN 0
            ELSE CAST(substr(concat(substr(a11.company_hier_id, 7, 2), '  '), 1, 2) as INT64)
          END AS store_id,
          a11.suppl_no AS suppl_no,
          a11.suppl_prod_no AS suppl_prod_no,
          a11.art_no AS art_no,
          sum(a11.stock_qty_colli) AS stock_qty_colli,
          sum(a11.stock_val_nnbp) AS stock_nnbp,
          max(last_recpt_date) AS last_recpt_date
        FROM
          metro-bi-dl-srb-prod.cc_dwh.dw_gm_stock AS a11
          LEFT OUTER JOIN metro-bi-dl-srb-prod.cc_dwh.dw_gm_activity AS a12 ON CASE
             substr(concat(substr(a11.company_hier_id, 7, 2), '  '), 1, 2)
            WHEN '' THEN 0
            ELSE CAST(substr(concat(substr(a11.company_hier_id, 7, 2), '  '), 1, 2) as INT64)
          END = CASE
             substr(concat(substr(a12.company_hier_id, 7, 2), '  '), 1, 2)
            WHEN '' THEN 0
            ELSE CAST(substr(concat(substr(a12.company_hier_id, 7, 2), '  '), 1, 2) as INT64)
          END
           AND a11.art_no = a12.art_no
        WHERE a11.stock_val_nnbp <> 0
        GROUP BY 1, 2, 3, 4
      UNION ALL
      SELECT
          a11.log_store_id AS store_id,
          a11.suppl_no AS suppl_no,
          a11.suppl_prod_no AS suppl_prod_no,
          a11.art_no AS art_no,
          sum(a11.log_stock_qty_colli) AS stock_qty_colli,
          sum(a11.log_stock_val_nnbp) AS stock_nnbp,
          max(a12.last_recpt_date) AS last_recpt_date
        FROM
          metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_stock AS a11
          LEFT OUTER JOIN metro-bi-dl-srb-prod.cc_dwh.dw_gm_activity AS a12 ON a11.log_store_id = CASE
             substr(concat(substr(a12.company_hier_id, 7, 2), '  '), 1, 2)
            WHEN '' THEN 0
            ELSE CAST(substr(concat(substr(a12.company_hier_id, 7, 2), '  '), 1, 2) as INT64)
          END
           AND a11.art_no = a12.art_no
        WHERE a11.log_stock_val_nnbp <> 0
        GROUP BY 1, 2, 3, 4
    ) AS b
  GROUP BY 1, 2, 3, 4, 5, 6, 7, 8
;


Executing query with job ID: 4a4d2e80-2a8b-4861-ae73-cd1e262bfaf3
Query executing: 0.58s


ERROR:
 400 No matching signature for function SUBSTR for argument types: INT64, INT64, INT64. Supported signatures: SUBSTR(STRING, INT64, [INT64]); SUBSTR(BYTES, INT64, [INT64]) at [78:28]

Location: EU
Job ID: 4a4d2e80-2a8b-4861-ae73-cd1e262bfaf3



Check old stock view

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select * from `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_gm_old_stock` limit 100;

Executing query with job ID: 3ae3b0bd-c4dd-4ecc-a28f-48cf932fc84d
Query executing: 0.41s


ERROR:
 404 Not found: Table metro-bi-wb-srb-im-s00:srb_qv_s00.dw_v_qlik_gm_old_stock was not found in location EU

Location: EU
Job ID: 3ae3b0bd-c4dd-4ecc-a28f-48cf932fc84d



In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.dw_v_qlik_gm_old_stock`

# 02 Customer Invoice Order

# 03 Goods Movements

## 03.01 Receipt

Create DW_V_QLIK_LOG_GM_RECPT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_RECPT
(
CLIENT_CD
,ART_NO
,var_tu_key 
,tunit_no 
,BUY_TUNIT_NO
,DATE_OF_DAY
,WEEK_ID
,MONTH_ID
,MONTH_WEEK_ID
,SUPPL_NO
,SUPPL_HIST_NO
,SUPPL_PROD_NO
,SUPPL_PROD_HIST_NO
,GM_COND_TYPE_CD
,GM_ORIGIN_CD
,LOG_RESERV_TYPE_CD
,LOG_ADVERT_ID
,LOG_RECPT_COMMISSION_IND
,LOG_RECPT_TYPE_CD
,LOG_RET_REASON_CD
,LOG_PLAN_DELVRY_DATE
,LOG_NAT_REB_IND
,LOG_BOOKING_DATE
,LOG_ORDER_PART_DELVRY_IND
,LOG_NUM_ART_DOC
,IMPORT_IND
,LOG_RECPT_IDENT_NO
,LOG_ORDER_IDENT_NO
,LOG_SUPPL_RETURN_IND
,LOG_RECPT_CORRECTION_IND
,LOG_DELVRY_NOTE_NO
,LOG_RECPT_QTY_PC
,LOG_RECPT_QTY_KG
,LOG_RECPT_QTY_COLLI
,LOG_RECPT_BUY_QTY_COLLI
,LOG_RECPT_VAL_NNBP
,LOG_RECPT_VAL_INVBP
,LOG_RECPT_VAL_NFP
,LOG_DELVRY_NOTE_QTY_PC
,LOG_DELVRY_NOTE_QTY_KG
,LOG_DELVRY_NOTE_QTY_COLLI
,LOG_DELVRY_NOTE_BUY_QTY_COLLI
,LOG_NORM_RECPT_QTY_PC
,LOG_NORM_RECPT_QTY_KG
,LOG_NORM_RECPT_QTY_COLLI
,LOG_NORM_RECPT_BUY_QTY_COLLI
,LOG_NORM_RECPT_VAL_NNBP
,LOG_NORM_RECPT_VAL_INVBP
,LOG_NORM_RECPT_VAL_NFP
,LOG_NORM_DELVRY_NOTE_QTY_PC
,LOG_NORM_DELVRY_NOTE_QTY_KG
,LOG_NORM_DELVRY_NOTE_QTY_COLLI
,LOG_NORM_DLVRY_NOTE_BY_QTY_COL
,LOG_PROM_RECPT_QTY_PC
,LOG_PROM_RECPT_QTY_KG
,LOG_PROM_RECPT_QTY_COLLI
,LOG_PROM_RECPT_BUY_QTY_COLLI  
,LOG_PROM_RECPT_VAL_NNBP
,LOG_PROM_RECPT_VAL_INVBP
,LOG_PROM_RECPT_VAL_NFP
,LOG_PROM_DELVRY_NOTE_QTY_PC
,LOG_PROM_DELVRY_NOTE_QTY_KG
,LOG_PROM_DELVRY_NOTE_QTY_COLLI
,LOG_PROM_DLVRY_NOTE_BY_QTY_COL
,LOG_ORDER_QTY_PC
,LOG_ORDER_QTY_KG
,LOG_ORDER_QTY_COLLI
,LOG_ORDER_BUY_QTY_COLLI
,LOG_DIFF_RECPT_DELVRY_QTY_COLL
,LOG_DIFF_RCPT_DLVRY_BY_QTY_COL
,ART_NAME
,ART_NAME_TL
,VAR_NO
,STRATBUY_DOMAIN_ID
,MGE_MAIN_CAT_ID
,MGE_CAT_ID
,MGE_SUB_CAT_ID
,MGE_MAIN_CAT_DESC
,MGE_MAIN_CAT_DESC_TL
,MGE_CAT_DESC
,MGE_CAT_DESC_TL
,MGE_SUB_CAT_DESC
,MGE_SUB_CAT_DESC_TL
,STRATBUY_DOMAIN_DESC
,STRATBUY_DOMAIN_DESC_TL
,MIKG_ART_NO
,MIKG_ART_NAME
,MIKG_ART_NAME_TL
,VAR_TU_NAME
,VAR_TU_NAME_TL
,IBW_ART_NO
,IBW_ART_NAME
,IBW_ART_NAME_TL
,SEASON_CD
,PACK_TYPE_CD
,ASSORT_SECTION_ID
,CATMAN_HEAD_ID
,CATMANAGER_ID
,SALES_SUB_DOMAIN_ID
,PCG_MAIN_CAT_ID
,PCG_CAT_ID
,PCG_SUB_CAT_ID
,SALES_DOMAIN_ID
,FNF_CD
,DEMAND_FIELD_ID
,DEMAND_FIELD_DOMAIN_ID
,PCG_MAIN_CAT_DESC
,PCG_MAIN_CAT_DESC_TL
,PCG_CAT_DESC
,PCG_CAT_DESC_TL
,PCG_SUB_CAT_DESC
,PCG_SUB_CAT_DESC_TL
,DEMAND_FIELD_DESC
,DEMAND_FIELD_DOMAIN_DESC
,DEMAND_FIELD_DOMAIN_DESC_TL
,LOG_STORE_ID
,LOG_DL_ID
,LOG_PROFIT_CENTER_ID
,LOG_STORE_DESC
,LOG_DL_DESC
,LOG_STORE_SDESC
,IBW_STORE_WEEK_SELL_FC
,IBW_STORE_SELL_FC
,WEEK_SELL_FC
,EXTRACTION_ID
)
AS 
SELECT
f.client_cd
,f.art_no
,f.var_tu_key
,f.tunit_no
,f.buy_tunit_no
,t.DATE_OF_DAY
,t.week_id
,t.month_id
,t.MONTH_ID * 1000000 + t.WEEK_ID
,f.SUPPL_NO
,f.SUPPL_NO
,f.SUPPL_PROD_NO
,f.SUPPL_PROD_NO
,f.GM_COND_TYPE_CD
,f.GM_ORIGIN_CD
,f.log_reserv_type_cd
,f.log_advert_id
,f.log_recpt_commission_ind
,f.log_recpt_type_cd
,f.log_ret_reason_cd
,f.log_plan_delvry_date
,f.log_nat_reb_ind
,f.log_booking_date
,f.log_order_part_delvry_ind
,f.log_num_art_doc
,f.import_ind
,f.log_recpt_ident_no
,f.log_order_ident_no
,f.log_suppl_return_ind
,f.log_recpt_correction_ind
,f.log_delvry_note_no
,f.LOG_RECPT_QTY_PC
,f.LOG_RECPT_QTY_KG
,f.LOG_RECPT_QTY_COLLI
,f.LOG_RECPT_BUY_QTY_COLLI
,f.LOG_RECPT_VAL_NNBP
,f.LOG_RECPT_VAL_INVBP
,f.LOG_RECPT_VAL_NFP 
,f.LOG_DELVRY_NOTE_QTY_PC
,f.LOG_DELVRY_NOTE_QTY_KG
,f.LOG_DELVRY_NOTE_QTY_COLLI
,f.LOG_DELVRY_NOTE_BUY_QTY_COLLI
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_RECPT_QTY_PC ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_RECPT_QTY_KG ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_RECPT_QTY_COLLI ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_RECPT_BUY_QTY_COLLI ELSE 0.0 END   
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN CAST(f.LOG_RECPT_VAL_NNBP  AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN CAST(f.LOG_RECPT_VAL_INVBP AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN CAST(f.LOG_RECPT_VAL_NFP   AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_DELVRY_NOTE_QTY_PC ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_DELVRY_NOTE_QTY_KG ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_DELVRY_NOTE_QTY_COLLI ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 0 THEN f.LOG_DELVRY_NOTE_BUY_QTY_COLLI ELSE 0.0 END  
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_RECPT_QTY_PC ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_RECPT_QTY_KG ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_RECPT_QTY_COLLI ELSE 0.0 END 
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_RECPT_BUY_QTY_COLLI ELSE 0.0 END  
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN CAST(f.LOG_RECPT_VAL_NNBP  AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN CAST(f.LOG_RECPT_VAL_INVBP AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN CAST(f.LOG_RECPT_VAL_NFP   AS DECIMAL(17,4)) ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_DELVRY_NOTE_QTY_PC ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_DELVRY_NOTE_QTY_KG ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_DELVRY_NOTE_QTY_COLLI ELSE 0.0 END
,CASE WHEN f.GM_COND_TYPE_CD = 1 THEN f.LOG_DELVRY_NOTE_BUY_QTY_COLLI ELSE 0.0 END
,f.LOG_ORDER_QTY_PC
,f.LOG_ORDER_QTY_KG
,f.LOG_ORDER_QTY_COLLI
,f.LOG_ORDER_BUY_QTY_COLLI
,f.LOG_RECPT_QTY_COLLI - f.LOG_DELVRY_NOTE_QTY_COLLI
,f.LOG_RECPT_BUY_QTY_COLLI - f.LOG_DELVRY_NOTE_BUY_QTY_COLLI   
,a.ART_NAME
,a.ART_NAME_TL
,f.var_no
,a.STRATBUY_DOMAIN_ID
,a.MGE_MAIN_CAT_ID
,a.MGE_CAT_ID
,a.MGE_SUB_CAT_ID
,a.MGE_MAIN_CAT_DESC
,a.MGE_MAIN_CAT_DESC_TL
,a.MGE_CAT_DESC
,a.MGE_CAT_DESC_TL
,a.MGE_SUB_CAT_DESC
,a.MGE_SUB_CAT_DESC_TL
,a.STRATBUY_DOMAIN_DESC
,a.STRATBUY_DOMAIN_DESC_TL
,a.MIKG_ART_NO
,a.ART_NAME
,a.ART_NAME_TL
,concat(coalesce(a.var_type_desc, ''), ', ', coalesce(a.var_size_desc, ''), ', ', coalesce(cast(a.tunit_qty as string), ''))
,concat(coalesce(a.var_type_desc_tl, ''), ', ', coalesce(a.var_size_desc_tl, ''), ', ', coalesce(cast(a.tunit_qty as string), ''))
,a.ibw_art_no
,a.art_name
,a.art_name_tl
,a.SEASON_CD
,COALESCE(a.pack_type_cd, '')
,a.ASSORT_SECTION_ID
,a.CATMAN_HEAD_ID
,a.CATMANAGER_ID
,a.SALES_SUB_DOMAIN_ID
,a.PCG_MAIN_CAT_ID
,a.PCG_CAT_ID
,a.PCG_SUB_CAT_ID
,a.SALES_DOMAIN_ID
,a.FNF_CD
,a.DEMAND_FIELD_ID
,a.DEMAND_FIELD_DOMAIN_ID
,a.PCG_MAIN_CAT_DESC
,a.PCG_MAIN_CAT_DESC_TL
,a.PCG_CAT_DESC
,a.PCG_CAT_DESC_TL
,a.PCG_SUB_CAT_DESC
,a.PCG_SUB_CAT_DESC_TL
,a.DEMAND_FIELD_DESC
,a.DEMAND_FIELD_DOMAIN_DESC
,a.DEMAND_FIELD_DOMAIN_DESC_TL
,co_s.LOG_STORE_ID
,co_s.LOG_DL_ID
,CAST(-1 as NUMERIC)
,co_s.LOG_STORE_DESC
,CAST(NULL AS STRING)
,CAST(NULL AS STRING)
,t.week_id*10000 + co_s.log_store_id
,co_s.log_store_id
,t.week_id
,f.EXTRACTION_ID
FROM `metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_recpt` f
INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_art_var_tu` a
ON f.art_no     = a.art_no
AND f.var_tu_key = a.var_tu_key
INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_log_co_store` co_s
ON f.log_company_hier_id = co_s.log_company_hier_id
INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_time_day` t
ON t.date_of_day = f.date_of_day
;

Delete DW_V_QLIK_LOG_GM_RECPT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_RECPT`

Create DW_V_QLIK_GM_RECPT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_RECPT
(
company_hier_id,
art_no,
var_tu_key,
suppl_no,
suppl_prod_no,
buy_tunit_no,
gm_cond_type_cd,
gm_origin_cd,
date_of_day,
private_label_cd,
prom_id,
advert_id,
recpt_commission_ind,
recpt_type_cd,
ret_reason_cd,
recpt_ident_no,
dispo_tool_cd,
tbp_validity_cd,
booking_date,
recpt_qty_pc,
recpt_qty_kg,
recpt_qty_colli,
recpt_val_nnbp,
recpt_val_invbp,
recpt_val_nsp,
recpt_val_gsp,
recpt_val_tbp,
order_qty_pc,
order_qty_kg,
order_qty_colli,
delvry_note_qty_pc,
delvry_note_qty_kg,
delvry_note_qty_colli,
plan_delvry_date,
set_type_cd,
set_art_no,
set_var_tu_key,
base_art_qty,
invbp_quota,
nat_reb_ind,
week_id,
month_id,
dest_company_hier_id,
conversion_rate_bc,
delvry_note_no,
order_ident_no,
order_part_delvry_ind,
best_before_date,
bbd_estimated,
best_before_date_cd,
refusal_1_cd,
refusal_2_cd,
num_days_over_bbd,
extraction_id,
dwh_insert_id,
client_cd
)
AS
SELECT
f.company_hier_id,
f.art_no,
f.var_tu_key,
f.suppl_no,
f.suppl_prod_no,
f.buy_tunit_no,
f.gm_cond_type_cd,
f.gm_origin_cd,
f.date_of_day,
f.private_label_cd,
f.prom_id,
f.advert_id,
f.recpt_commission_ind,
f.recpt_type_cd,
f.ret_reason_cd,
f.recpt_ident_no,
f.dispo_tool_cd,
f.tbp_validity_cd,
f.booking_date,
f.recpt_qty_pc,
f.recpt_qty_kg,
f.recpt_qty_colli,
f.recpt_val_nnbp,
f.recpt_val_invbp,
f.recpt_val_nsp,
f.recpt_val_gsp,
f.recpt_val_tbp,
f.order_qty_pc,
f.order_qty_kg,
f.order_qty_colli,
f.delvry_note_qty_pc,
f.delvry_note_qty_kg,
f.delvry_note_qty_colli,
f.plan_delvry_date,
f.set_type_cd,
f.set_art_no,
f.set_var_tu_key,
f.base_art_qty,
f.invbp_quota,
f.nat_reb_ind,
f.week_id,
f.month_id,
f.dest_company_hier_id,
f.conversion_rate_bc,
f.delvry_note_no,
f.order_ident_no,
f.order_part_delvry_ind,
f.best_before_date,
f.bbd_estimated,
f.best_before_date_cd,
f.refusal_1_cd,
f.refusal_2_cd,
f.num_days_over_bbd,
f.extraction_id,
f.dwh_insert_id,
f.client_cd
FROM `metro-bi-dl-srb-prod.cc_dwh.dw_gm_recpt` f
WHERE f.date_of_day BETWEEN DATE(EXTRACT(YEAR FROM current_date())-4 , 1, 1) AND current_date()	
;

Drop DW_V_QLIK_GM_RECT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_RECPT`

# Old Stock View

Create DW_V_QLIK_GM_OLD_STOCK

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_OLD_STOCK
(
store_id,
suppl_no,
suppl_prod_no,
art_no,
date_of_day,
Stock_Age,
m,
q,  
stock_qty_colli,
stock_nnbp
) 
AS 
SELECT
b.store_id,
b.suppl_no,
b.suppl_prod_no,
b.art_no,
current_date - 1 AS date_of_day,
CASE 
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) BETWEEN   0 AND  30 THEN 1
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) BETWEEN  31 AND  60 THEN 2
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) BETWEEN  61 AND  90 THEN 3
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) BETWEEN  91 AND 180 THEN 4
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) BETWEEN 181 AND 360 THEN 5
	WHEN date_diff(date (current_date-1),date(b.last_recpt_date), day) > 360 THEN 6
	ELSE 7
END AS stock_age,
CASE
	WHEN  b.last_recpt_date >  (current_date -  32) THEN 1
	WHEN  b.last_recpt_date <= (current_date -  32) AND b.last_recpt_date > (current_date -  62) THEN 2
	WHEN  b.last_recpt_date <= (current_date -  62) AND b.last_recpt_date > (current_date -  92) THEN 3
	WHEN  b.last_recpt_date <= (current_date -  92) AND b.last_recpt_date > (current_date - 122) THEN 4
	WHEN  b.last_recpt_date <= (current_date - 122) AND b.last_recpt_date > (current_date - 152) THEN 5
	WHEN  b.last_recpt_date <= (current_date - 152) AND b.last_recpt_date > (current_date - 182) THEN 6
	WHEN  b.last_recpt_date <= (current_date - 182) AND b.last_recpt_date > (current_date - 212) THEN 7
	WHEN  b.last_recpt_date <= (current_date - 212) AND b.last_recpt_date > (current_date - 242) THEN 8
	WHEN  b.last_recpt_date <= (current_date - 242) AND b.last_recpt_date > (current_date - 272) THEN 9
	WHEN  b.last_recpt_date <= (current_date - 272) AND b.last_recpt_date > (current_date - 302) THEN 10
	WHEN  b.last_recpt_date <= (current_date - 302) AND b.last_recpt_date > (current_date - 332) THEN 11
	WHEN  b.last_recpt_date <= (current_date - 332) AND b.last_recpt_date > (current_date - 362) THEN 12
	WHEN  b.last_recpt_date <= (current_date - 362) AND b.last_recpt_date > (current_date - 722) THEN 13
	WHEN  b.last_recpt_date IS NULL                                                              THEN 14
ELSE  15 END AS m,
CASE
	WHEN  b.last_recpt_date >  (current_date -  92) THEN 1
	WHEN  b.last_recpt_date <= (current_date -  92) AND b.last_recpt_date > (current_date - 182) THEN 2
	WHEN  b.last_recpt_date <= (current_date - 182) AND b.last_recpt_date > (current_date - 272) THEN 3
	WHEN  b.last_recpt_date <= (current_date - 272) AND b.last_recpt_date > (current_date - 362) THEN 4
	WHEN  b.last_recpt_date <= (current_date - 362) AND b.last_recpt_date > (current_date - 722) THEN 5
	WHEN  b.last_recpt_date IS NULL THEN 6
ELSE 7 END AS q,  
SUM(b.stock_qty_colli) AS stock_qty_colli,
SUM(b.stock_nnbp) AS stock_nnbp
FROM
(
SELECT 
	 MOD(a11.company_hier_id,100) AS store_id,
	 a11.suppl_no AS suppl_no,
	 a11.suppl_prod_no AS suppl_prod_no,
	 a11.art_no AS art_no,
	 SUM(a11.stock_qty_colli) AS stock_qty_colli,
	 SUM(a11.stock_val_nnbp) AS stock_nnbp,
	 MAX(a12.last_recpt_date) AS last_recpt_date
FROM metro-bi-dl-srb-prod.cc_dwh.dw_gm_stock a11
LEFT JOIN metro-bi-dl-srb-prod.cc_dwh.dw_gm_activity a12 ON (MOD(a11.company_hier_id,100) = MOD(a12.company_hier_id,100)  AND   a11.art_no = a12.art_no)
    WHERE a11.stock_val_nnbp <> 0 
GROUP BY 1,2,3,4
	UNION ALL
SELECT  
	 a11.log_store_id AS store_id,
	 a11.suppl_no AS suppl_no,
	 a11.suppl_prod_no AS suppl_prod_no,
	 a11.art_no AS art_no,
	 sum(a11.log_stock_qty_colli) AS stock_qty_colli,
	 sum(a11.log_stock_val_nnbp) AS stock_nnbp,
	max(a12.last_recpt_date) AS last_recpt_date
FROM metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_stock a11
LEFT JOIN metro-bi-dl-srb-prod.cc_dwh.dw_gm_activity a12  ON (a11.log_store_id = MOD(a12.company_hier_id,100) AND   a11.art_no=a12.art_no )
    WHERE a11.log_stock_val_nnbp <> 0 
GROUP BY 1,2,3,4
) AS b
GROUP BY 1,2,3,4,5,6,7,8
;



ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f3fafa58d30>)


Delete DW_V_QLIK_GM_OLD_STOCK

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_OLD_STOCK`

# Log_Gm_Hidden_Profit

LOG_GM_HIDDEN_PROFIT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_HIDDEN_PROFIT
(
CLIENT_CD,
ART_NO,
var_tu_key,
VAR_NO,
tunit_no,
BUY_TUNIT_NO,
SUPPL_NO,
SUPPL_HIST_NO,
SUPPL_PROD_NO,
SUPPL_PROD_HIST_NO,
LOG_RESERV_TYPE_CD,
GM_ORIGIN_CD,
DATE_OF_DAY,
WEEK_ID,
MONTH_ID,
LOG_HDP_QTY_PC,
LOG_HDP_QTY_KG,
LOG_HDP_QTY_COLLI,
LOG_HDP_buy_QTY_COLLI,
LOG_HDP_VAL_NNBP,
LOG_HDP_VAL_INVBP,
LOG_HDP_VAL_NFP,
LOG_BOOKING_DATE,
IMPORT_IND,
ART_NAME,
ART_NAME_TL,
STRATBUY_DOMAIN_ID,
MGE_MAIN_CAT_ID,
MGE_CAT_ID,
MGE_SUB_CAT_ID,
MGE_MAIN_CAT_DESC,
MGE_MAIN_CAT_DESC_TL,
MGE_CAT_DESC,
MGE_CAT_DESC_TL,
MGE_SUB_CAT_DESC,
MGE_SUB_CAT_DESC_TL,
STRATBUY_DOMAIN_DESC,
STRATBUY_DOMAIN_DESC_TL,
SEASON_CD,
ASSORT_SECTION_ID,
CATMAN_HEAD_ID,
CATMANAGER_ID,
SALES_SUB_DOMAIN_ID,
PCG_MAIN_CAT_ID,
PCG_CAT_ID,
PCG_SUB_CAT_ID,
SALES_DOMAIN_ID,
FNF_CD,
DEMAND_FIELD_ID,
DEMAND_FIELD_DOMAIN_ID,
PCG_MAIN_CAT_DESC,
PCG_MAIN_CAT_DESC_TL,
PCG_CAT_DESC,
PCG_CAT_DESC_TL,
PCG_SUB_CAT_DESC,
PCG_SUB_CAT_DESC_TL,
DEMAND_FIELD_DESC,
DEMAND_FIELD_DOMAIN_DESC,
DEMAND_FIELD_DOMAIN_DESC_TL,
LOG_STORE_ID,
LOG_DL_ID,
LOG_PROFIT_CENTER_ID,
LOG_STORE_DESC,
LOG_DL_DESC,
EXTRACTION_ID
)
AS 
SELECT
f.CLIENT_CD,
f.ART_NO,
f.var_tu_key, 
f.VAR_NO,
f.tunit_no, 
f.BUY_TUNIT_NO,
f.SUPPL_NO,
f.SUPPL_NO,
f.SUPPL_PROD_NO,
f.SUPPL_PROD_NO,
f.LOG_RESERV_TYPE_CD,
f.GM_ORIGIN_CD,
f.DATE_OF_DAY,
f.WEEK_ID,
f.MONTH_ID,
f.LOG_HDP_QTY_PC,
f.LOG_HDP_QTY_KG,
f.LOG_HDP_QTY_COLLI,
f.LOG_HDP_buy_QTY_COLLI,
f.log_hdp_val_nnbp,			
f.log_hdp_val_invbp,		
f.LOG_HDP_VAL_NFP,		
f.LOG_BOOKING_DATE,
f.IMPORT_IND,
a.ART_NAME,
a.ART_NAME_TL,
a.STRATBUY_DOMAIN_ID,
a.MGE_MAIN_CAT_ID,
a.MGE_CAT_ID,
a.MGE_SUB_CAT_ID,
a.MGE_MAIN_CAT_DESC,
a.MGE_MAIN_CAT_DESC_TL,
a.MGE_CAT_DESC,
a.MGE_CAT_DESC_TL,
a.MGE_SUB_CAT_DESC,
a.MGE_SUB_CAT_DESC_TL,
a.STRATBUY_DOMAIN_DESC,
a.STRATBUY_DOMAIN_DESC_TL,
COALESCE(a.SEASON_CD, ''),
a.ASSORT_SECTION_ID,
a.CATMAN_HEAD_ID,
a.CATMANAGER_ID,
a.SALES_SUB_DOMAIN_ID,
a.PCG_MAIN_CAT_ID,
a.PCG_CAT_ID,
a.PCG_SUB_CAT_ID,
a.SALES_DOMAIN_ID,
a.FNF_CD,
a.DEMAND_FIELD_ID,
a.DEMAND_FIELD_DOMAIN_ID,
a.PCG_MAIN_CAT_DESC,
a.PCG_MAIN_CAT_DESC_TL,
a.PCG_CAT_DESC,
a.PCG_CAT_DESC_TL,
a.PCG_SUB_CAT_DESC,
a.PCG_SUB_CAT_DESC_TL,
a.DEMAND_FIELD_DESC,
a.DEMAND_FIELD_DOMAIN_DESC,
a.DEMAND_FIELD_DOMAIN_DESC_TL,
co_s.LOG_STORE_ID,
co_s.LOG_DL_ID,
CAST(-1 as NUMERIC),  
co_s.LOG_STORE_DESC,
CAST(NULL AS STRING),
f.EXTRACTION_ID
FROM metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_hidden_profit f
INNER JOIN metro-bi-dl-srb-prod.cc_dwh.dw_article a ON  f.art_no = a.art_no
INNER JOIN metro-bi-dl-srb-prod.cc_dwh.dw_log_co_store co_s ON  co_s.log_company_hier_id = f.log_company_hier_id
;

Query is running:   0%|          |

""


Delete DW_V_QLIK_LOG_GM_HIDDEN_PROFIT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_HIDDEN_PROFIT`

Query is running:   0%|          |

""


# Stock

Create DW_V_QLIK_GM_STOCK_ALL

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_STOCK_ALL
(
company_hier_id,
art_no,
var_tu_key,
gm_origin_cd,
suppl_no,
suppl_prod_no,
date_of_day,
private_label_cd,
dispo_tool_cd,
art_block_cd,
presence_week_from,
presence_week_to,
presence_ind,
days_out_of_stock,
stock_cons_qty_pc,
stock_cons_qty_kg,
stock_cons_qty_colli,
stock_qty_pc,
stock_qty_kg,
stock_qty_colli,
stock_val_nnbp,
stock_val_invbp,
stock_val_tbp,
stock_val_nsp,
stock_val_gsp,
week_id,
month_id,
dwh_insert_id,
extraction_id,
conversion_rate_bc,
client_cd
) 
AS
SELECT 
company_hier_id,
art_no,
var_tu_key,
gm_origin_cd,
suppl_no,
suppl_prod_no,
date_of_day,
private_label_cd,
dispo_tool_cd,
art_block_cd,
presence_week_from,
presence_week_to,
presence_ind,
days_out_of_stock,
stock_cons_qty_pc,
stock_cons_qty_kg,
stock_cons_qty_colli,
stock_qty_pc,
stock_qty_kg,
stock_qty_colli,
stock_val_nnbp,
stock_val_invbp,
stock_val_tbp,
stock_val_nsp,
stock_val_gsp,
week_id,
month_id,
dwh_insert_id,
extraction_id,
conversion_rate_bc,
client_cd
FROM metro-bi-dl-srb-prod.cc_dwh.dw_gm_stock 
;


Delete DW_V_QLIK_GM_STOCK_ALL

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_STOCK_ALL`

Create dw_log_gm_stock

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_STOCK_ALL
(
log_company_hier_id,
log_store_id,
art_no,
var_tu_key,
var_no,
tunit_no,
buy_tunit_no,
gm_origin_cd,
log_reserv_type_cd,
suppl_no,
suppl_prod_no,
date_of_day,
log_last_recpt_date,
log_last_sale_date,
log_stock_qty_pc,
log_stock_qty_kg,
log_stock_qty_colli,
log_stock_buy_qty_colli,
log_stock_val_nnbp,
log_stock_val_invbp,
log_stock_val_nfp,
import_ind,
conversion_rate_bc,
dwh_insert_id,
extraction_id,
client_cd
) 
AS
SELECT
log_company_hier_id,
log_store_id,
art_no,
var_tu_key,
var_no,
tunit_no,
buy_tunit_no,
gm_origin_cd,
log_reserv_type_cd,
suppl_no,
suppl_prod_no,
date_of_day,
log_last_recpt_date,
log_last_sale_date,
log_stock_qty_pc,
log_stock_qty_kg,
log_stock_qty_colli,
log_stock_buy_qty_colli,
log_stock_val_nnbp,
log_stock_val_invbp,
log_stock_val_nfp,
import_ind,
conversion_rate_bc,
dwh_insert_id,
extraction_id,
client_cd
FROM metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_stock 
;

Delete DW_V_QLIK_LOG_GM_STOCK_ALL

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_STOCK_ALL`

Create view DW_V_QLIK_LOG_GM_STOCK

In [ ]:
%%bigquery --project cf-michelange-dev-sx 
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_STOCK
 (
    client_cd
   ,art_no
   ,var_tu_key
   ,tunit_no
   ,buy_tunit_no
   ,date_of_day
   ,week_id
   ,month_id
   ,month_week_id
   ,suppl_no
   ,suppl_hist_no
   ,suppl_prod_no
   ,suppl_prod_hist_no
   ,gm_origin_cd
   ,log_reserv_type_cd

   ,import_ind

   ,log_stock_qty_pc
   ,log_stock_qty_kg
   ,log_stock_qty_colli
   ,log_stock_buy_qty_colli
   ,log_stock_val_nnbp
   ,log_stock_val_invbp
   ,log_stock_val_nfp

   ,log_last_recpt_date
   ,log_last_sale_date

   ,art_name
   ,art_name_tl
   ,var_no
   ,stratbuy_domain_id
   ,mge_main_cat_id
   ,mge_cat_id
   ,mge_sub_cat_id
   ,mge_main_cat_desc
   ,mge_main_cat_desc_tl
   ,mge_cat_desc
   ,mge_cat_desc_tl
   ,mge_sub_cat_desc
   ,mge_sub_cat_desc_tl
   ,stratbuy_domain_desc
   ,stratbuy_domain_desc_tl
   ,mikg_art_no
   ,mikg_art_name
   ,mikg_art_name_tl
   ,var_tu_name
   ,var_tu_name_tl
   ,ibw_art_no
   ,ibw_art_name

   ,ibw_art_name_tl

   ,season_cd
   ,pack_type_cd
   ,assort_section_id
   ,catman_head_id
   ,catmanager_id
   ,sales_sub_domain_id
   ,pcg_main_cat_id
   ,pcg_cat_id
   ,pcg_sub_cat_id
   ,sales_domain_id
   ,fnf_cd
   ,demand_field_id
   ,demand_field_domain_id
   ,pcg_main_cat_desc
   ,pcg_main_cat_desc_tl
   ,pcg_cat_desc
   ,pcg_cat_desc_tl
   ,pcg_sub_cat_desc
   ,pcg_sub_cat_desc_tl
   ,demand_field_desc
   ,demand_field_domain_desc

   ,demand_field_domain_desc_tl

   ,log_store_id
   ,log_dl_id
   ,log_profit_center_id
   ,log_store_desc
   ,log_dl_desc
   ,log_store_sdesc
   ,ibw_store_week_sell_fc
   ,ibw_store_sell_fc
   ,week_sell_fc
   ,extraction_id
 )
 AS 
 SELECT
    a.client_cd
   ,a.art_no
   ,f.var_tu_key
   ,f.tunit_no
   ,f.buy_tunit_no
   ,dtd.date_of_day
   ,dtd.week_id
   ,dtd.month_id
   ,(dtd.month_id * 1000000) + dtd.week_id
   ,f.suppl_no
   ,f.suppl_no
   ,f.suppl_prod_no
   ,f.suppl_prod_no
   ,f.gm_origin_cd
   ,f.log_reserv_type_cd

   ,f.import_ind

   ,f.log_stock_qty_pc
   ,f.log_stock_qty_kg
   ,f.log_stock_qty_colli
   ,f.log_stock_buy_qty_colli
   ,f.log_stock_val_nnbp   
   ,f.log_stock_val_invbp   
   ,f.log_stock_val_nfp    

   ,f.log_last_recpt_date
   ,f.log_last_sale_date

   ,a.art_name 
   ,a.art_name_tl
   ,f.var_no
   ,a.stratbuy_domain_id
   ,a.mge_main_cat_id
   ,a.mge_cat_id
   ,a.mge_sub_cat_id
   ,a.mge_main_cat_desc 
   ,a.mge_main_cat_desc_tl
   ,a.mge_cat_desc 
   ,a.mge_cat_desc_tl
   ,a.mge_sub_cat_desc 
   ,a.mge_sub_cat_desc_tl
   ,a.stratbuy_domain_desc
   ,a.stratbuy_domain_desc_tl
   ,a.mikg_art_no
   ,a.art_name
   ,a.art_name_tl
   ,concat(coalesce(a.var_type_desc, ''), ', ', coalesce(a.var_size_desc, ''), ', ', coalesce(cast(a.tunit_qty as string), ''))
   ,concat(coalesce(a.var_type_desc_tl, ''), ', ', coalesce(a.var_size_desc_tl, ''), ', ', coalesce(cast(a.tunit_qty as string), ''))	   
   
   ,a.ibw_art_no
   ,a.art_name 

   ,a.art_name_tl

   ,a.season_cd
   ,COALESCE(a.pack_type_cd, '')
   ,a.assort_section_id
   ,a.catman_head_id
   ,a.catmanager_id
   ,a.sales_sub_domain_id
   ,a.pcg_main_cat_id
   ,a.pcg_cat_id
   ,a.pcg_sub_cat_id
   ,a.sales_domain_id
   ,a.fnf_cd
   ,a.demand_field_id
   ,a.demand_field_domain_id
   ,a.pcg_main_cat_desc 
   ,a.pcg_main_cat_desc_tl
   ,a.pcg_cat_desc 
   ,a.pcg_cat_desc_tl
   ,a.pcg_sub_cat_desc
   ,a.pcg_sub_cat_desc_tl
   ,a.demand_field_desc
   ,a.demand_field_domain_desc

   ,a.demand_field_domain_desc_tl

   ,co_s.log_store_id
   ,co_s.log_dl_id
   ,CAST(-1 as NUMERIC)
   ,co_s.log_store_desc
   ,CAST(NULL as STRING)
   ,CAST(NULL as STRING)
   ,dtd.week_id*10000 + co_s.log_store_id
   ,co_s.log_store_id
   ,dtd.week_id
   ,f.extraction_id

 FROM `metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_stock_per` f


INNER JOIN  (
       select  SDay.log_company_hier_id,
               SDay.client_cd,
               -- cast ((case when MinDay.minday  is not null then minday.minday else (CAST( ' 1901-01-01' as  DATE)) end) as date format 'YYYY-MM-DD')  as date_from,
			   CASE WHEN minday.minday IS NOT NULL THEN minday.minday ELSE CAST(' 1901-01-01' as DATE) END AS date_from,
               SDay.last_day as date_to
      from
             ( select  log_company_hier_id
                       ,client_cd
                       ,max(date_of_day) last_day
                from `metro-bi-dl-srb-prod.cc_dwh.dw_log_gm_stock`
                where gm_origin_cd = 10
                group by 1,2
              ) SDay
     CROSS JOIN
     ( select client_cd, min(date_of_day) as minday
        from `metro-bi-dl-srb-prod.cc_dwh.dw_time_day`
        group by 1
     ) MinDay

 ) TF
 ON f.log_company_hier_id=TF.log_company_hier_id
 AND f.client_cd = TF.client_cd
 INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_art_var_tu` a
 ON  f.art_no = a.art_no
 AND f.var_tu_key = a.var_tu_key


INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_log_co_store` co_s
ON co_s.log_company_hier_id = f.log_company_hier_id

 INNER JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_time_day` dtd
 ON dtd.date_of_day BETWEEN f.date_from AND f.date_to
 AND dtd.date_of_day BETWEEN tf.date_from and tf.date_to
 AND dtd.date_of_day <= TF.date_to

 WHERE dtd.date_of_day BETWEEN DATE(EXTRACT(YEAR FROM current_date())-4 , 1, 1) AND current_date()	
 ;

Query is running:   0%|          |

""


Delete view DW_V_QLIK_LOG_GM_STOCK

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_LOG_GM_STOCK`

Query is running:   0%|          |

""


# Advertising

Create view DW_V_ADVERTISING

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_ADVERTISING
(
CLIENT_CD
,ADVERT_ID
,ADVERT_DESC
,PROM_TYPE_CD
,PROM_DL_ID
,PROM_ID
,PROM_YEAR_ID
,PROM_SEQUENCE_ID
,PROM_START_DATE
,PROM_END_DATE
,MIN_PROM_START_DATE
,MIN_PROM_END_DATE
,MAX_PROM_START_DATE
,MAX_PROM_END_DATE
,PROM_PROCESS_CD
,PROM_START_WEEK_ID
,PROM_END_WEEK_ID
,PROM_CHANNEL_CD
,PROM_END_MONTH_ID
,COMMENT_FOR_ADV
,COMMENT_FOR_ADV_TL
)
AS 
SELECT
a.client_cd
,a.advert_id
,a.advert_desc
,a.prom_type_cd
,a.dl_id
,a.prom_id
,a.year_id
,a.sequence_id
,a.prom_start_date
,a.prom_end_date
,a.prom_start_date
,a.prom_end_date
,a.prom_start_date
,a.prom_end_date
,a.prom_process_cd
,a.prom_start_week_id
,a.prom_end_week_id
,a.prom_channel_cd
,b.month_id
,a.COMMENT_FOR_ADV
,a.COMMENT_FOR_ADV_TL
FROM `metro-bi-dl-srb-prod.cc_dwh.dw_advertising` a
CROSS JOIN `metro-bi-dl-srb-prod.cc_dwh.dw_time_day` b
WHERE a.prom_end_date = b.date_of_day
;

Query is running:   0%|          |

""


Delete view DW_V_ADVERTISING

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_ADVERTISING`

Query is running:   0%|          |

""


# Sales

Create view GM_SALES

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE OR REPLACE VIEW metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_SALES
(
company_hier_id,
art_no,
var_tu_key,
suppl_no,
suppl_prod_no,
gm_cond_type_cd,
gm_origin_cd,
prom_id,
advert_id,
dispo_tool_cd,
date_of_day,
private_label_cd,
tbp_validity_cd,
booking_date,
sell_qty_pc,
sell_qty_kg,
sell_qty_colli,
sell_val_nnbp,
sell_val_invbp,
sell_val_nsp,
sell_val_gsp,
sell_val_tbp,
week_id,
month_id,
extraction_id,
dwh_insert_id,
conversion_rate_bc,
client_cd
)
AS
SELECT  
f.company_hier_id,
f.art_no,
f.var_tu_key,
f.suppl_no,
f.suppl_prod_no,
f.gm_cond_type_cd,
f.gm_origin_cd,
f.prom_id,
f.advert_id,
f.dispo_tool_cd,
f.date_of_day,
f.private_label_cd,
f.tbp_validity_cd,
f.booking_date,
f.sell_qty_pc,
f.sell_qty_kg,
f.sell_qty_colli,
f.sell_val_nnbp,
f.sell_val_invbp,
f.sell_val_nsp,
f.sell_val_gsp,
f.sell_val_tbp,
f.week_id,
f.month_id,
f.extraction_id,
f.dwh_insert_id,
f.conversion_rate_bc,
f.client_cd
FROM `metro-bi-dl-srb-prod.cc_dwh.dw_gm_sales` f
WHERE f.date_of_day BETWEEN DATE(EXTRACT(YEAR FROM current_date())-4 , 1, 1) AND current_date()	
;

Delate view GM_SALES

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_QLIK_GM_SALES`

Create view LOG_GM_SALES

In [ ]:
%%bigquery --project cf-michelange-dev-sx


Delate view LOG_GM_SALES

In [ ]:
%%bigquery --project cf-michelange-dev-sx
drop view `metro-bi-wb-srb-im-s00.srb_qv_s00.DW_V_ADVERTISING`

# 04 SAP data

# 00 Lookup tables

Articles
%%bigquery --project metro-bi-wb-srb-im-s00
-- create or replace view srb_qv_s00.qv_v_lu_article as
-- create or replace view srb_qv_s00.qv_v_lu_customer as

In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- create or replace view srb_qv_s00.qv_v_lu_article as
SELECT
    a.mge_sub_cat_id AS mge_sub_cat_id,
    a.mge_main_cat_id AS mge_main_cat_id,
    a.mge_cat_id AS mge_cat_id,
    a.enter_date AS art_enter_date,
    a.private_label_cd AS art_private_label_cd,
    a.art_no AS art_no,
    avt.mikg_art_no AS mikg_art_no,
    avt.var_tu_key AS var_tu_key,
    avt.var_no AS var_no,
    avt.tunit_no AS tunit_no,
    avt.mge_incl_adj_sales_ind AS mge_incl_adj_sales_ind,
    avt.brand_name AS brand_name,
    avt.private_label_cd AS private_label_cd,
    avt.concessionnaire_cd AS concessionnaire_cd,
    avt.lead_art_no AS lead_art_no,
    avt.lead_var_tu_key AS lead_var_tu_key,
    avt.clcm_brand_category_cd AS clcm_brand_category_cd,
    avt.sub_brand_name AS sub_brand_name,
    avt.pack_size_cd AS pack_size_cd,
    artt.tunit_buy_ind AS tunit_buy_ind,
    artt.tunit_sell_ind AS tunit_sell_ind,
    artt.del_date AS del_date,
    artt.tunit_width AS tunit_width,
    artt.tunit_gross_weight AS tunit_gross_weight,
    artt.tunit_height AS tunit_height,
    artt.tunit_depth AS tunit_depth,
    artt.tunit_volume AS tunit_volume,
    artt.stratbuy_domain_id AS stratbuy_domain_id,
    artv.sort_order_var AS sort_order_var,
    artv.var_size_desc AS var_size_desc,
    artv.var_size_desc_tl AS var_size_desc_tl,
    artv.var_type_desc AS var_type_desc,
    artv.var_type_desc_tl AS var_type_desc_tl,
    artv.var_type_cd AS var_type_cd,
    artv.net_weight AS net_weight,
    artv.net_weight_unit_cd AS net_weight_unit_cd,
    artv.base_size_price_labelling AS base_size_price_labelling,
    artv.measure_unit_cd AS measure_unit_cd,
    artv.distribution_cd AS distribution_cd,
    artv.price_category_cd AS price_category_cd,
    artv.colour_cd AS colour_cd,
    artv.taste_flavour_cd AS taste_flavour_cd,
    artv.material_cd AS material_cd,
    a.art_name_tl AS art_name_tl,
    a.art_name AS art_name,
    a.season_cd AS season_cd,
    p.pack_type_cd AS pack_type_cd,
    p.pack_type_desc AS pack_type_desc,
    artt.tunit_qty AS tunit_qty,
    a.vat_cd AS vat_cd,
    a.catman_head_id AS catman_head_id,
    a.catmanager_id AS catmanager_id,
    a.pcg_main_cat_id AS pcg_main_cat_id,
    a.pcg_cat_id AS pcg_cat_id,
    a.pcg_sub_cat_id AS pcg_sub_cat_id,
    a.mge_main_cat_desc AS mge_main_cat_desc,
    a.mge_cat_desc AS mge_cat_desc,
    a.mge_sub_cat_desc AS mge_sub_cat_desc,
    a.is_private_label_ind AS art_is_private_label_ind,
    CASE
      WHEN a.fnf_cd IN(
        'F'
      ) THEN 'Food'
      ELSE 'Non_Food'
    END AS fnf_cd,
    CASE
      WHEN a.fnf_cd IN(
        'F'
      ) THEN 1
      ELSE 2
    END AS fnf_cd_no,
    CASE
      WHEN a.demand_field_domain_id IN(
        611, 612, 613, 614, 617
      ) THEN '1_Food_Fresh'
      WHEN a.demand_field_domain_id IN(
        615, 616, 618, 619
      ) THEN '1_Food_Ultra_Fresh'
      WHEN a.demand_field_domain_id IN(
        651, 652, 653, 655, 656, 657, 662
      ) THEN '2_Food_Dry'
      WHEN a.demand_field_domain_id IN(
        678, 658, 659
      ) THEN '2_Near_Food'
      WHEN a.demand_field_domain_id > 400
       AND a.demand_field_domain_id < 500
       OR a.demand_field_domain_id IN(
        12
      ) THEN '3_Non_Food'
      ELSE 'Other'
    END AS sales_division,
    CASE
      WHEN a.demand_field_domain_id IN(
        611, 612, 613, 614, 617
      ) THEN CAST(1 as STRING)
      WHEN a.demand_field_domain_id IN(
        615, 616, 618, 619
      ) THEN CAST(2 as STRING)
      WHEN a.demand_field_domain_id IN(
        651, 652, 653, 655, 656, 657, 662
      ) THEN CAST(3 as STRING)
      WHEN a.demand_field_domain_id IN(
        678, 658, 659
      ) THEN CAST(4 as STRING)
      WHEN a.demand_field_domain_id > 400
       AND a.demand_field_domain_id < 500
       OR a.demand_field_domain_id IN(
        12
      ) THEN CAST(5 as STRING)
      ELSE 'Other'
    END AS sales_division_no,
    CASE
      WHEN a.demand_field_domain_id > 610
       AND a.demand_field_domain_id < 620 THEN '1_Food_Fresh+Ultra_Fresh'
      WHEN a.demand_field_domain_id IN(
        651, 652, 653, 655, 656, 657, 662, 678, 658, 659
      ) THEN '2_Food_Dry+Near_Food'
      WHEN a.demand_field_domain_id > 400
       AND a.demand_field_domain_id < 500
       OR a.demand_field_domain_id IN(
        12
      ) THEN '3_Non_Food'
      ELSE 'Other'
    END AS sales_division_2,

    CASE
      WHEN a.demand_field_domain_id > 610
       AND a.demand_field_domain_id < 620 THEN CAST(1 as STRING)
      WHEN a.demand_field_domain_id IN(
        651, 652, 653, 655, 656, 657, 662, 678, 658, 659
      ) THEN CAST(2 as STRING)
      WHEN a.demand_field_domain_id > 400
       AND a.demand_field_domain_id < 500
       OR a.demand_field_domain_id IN(
        12
      ) THEN CAST(3 as STRING)
      ELSE 'Other'
    END AS sales_division_no_2,

    CASE
      WHEN a.demand_field_domain_id = 611 THEN '611_BAKERY'
      WHEN a.demand_field_domain_id = 612 THEN '612_DAIRY'
      WHEN a.demand_field_domain_id = 613 THEN '613_DELICATESSEN'
      WHEN a.demand_field_domain_id = 614 THEN '614_DEEP FROZEN'
      WHEN a.demand_field_domain_id = 615 THEN '615_MEAT'
      WHEN a.demand_field_domain_id = 616 THEN '616_FRESH FISH'
      WHEN a.demand_field_domain_id = 617 THEN '617_PROCESSED MEAT'
      WHEN a.demand_field_domain_id = 618 THEN '618_FRUITS/VEGETABLES'
      WHEN a.demand_field_domain_id = 619 THEN '619_FLOWERS/PLANTS'
      WHEN a.demand_field_domain_id = 651 THEN '651_CANNED GOODS'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN '652_EDIBLE GROCERY'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN '662_COMMODITIES'
      WHEN a.demand_field_domain_id = 653 THEN '653_SWEET/CONFECTIONERY'
      WHEN a.demand_field_domain_id = 654 THEN '654_TOBACCO'
      WHEN a.demand_field_domain_id = 655 THEN '655_SPIRITS'
      WHEN a.demand_field_domain_id = 656 THEN '656_WINES'
      WHEN a.demand_field_domain_id = 657 THEN '657_BEER/SOFT DRINKS'
      WHEN a.demand_field_domain_id = 658 THEN '658_DETERGENTS'
      WHEN a.demand_field_domain_id = 659 THEN '659_TOILETRIES'
      WHEN a.demand_field_domain_id = 678 THEN '678_PROFESSIONAL CLEANING & HYGIENE'
      WHEN a.demand_field_domain_id = 661 THEN '661_EMPTIES'
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406
      ) THEN '10_KITCHEN, COOKING & TABLE'
      WHEN a.demand_field_domain_id IN(
        421, 422, 423, 424, 425, 426, 427, 428
      ) THEN '20_SEASONAL'
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449
      ) THEN '30_BUSINESS & HOME CARE'
      WHEN a.demand_field_domain_id IN(
        461, 462, 463, 464, 465
      ) THEN '40_OFFICE & MEDIA'
      WHEN a.demand_field_domain_id IN(
        481, 482, 483, 484, 485, 486, 487, 488
      ) THEN '50_APPAREL'
      WHEN a.demand_field_domain_id IN(
        12
      ) THEN '12_PRELIMINARY GROUP NONFOOD'
      ELSE '00_OTHER'
    END AS world,
    
    CASE
      WHEN a.demand_field_domain_id = 611 THEN 'BAKERY'
      WHEN a.demand_field_domain_id = 612 THEN 'DAIRY'
      WHEN a.demand_field_domain_id = 613 THEN 'DELICATESSEN'
      WHEN a.demand_field_domain_id = 614 THEN 'DEEP FROZEN'
      WHEN a.demand_field_domain_id = 615 THEN 'MEAT'
      WHEN a.demand_field_domain_id = 616 THEN 'FRESH FISH'
      WHEN a.demand_field_domain_id = 617 THEN 'PROCESSED MEAT'
      WHEN a.demand_field_domain_id = 618 THEN 'FRUITS/VEGETABLES'
      WHEN a.demand_field_domain_id = 619 THEN 'FLOWERS/PLANTS'
      WHEN a.demand_field_domain_id = 651 THEN 'CANNED GOODS'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 'EDIBLE GROCERY'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 'COMMODITIES'
      WHEN a.demand_field_domain_id = 653 THEN 'SWEET/CONFECTIONERY'
      WHEN a.demand_field_domain_id = 654 THEN 'TOBACCO'
      WHEN a.demand_field_domain_id = 655 THEN 'SPIRITS'
      WHEN a.demand_field_domain_id = 656 THEN 'WINES'
      WHEN a.demand_field_domain_id = 657 THEN 'BEER/SOFT DRINKS'
      WHEN a.demand_field_domain_id = 658 THEN 'DETERGENTS'
      WHEN a.demand_field_domain_id = 659 THEN 'TOILETRIES'
      WHEN a.demand_field_domain_id = 661 THEN 'EMPTIES'
      WHEN a.demand_field_domain_id = 678 THEN 'PROFESSIONAL CLEANING & HYGIENE'
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406
      ) THEN 'KITCHEN, COOKING & TABLE'
      WHEN a.demand_field_domain_id IN(
        421, 422, 423, 424, 425, 426, 427, 428
      ) THEN 'SEASONAL'
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449
      ) THEN 'BUSINESS & HOME CARE'
      WHEN a.demand_field_domain_id IN(
        461, 462, 463, 464, 465
      ) THEN 'OFFICE & MEDIA'
      WHEN a.demand_field_domain_id IN(
        481, 482, 483, 484, 485, 486, 487, 488
      ) THEN 'APPAREL'
      WHEN a.demand_field_domain_id IN(
        12
      ) THEN 'PRELIMINARY GROUP NONFOOD'
      ELSE 'OTHER'
    END AS worlds,
    CASE
      WHEN a.demand_field_domain_id = 611 THEN '611'
      WHEN a.demand_field_domain_id = 612 THEN '612'
      WHEN a.demand_field_domain_id = 613 THEN '613'
      WHEN a.demand_field_domain_id = 614 THEN '614'
      WHEN a.demand_field_domain_id = 615 THEN '615'
      WHEN a.demand_field_domain_id = 616 THEN '616'
      WHEN a.demand_field_domain_id = 617 THEN '617'
      WHEN a.demand_field_domain_id = 618 THEN '618'
      WHEN a.demand_field_domain_id = 619 THEN '619'
      WHEN a.demand_field_domain_id = 651 THEN '651'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN '652'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN '662'
      WHEN a.demand_field_domain_id = 653 THEN '653'
      WHEN a.demand_field_domain_id = 654 THEN '654'
      WHEN a.demand_field_domain_id = 655 THEN '655'
      WHEN a.demand_field_domain_id = 656 THEN '656'
      WHEN a.demand_field_domain_id = 657 THEN '657'
      WHEN a.demand_field_domain_id = 658 THEN '658'
      WHEN a.demand_field_domain_id = 659 THEN '659'
      WHEN a.demand_field_domain_id = 661 THEN '661'
      WHEN a.demand_field_domain_id = 678 THEN '678'
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406
      ) THEN '10'
      WHEN a.demand_field_domain_id IN(
        421, 422, 423, 424, 425, 426, 427, 428
      ) THEN '20'
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449
      ) THEN '30'
      WHEN a.demand_field_domain_id IN(
        461, 462, 463, 464, 465
      ) THEN '40'
      WHEN a.demand_field_domain_id IN(
        481, 482, 483, 484, 485, 486, 487, 488
      ) THEN '50'
      WHEN a.demand_field_domain_id IN(
        12
      ) THEN '12'
      ELSE '00'
    END AS worlds_id,
    CASE
      WHEN a.demand_field_domain_id = 611 THEN 'FD111'
      WHEN a.demand_field_domain_id = 612 THEN 'FD112'
      WHEN a.demand_field_domain_id = 613 THEN 'FD113'
      WHEN a.demand_field_domain_id = 614 THEN 'FD114'
      WHEN a.demand_field_domain_id = 615 THEN 'FD115'
      WHEN a.demand_field_domain_id = 616 THEN 'FD116'
      WHEN a.demand_field_domain_id = 617 THEN 'FD117'
      WHEN a.demand_field_domain_id = 618 THEN 'FD118'
      WHEN a.demand_field_domain_id = 619 THEN 'FD119'
      WHEN a.demand_field_domain_id = 651 THEN 'FD151'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 'FD152'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 'FD162'
      WHEN a.demand_field_domain_id = 653 THEN 'FD153'
      WHEN a.demand_field_domain_id = 654 THEN 'FD154'
      WHEN a.demand_field_domain_id = 655 THEN 'FD155'
      WHEN a.demand_field_domain_id = 656 THEN 'FD156'
      WHEN a.demand_field_domain_id = 657 THEN 'FD157'
      WHEN a.demand_field_domain_id = 658 THEN 'FD158'
      WHEN a.demand_field_domain_id = 659 THEN 'FD159'
      WHEN a.demand_field_domain_id = 661 THEN 'FD161'
      WHEN a.demand_field_domain_id = 678 THEN 'FD178'
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406
      ) THEN 'NFD10'
      WHEN a.demand_field_domain_id IN(
        421, 422, 423, 424, 425, 426, 427, 428
      ) THEN 'NFD20'
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449
      ) THEN 'NFD30'
      WHEN a.demand_field_domain_id IN(
        461, 462, 463, 464, 465
      ) THEN 'NFD40'
      WHEN a.demand_field_domain_id IN(
        481, 482, 483, 484, 485, 486, 487, 488
      ) THEN 'NFD50'
      WHEN a.demand_field_domain_id IN(
        12
      ) THEN 'NFD12'
      ELSE 'NFD00'
    END AS buyer_uid,
    CASE
      WHEN a.demand_field_domain_id = 611 THEN 611
      WHEN a.demand_field_domain_id = 612 THEN 612
      WHEN a.demand_field_domain_id = 613 THEN 613
      WHEN a.demand_field_domain_id = 614 THEN 614
      WHEN a.demand_field_domain_id = 615 THEN 615
      WHEN a.demand_field_domain_id = 616 THEN 616
      WHEN a.demand_field_domain_id = 617 THEN 617
      WHEN a.demand_field_domain_id = 618 THEN 618
      WHEN a.demand_field_domain_id = 619 THEN 619
      WHEN a.demand_field_domain_id = 651 THEN 651
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      WHEN a.demand_field_domain_id = 653 THEN 653
      WHEN a.demand_field_domain_id = 654 THEN 654
      WHEN a.demand_field_domain_id = 655 THEN 655
      WHEN a.demand_field_domain_id = 656 THEN 656
      WHEN a.demand_field_domain_id = 657 THEN 657
      WHEN a.demand_field_domain_id = 658 THEN 658
      WHEN a.demand_field_domain_id = 659 THEN 659
      WHEN a.demand_field_domain_id = 661 THEN 661
      WHEN a.demand_field_domain_id = 678 THEN 678
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406
      ) THEN 10
      WHEN a.demand_field_domain_id IN(
        421, 422, 423, 424, 425, 426, 427, 428
      ) THEN 20
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449
      ) THEN 30
      WHEN a.demand_field_domain_id IN(
        461, 462, 463, 464, 465
      ) THEN 40
      WHEN a.demand_field_domain_id IN(
        481, 482, 483, 484, 485, 486, 487, 488
      ) THEN 50
      WHEN a.demand_field_domain_id IN(
        12
      ) THEN 12
      ELSE 0
    END AS worlds_no,
    CASE
      WHEN a.demand_field_domain_id IN(
        611, 612, 613, 614, 617
      ) THEN 'Fresh'
      WHEN a.demand_field_domain_id IN(
        615
      ) THEN 'Meat'
      WHEN a.demand_field_domain_id IN(
        616
      ) THEN 'Fish'
      WHEN a.demand_field_domain_id IN(
        618
      ) THEN 'Fruits/Vegetables'
      WHEN a.demand_field_domain_id IN(
        651, 652, 653, 661, 662
      ) THEN 'Basic_food'
      WHEN a.demand_field_domain_id IN(
        658, 659, 678
      ) THEN 'Detergents'
      WHEN a.demand_field_domain_id IN(
        441, 442, 443, 444, 445, 446, 447, 448, 449, 461, 462, 463, 464, 465
      ) THEN 'NFDI'
      WHEN a.demand_field_domain_id IN(
        401, 402, 403, 404, 405, 406, 421, 422, 423, 424, 425, 426, 427, 428, 481, 482, 483, 484, 485, 486, 487, 488
      ) THEN 'NFDII'
      ELSE 'Other'
    END AS store_department_manager,
    CASE
      WHEN a.demand_field_domain_id IN(
        611, 612, 613, 614, 615, 616, 617, 618, 651, 652, 653, 655, 656, 657, 661, 662
      ) THEN 'FMI'
      ELSE 'FMII'
    END AS floor_manager,
    CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 'Edible_II_Commodities'
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 'Edible_I_Groceries'
      ELSE '0'
    END AS edible_view,
    a.demand_field_domain_id AS demand_field_domain_id,
    a.pcg_main_cat_desc AS pcg_main_cat_desc,
    a.pcg_cat_desc AS pcg_cat_desc,
    a.pcg_sub_cat_desc AS pcg_sub_cat_desc,
    a.demand_field_desc AS demand_field_desc,
    a.demand_field_domain_desc AS demand_field_domain_desc,
    a.catman_buy_domain_desc AS catman_buy_domain_desc,
    concat(substr(concat(CAST(CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      ELSE a.demand_field_domain_id
    END as STRING), '   '), 1, 3), '_', a.demand_field_domain_desc) AS department,
    concat(substr(concat(CAST(CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      ELSE a.demand_field_domain_id
    END as STRING), '   '), 1, 3), '_', a.demand_field_domain_desc) AS department2,
    concat(substr(concat(CAST(CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      ELSE a.demand_field_domain_id
    END as STRING), '   '), 1, 3), '_', a.pcg_main_cat_id, '_', a.pcg_main_cat_desc) AS main_category_local,
    concat(substr(concat(CAST(CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      ELSE a.demand_field_domain_id
    END as STRING), '   '), 1, 3), '_', a.pcg_main_cat_id, '_', trim(CAST(CASE
       a.pcg_cat_id
      WHEN 0 THEN 0
      ELSE CAST(a.pcg_cat_id as INT64)
    END as STRING)), '_', a.pcg_main_cat_desc, '_', a.pcg_cat_desc) AS category_local,
    concat(substr(concat(CAST(CASE
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65237155', '65238010', '65238199', '65238250', '65238310', '65238320', '65238330', '65238340', '65238350', '65238360', '6523845', '65238410', '65238420', '65238415', '65238430', '65238440', '65238450', '65238455', '65238460', '65238465', '65238470', '65238510', '65238530', '65238550', '65238560', '65238610', '65238620', '65238640', '65238650', '65238710', '65238820', '65238830', '65238920', '65238930', '65238940', '65239110', '65239120', '65239210', '65239220', '65239230', '65239240', '65239250', '65239310', '65239320', '65239420', '65239430', '65239450', '65239440', '65239470', '65239460', '65239520', '65239530', '65239610', '65239620', '65239630', '65239640', '65239650', '65239720', '65239750', '65239760', '65239770', '65239810', '65239820', '65239830', '65239840', '65239850', '65239910', '65239920', '65239930', '6524005', '65240010', '65240015', '65240020', '65240025', '65240030', '65240035', '65240040', '65239130', '65238630', '6528544', '65285312', '65239140', '65239510'
      ) THEN 652
      WHEN concat(substr(concat(CAST(CASE
        WHEN a.demand_field_domain_id = 662 THEN 652
        ELSE a.demand_field_domain_id
      END as STRING), '   '), 1, 3), 0, a.pcg_main_cat_id, 0, trim(CAST(CASE
         a.pcg_cat_id
        WHEN 0 THEN 0
        ELSE CAST(a.pcg_cat_id as INT64)
      END as STRING))) IN(
        '65238020', '65238030', '65238040', '65238210', '65238230', '65238240', '65239410', '65239710', '65239730', '65239740', '65239730', '65239740', '65238299', '65239799'
      ) THEN 662
      ELSE a.demand_field_domain_id
    END as STRING), '   '), 1, 3), '_', a.pcg_main_cat_id, '_', trim(CAST(CASE
       a.pcg_cat_id
      WHEN 0 THEN 0
      ELSE CAST(a.pcg_cat_id as INT64)
    END as STRING)), '_', trim(CAST(CASE
       a.mge_sub_cat_id
      WHEN 0 THEN 0
      ELSE CAST(a.mge_sub_cat_id as INT64)
    END as STRING)), '_', a.pcg_main_cat_desc, '_', a.pcg_cat_desc, '_', a.mge_sub_cat_desc) AS sub_category_global,
    clcm.clcm_brand_category_desc AS clcm_brand_category_desc,
    pl.private_label_desc AS private_label_desc,
    pl.private_label_ind AS private_label_ind,
    CASE
      WHEN avt.private_label_cd > 0 THEN 'Yes'
      ELSE 'No'
    END AS own_brand,
    concat(trim(CAST(CASE
       pl.private_label_cd
      WHEN 0 THEN 0
      ELSE CAST(pl.private_label_cd as INT64)
    END as STRING)), '_', pl.private_label_desc) AS own_brand_name
  FROM
    cf-michelange-dev-sx.cc_dwh.dw_art_var_tu AS avt
    LEFT OUTER JOIN cf-michelange-dev-sx.cc_dwh.dw_article AS a ON avt.art_no = a.art_no
    LEFT OUTER JOIN (
      SELECT
          p_0.client_cd AS client_cd,
          p_0.private_label_cd AS private_label_cd,
          p_0.private_label_desc AS private_label_desc,
          p_0.private_label_desc_tl AS private_label_desc_tl,
          p_0.private_label_ind AS private_label_ind
        FROM
          cf-michelange-dev-sx.cc_dwh.dw_rt_private_label AS p_0
    ) AS pl ON a.private_label_cd = pl.private_label_cd
    LEFT OUTER JOIN (
      SELECT
          rtl.rt_numeric_cd AS clcm_brand_category_cd,
          rtl.rt_domain_desc AS clcm_brand_category_desc,
          rtl.client_cd AS client_cd
        FROM
          cf-michelange-dev-sx.cc_dwh.dw_rt_lookup AS rtl
        WHERE upper(rtl.rt_domain_cd) = 'DW_RT_CLCM_BRAND_CATEGORY'
    ) AS clcm ON a.clcm_brand_category_cd = clcm.clcm_brand_category_cd
    LEFT OUTER JOIN cf-michelange-dev-sx.cc_dwh.dw_art_tunit AS artt ON avt.art_no = artt.art_no
     AND avt.tunit_no = artt.tunit_no
    LEFT OUTER JOIN cf-michelange-dev-sx.cc_dwh.dw_art_variation AS artv ON avt.art_no = artv.art_no
     AND avt.var_no = artv.var_no
    LEFT OUTER JOIN cf-michelange-dev-sx.cc_dwh.dw_rt_pack_type AS p ON artt.pack_type_cd = p.pack_type_cd


Query is running:   0%|          |

Downloading:   0%|          |

,mge_sub_cat_id,mge_main_cat_id,mge_cat_id,art_enter_date,art_private_label_cd,art_no,mikg_art_no,var_tu_key,var_no,tunit_no,...,department,department2,main_category_local,category_local,sub_category_global,clcm_brand_category_desc,private_label_desc,private_label_ind,own_brand,own_brand_name
0,10,212,10,2009-01-30,0,7589,484450,1001,1,1,...,560_Posten Computer / Medien,560_Posten Computer / Medien,560_760_POSTEN COMPUTER,560_760_1_POSTEN COMPUTER_POSTEN COMPUTER,560_760_1_10_POSTEN COMPUTER_POSTEN COMPUTER_B...,None,None,None,No,None
1,15,146,20,2004-07-14,0,90460,301414,4001,4,1,...,792_HAUSHALTSKERZEN,792_HAUSHALTSKERZEN,792_360_KERZEN,792_360_10_KERZEN_DUFTKERZEN,None,Default value / undefiniert,None,None,No,None
2,5,704,10,2013-08-07,238,55753,-1,5002,5,2,...,777_SONSTIGES,777_SONSTIGES,777_52_EBAY/B8-Concept,777_52_1_EBAY/B8-Concept_EBAY,777_52_1_5_EBAY/B8-Concept_EBAY_ARBEITS-HOSEN,Default value / undefiniert,FAIRLINE,1.000000000,Yes,238_FAIRLINE
3,40,651,10,2013-09-04,0,60534,678879,1001,1,1,...,795_SB-MOEBEL,795_SB-MOEBEL,795_431_BUEROMOEBEL,795_431_1_BUEROMOEBEL_BUEROMOEBELPROGRAMME,795_431_1_40_BUEROMOEBEL_BUEROMOEBELPROGRAMME_...,Default value / undefiniert,None,None,No,None
4,25,627,10,2020-11-19,0,9258,177272,8002,8,2,...,792_HAUSHALTSKERZEN,792_HAUSHALTSKERZEN,792_360_KERZEN,792_360_1_KERZEN_KERZEN,792_360_1_25_KERZEN_KERZEN_DEKORIERTE STUMPENK...,None,None,None,No,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484333,1,604,1,2009-09-23,18,325645,377106,1001,1,1,...,-1 _undefined / undefiniert,-1 _undefined / undefiniert,-1 _-1_undefined / undefiniert,-1 _-1_-1_undefined / undefiniert_undefined / ...,None,Default value / undefiniert,AUTHENTIC,1.000000000,Yes,18_AUTHENTIC
1484334,1,596,5,2005-08-15,0,845701,345302,13001,13,1,...,-1 _undefined / undefiniert,-1 _undefined / undefiniert,-1 _-1_undefined / undefiniert,-1 _-1_-1_undefined / undefiniert_undefined / ...,None,Default value / undefiniert,None,None,No,None
1484335,11,661,10,2000-02-10,109,899972,-1,13003,13,3,...,-1 _undefined / undefiniert,-1 _undefined / undefiniert,-1 _-1_undefined / undefiniert,-1 _-1_-1_undefined / undefiniert_undefined / ...,None,Default value / undefiniert,None,None,Yes,None
1484336,1,595,1,2008-01-31,0,469735,432468,1001,1,1,...,-1 _undefined / undefiniert,-1 _undefined / undefiniert,-1 _-1_undefined / undefiniert,-1 _-1_-1_undefined / undefiniert_undefined / ...,None,None,None,None,No,None


# New Section

# New Section

Customer

%%bigquery --project metro-bi-wb-srb-im-s00
%%bigquery --project cf-michelange-dev-sx
create or replace view srb_qv_s00.qv_v_lu_customer as

! Some tables are missing from cc_dwh and we hawe them in mdw ?

In [ ]:
%%bigquery --project metro-bi-wb-mag-sci-s00
-- create or replace view srb_qv_s00.qv_v_lu_customer as
SELECT
    a11.spec_vat_no AS spec_vat_no,
    a11.assigned_store_id AS assigned_store_id,
    a11.tax_office_id AS tax_office_id,
    a11.cust_no AS cust_no,
    a11.home_store_id AS home_store_id,
    a11.cust_name AS cust_name,
    a11.date_created AS date_created,
    a11.prom_number AS prom_number,
    a11.date_last_change AS date_last_change,
    a11.prom_status_cd AS prom_status_cd,
    a11.status_cd AS status_cd,
    a11.payment_mode_cd AS payment_mode_cd,
    a11.top_develop_class_cd AS top_develop_class_cd,
    a11.blocking_reason_cd AS blocking_reason_cd,
    a11.acqui_district_no AS acqui_district_no,
    a11.acqui_area_no AS acqui_area_no,
    a11.branch_id AS branch_id,
    -- CASE
    --   WHEN upper(a11.cust_promoter_id) = '5' THEN 0
    --   WHEN upper(a11.cust_promoter_id) = '4' THEN 0
    --   WHEN upper(a11.cust_promoter_id) = 'TS5' THEN 0
    --   ELSE a11.cust_promoter_id
    -- END AS cust_promoter_id,
    a11.num_authorizations AS num_authorizations,
    a11.metro_card_type_cd AS metro_card_type_cd,
    a11.full_credit_ind AS full_credit_ind,
    a11.credit_limit AS credit_limit,
    a11.credit_limit_exhaustion AS credit_limit_exhaustion,
    st.store_desc AS home_store_desc,
    a11.catchment_unit_id AS catchment_unit_id,
    a11.drive_time AS drive_time,
    a11.key_account_ind AS key_account_ind,
    a11.home_store_id * 100000000 + a11.cust_no AS cuconc,
    concat(substr(concat(CAST(a11.home_store_id * 100000000 + a11.cust_no as STRING), repeat(' ', 10)), 1, 10), '_', a11.cust_name) AS cu_name,
    concat(a11.commercial_name_1, '_', a11.cust_name) AS seq_cu_name,
    concat(a11.branch_id, '_', a12.branch_desc) AS branch,
    concat(a12.branch_family_id, '_', a13.branch_family_desc) AS branch_family,
    a13.branch_main_group_id AS branch_main_group_id,
    a13.cust_assort_section_id AS cust_assort_section_id,
    a14.branch_main_group_desc AS branch_main_group_desc,
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id = 5 THEN 'Services'
      WHEN a13.cust_assort_section_id = 7 THEN 'Companies & offices'
      ELSE '0'
    END AS cust_assort_section_desc,
    CASE
      WHEN cast(a11.branch_id as int64) IN (
        101, 104, 105, 109, 112, 115, 200, 136, 137, 203, 140, 301, 302, 303, 304, 305, 306, 400, 401, 316, 317, 318, 402
      ) THEN 1
      ELSE 0
    END AS ctg_flag,
    CASE
      WHEN cast(a11.branch_id as int64) IN(
        140
      ) THEN 'Accommodations with restaurant'
      WHEN cast(a11.branch_id as int64) IN(
        101, 104, 105, 109, 112, 115, 200
      ) THEN 'Restaurants'
      WHEN cast(a11.branch_id as int64) IN(
        301, 302, 303, 304, 305, 306, 400, 401
      ) THEN 'Generalists Food'
      WHEN cast(a11.branch_id as int64) IN(
        136, 137, 203
      ) THEN 'Fast Food'
      WHEN cast(a11.branch_id as int64) IN(
        316, 317, 318, 402
      ) THEN 'Specialists Food'
      ELSE '0'
    END AS ctg_groups,
    CASE
      WHEN a13.cust_assort_section_id > 4 THEN 5
      ELSE a13.cust_assort_section_id
    END AS hts,
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
        5, 7
      ) THEN 'SCO'
      ELSE 'SCO'
    END AS cust_assort_section_desc_calc,
    concat(
    CASE
      WHEN a12.cust_assort_section_id > 4 THEN 5
      ELSE a13.cust_assort_section_id
    END, '_', 
    CASE
      WHEN a12.cust_assort_section_id > 4 THEN 'SCO'
      ELSE 
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
       5, 7
       ) THEN 'SCO'
      ELSE 'SCO'
      END
    END) AS customer_main_segment_lvl1,
    concat(a13.cust_assort_section_id, '_', CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
        5, 7
      ) THEN 'SCO'
      ELSE 'SCO'
    END, '_', a13.branch_main_group_id, '_', a14.branch_main_group_desc) AS branch_main_group_lvl2,
    concat(a13.cust_assort_section_id, '_', a13.branch_main_group_id, '_', a12.branch_family_id, '_', a13.branch_family_desc) AS branch_family_lvl3,
    concat(a13.cust_assort_section_id, '_', a13.branch_main_group_id, '_', a12.branch_id, '_', a12.branch_desc) AS branch_lvl4,
    concat(CAST(a11.home_store_id * 100000000 + a11.cust_no as STRING), '_', a11.cust_name) AS customer,
    concat(a11.home_store_id, '_', a11.acqui_area_no) AS acqui_hs_area,
    CASE
      WHEN cast(a11.branch_id as int64) = 830
       AND a11.cust_no <> 10 THEN 1
      ELSE 0
    END AS kmp_flag
  FROM `metro-bi-dl-ger-prod.cc_dwh.dw_customer` AS a11
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_cust_branch` AS a12 ON a11.branch_id = a12.branch_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_cust_comment` AS cc ON a11.cust_no = cc.cust_no AND a11.home_store_id = cc.home_store_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_co_store_ha` AS st ON a11.home_store_id = st.store_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_cust_branch_family` AS a13 ON a12.branch_family_id = a13.branch_family_id AND a12.cust_assort_section_id = a13.cust_assort_section_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_cust_branch_main_group` AS a14 ON a13.branch_main_group_id = a14.branch_main_group_id AND a13.cust_assort_section_id = a14.cust_assort_section_id


Executing query with job ID: c02028d2-cff6-4042-a1af-1d6135dc9d43
Query executing: 1.02s


ERROR:
 404 Not found: Table metro-bi-dl-ger-prod:cc_dwh.dw_cust_branch_family was not found in location EU

Location: EU
Job ID: c02028d2-cff6-4042-a1af-1d6135dc9d43



In [ ]:
%%bigquery --project metro-bi-wb-mag-sci-s00
-- create or replace view srb_qv_s00.qv_v_lu_customer as
SELECT
    a11.spec_vat_no AS spec_vat_no,
    a11.assigned_store_id AS assigned_store_id,
    a11.tax_office_id AS tax_office_id,
    a11.cust_no AS cust_no,
    a11.home_store_id AS home_store_id,
    a11.cust_name AS cust_name,
    a11.date_created AS date_created,
    a11.prom_number AS prom_number,
    a11.date_last_change AS date_last_change,
    a11.prom_status_cd AS prom_status_cd,
    a11.status_cd AS status_cd,
    a11.payment_mode_cd AS payment_mode_cd,
    a11.top_develop_class_cd AS top_develop_class_cd,
    a11.blocking_reason_cd AS blocking_reason_cd,
    a11.acqui_district_no AS acqui_district_no,
    a11.acqui_area_no AS acqui_area_no,
    a11.branch_id AS branch_id,
    -- CASE
    --   WHEN upper(a11.cust_promoter_id) = '5' THEN 0
    --   WHEN upper(a11.cust_promoter_id) = '4' THEN 0
    --   WHEN upper(a11.cust_promoter_id) = 'TS5' THEN 0
    --   ELSE a11.cust_promoter_id
    -- END AS cust_promoter_id,
    a11.num_authorizations AS num_authorizations,
    a11.metro_card_type_cd AS metro_card_type_cd,
    a11.full_credit_ind AS full_credit_ind,
    a11.credit_limit AS credit_limit,
    a11.credit_limit_exhaustion AS credit_limit_exhaustion,
    st.store_desc AS home_store_desc,
    a11.catchment_unit_id AS catchment_unit_id,
    a11.drive_time AS drive_time,
    a11.key_account_ind AS key_account_ind,
    a11.home_store_id * 100000000 + a11.cust_no AS cuconc,
    concat(substr(concat(CAST(a11.home_store_id * 100000000 + a11.cust_no as STRING), repeat(' ', 10)), 1, 10), '_', a11.cust_name) AS cu_name,
    concat(a11.commercial_name_1, '_', a11.cust_name) AS seq_cu_name,
    concat(a11.branch_id, '_', a12.branch_desc) AS branch,
    concat(a12.branch_family_id, '_', a13.branch_family_desc) AS branch_family,
    a13.branch_main_group_id AS branch_main_group_id,
    a13.cust_assort_section_id AS cust_assort_section_id,
    a14.branch_main_group_desc AS branch_main_group_desc,
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id = 5 THEN 'Services'
      WHEN a13.cust_assort_section_id = 7 THEN 'Companies & offices'
      ELSE '0'
    END AS cust_assort_section_desc,
    CASE
      WHEN cast(a11.branch_id as int64) IN (
        101, 104, 105, 109, 112, 115, 200, 136, 137, 203, 140, 301, 302, 303, 304, 305, 306, 400, 401, 316, 317, 318, 402
      ) THEN 1
      ELSE 0
    END AS ctg_flag,
    CASE
      WHEN cast(a11.branch_id as int64) IN(
        140
      ) THEN 'Accommodations with restaurant'
      WHEN cast(a11.branch_id as int64) IN(
        101, 104, 105, 109, 112, 115, 200
      ) THEN 'Restaurants'
      WHEN cast(a11.branch_id as int64) IN(
        301, 302, 303, 304, 305, 306, 400, 401
      ) THEN 'Generalists Food'
      WHEN cast(a11.branch_id as int64) IN(
        136, 137, 203
      ) THEN 'Fast Food'
      WHEN cast(a11.branch_id as int64) IN(
        316, 317, 318, 402
      ) THEN 'Specialists Food'
      ELSE '0'
    END AS ctg_groups,
    CASE
      WHEN a13.cust_assort_section_id > 4 THEN 5
      ELSE a13.cust_assort_section_id
    END AS hts,
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
        5, 7
      ) THEN 'SCO'
      ELSE 'SCO'
    END AS cust_assort_section_desc_calc,
    concat(
    CASE
      WHEN a12.cust_assort_section_id > 4 THEN 5
      ELSE a13.cust_assort_section_id
    END, '_', 
    CASE
      WHEN a12.cust_assort_section_id > 4 THEN 'SCO'
      ELSE 
    CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
       5, 7
       ) THEN 'SCO'
      ELSE 'SCO'
      END
    END) AS customer_main_segment_lvl1,
    concat(a13.cust_assort_section_id, '_', CASE
      WHEN a13.cust_assort_section_id = 1 THEN 'HoReCa'
      WHEN a13.cust_assort_section_id = 3 THEN 'Trader'
      WHEN a13.cust_assort_section_id IN(
        5, 7
      ) THEN 'SCO'
      ELSE 'SCO'
    END, '_', a13.branch_main_group_id, '_', a14.branch_main_group_desc) AS branch_main_group_lvl2,
    concat(a13.cust_assort_section_id, '_', a13.branch_main_group_id, '_', a12.branch_family_id, '_', a13.branch_family_desc) AS branch_family_lvl3,
    concat(a13.cust_assort_section_id, '_', a13.branch_main_group_id, '_', a12.branch_id, '_', a12.branch_desc) AS branch_lvl4,
    concat(CAST(a11.home_store_id * 100000000 + a11.cust_no as STRING), '_', a11.cust_name) AS customer,
    concat(a11.home_store_id, '_', a11.acqui_area_no) AS acqui_hs_area,
    CASE
      WHEN cast(a11.branch_id as int64) = 830
       AND a11.cust_no <> 10 THEN 1
      ELSE 0
    END AS kmp_flag
  FROM `metro-bi-dl-ger-prod.mdw.dw_customer` AS a11
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.mdw.dw_cust_branch` AS a12 ON a11.branch_id = a12.branch_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.mdw.dw_cust_comment` AS cc ON a11.cust_no = cc.cust_no AND a11.home_store_id = cc.home_store_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` AS st ON a11.home_store_id = st.store_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.mdw.dw_cust_branch_family` AS a13 ON a12.branch_family_id = a13.branch_family_id AND a12.cust_assort_section_id = a13.cust_assort_section_id
    LEFT OUTER JOIN `metro-bi-dl-ger-prod.mdw.dw_cust_branch_main_group` AS a14 ON a13.branch_main_group_id = a14.branch_main_group_id AND a13.cust_assort_section_id = a14.cust_assort_section_id


Executing query with job ID: 8a280f1b-42f0-4d1b-b495-088306f5408b
Query executing: 2.96s


ERROR:
 400 Bad int64 value: L32

Location: EU
Job ID: 8a280f1b-42f0-4d1b-b495-088306f5408b



# Selection of BQ codes

In database **cf-michelange-dev-sx** I have collection of sqls for Google BigQuery which I want to optimise 

Query returns number of cores executed on average on a daily basis

billing_cost - saved views

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  FORMAT_DATETIME('%Y%m%d', DATETIME(usage_end_time)) AS date_ymdh, 
  AVG(CAST(system_labels.value AS INT64)) AS avg_core_usage
FROM
  `bqutil.billing.billing_dashboard_export`,
  UNNEST(system_labels) AS system_labels
WHERE 
  service.id = '6F81-5844-456A'
  AND system_labels.key = 'compute.googleapis.com/cores'
GROUP BY
  date_ymdh
ORDER BY
  date_ymdh ASC
LIMIT 100
;

Query is running:   0%|          |

Downloading:   0%|          |

,date_ymdh,avg_core_usage
0,20180917,2.830176
1,20180918,2.819104
2,20180919,2.758823
3,20180920,2.713549
4,20180921,2.755280
...,...,...
95,20190415,2.668718
96,20190416,2.634182
97,20190417,2.617882
98,20190418,2.628584


Query returns compute cost including discounts on a daily basis

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  FORMAT_DATETIME('%Y%m%d', DATETIME(usage_end_time)) AS date_ymd, 
  ROUND(SUM(cost - credits.amount) * 100, 2) / 100 AS actual_cost
FROM
  `bqutil.billing.billing_dashboard_export`,
  UNNEST(credits) AS credits
WHERE service.id = '6F81-5844-456A'
GROUP BY
  date_ymd
ORDER BY
  date_ymd ASC
  ;

Query is running:   0%|          |

Downloading:   0%|          |

,date_ymd,actual_cost
0,20171011,100.1266
1,20171012,292.7361
2,20171013,294.8027
3,20171014,276.3318
4,20171015,254.9228
...,...,...
497,20190902,116.4204
498,20190903,113.1534
499,20190904,122.5137
500,20190905,113.7012


Costs of running SQL: 
SELECT date_of_day FROM `metro-bi-dl-ger-prod.cc_dwh.dw_cust_invoice_line` where date_of_day> '2021-12-22' LIMIT 100 ;
Find System_ info around target database

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  FORMAT_DATETIME('%Y%m%d', DATETIME(usage_end_time)) AS date_ymdh, 
  AVG(CAST(system_labels.value AS INT64)) AS avg_core_usage
FROM
  `metro-bi-dl-ger-prod.cc_dwh.dw_cust_invoice_line`,
  UNNEST(system_labels) AS system_labels
/*
WHERE 
  service.id = '6F81-5844-456A'
  AND system_labels.key = 'compute.googleapis.com/cores'
*/
GROUP BY
  date_ymdh
ORDER BY
  date_ymdh ASC
LIMIT 100
;


Executing query with job ID: 57f8e404-4fa1-410d-88df-02bb2e61280f
Query executing: 0.83s


ERROR:
 400 Unrecognized name: system_labels at [6:10]

Location: EU
Job ID: 57f8e404-4fa1-410d-88df-02bb2e61280f



In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-dl-ger-prod.cc_dwh_views.DW_V_ARTICLE` LIMIT 1000 ;

Target database

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT date_of_day FROM `metro-bi-dl-ger-prod.cc_dwh.dw_cust_invoice_line` where date_of_day> '2021-12-22' LIMIT 100 ;


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-dl-ger-prod.cc_dwh_views.DW_V_ARTICLE` LIMIT 1000 ;

Query returns compute cost including discounts on a daily basis 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  FORMAT_DATETIME('%Y%m%d', DATETIME(usage_end_time)) AS date_ymd, 
  ROUND(SUM(cost - credits.amount) * 100, 2) / 100 AS actual_cost
FROM
  `bqutil.billing.billing_dashboard_export`,
  UNNEST(credits) AS credits
WHERE service.id = '6F81-5844-456A'
GROUP BY
  date_ymd
ORDER BY
  date_ymd ASC
  ;

Query returns compute usage hours by sku and component on a daily basis

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  FORMAT_DATETIME('%Y%m%d', DATETIME(usage_end_time)) AS date_ymd, 
  sku.id AS sku_id,
  sku.description AS component,
  SUM(usage.amount) AS component_usage,
  usage.unit AS usage_unit,
  ROUND(SUM(cost - credits.amount) * 100, 2) / 100 AS actual_cost
FROM
  `bqutil.billing.billing_dashboard_export`,
  UNNEST(credits) AS credits
WHERE service.id = '6F81-5844-456A' 
AND CAST(DATETIME(usage_start_time, "America/Los_Angeles") AS DATE) >= "2019-01-01"  /* ADDED */
GROUP BY
  date_ymd,
  sku.id,
  sku.description,
  usage.unit
ORDER BY
  date_ymd ASC,
  sku.description ASC
;

/* Query returns monthly costs and credits by project */

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  project.name AS project,
  EXTRACT(MONTH FROM usage_start_time) AS month,
  ROUND(SUM(cost), 2) AS costs,
  ROUND(SUM((SELECT SUM(amount) FROM UNNEST(credits))), 2) AS credits
FROM `bqutil.billing.billing_dashboard_export`
GROUP BY project, month
ORDER by project, month

Query is running:   0%|          |

Downloading:   0%|          |

,project,month,costs,credits
0,None,1,0.00,NaN
1,None,2,5.17,NaN
2,None,3,31.96,NaN
3,None,4,36.57,NaN
4,None,5,180.38,-40.41
...,...,...,...,...
6530,zx0b48VXVlDnD2wHJ+SytQ==,8,399.69,-62.13
6531,zx0b48VXVlDnD2wHJ+SytQ==,9,210.53,-30.55
6532,zx0b48VXVlDnD2wHJ+SytQ==,10,182.92,-39.95
6533,zx0b48VXVlDnD2wHJ+SytQ==,11,199.26,-40.98


/* Query returns monthly total costs and credits */

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  EXTRACT(MONTH FROM export_time) AS month,
  EXTRACT(YEAR FROM export_time) AS year,
  ROUND(SUM(cost), 2) AS costs,
  ROUND(SUM((SELECT SUM(amount) FROM UNNEST(credits))), 2) AS credits
FROM `bqutil.billing.billing_dashboard_export`
GROUP BY year, month
ORDER by year, month

Query returns the invoice total for each month as a sum of regular costs, taxes, adjustments, and rounding errors. 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  invoice.month AS invoice_month,
  SUM(cost) + SUM(IFNULL(credits.amount, 0)) AS total,
  (SUM(CAST(cost * 1000000 AS INT64)) +
    SUM(IFNULL(CAST(credits.amount * 1000000 AS INT64), 0))) / 1000000
    AS total_exact
FROM `bqutil.billing.billing_dashboard_export`
LEFT JOIN UNNEST(credits) AS credits
GROUP BY invoice_month
ORDER BY invoice_month ASC

Query returns the previous day's total costs by project name

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  project.name AS project_name,
  sum(cost) AS costs 
FROM `bqutil.billing.billing_dashboard_export` 
WHERE DATE(export_time) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
GROUP BY project_name
ORDER BY costs DESC

Query is running:   0%|          |

Downloading: |          |

,project_name,costs


 Query returns the previous day's total costs by service 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  service.id AS service_id,
  service.description AS service_description,
  sum(cost) AS costs
FROM `bqutil.billing.billing_dashboard_export` 
WHERE DATE(export_time) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
GROUP BY service_id, service_description
ORDER BY costs DESC
;

Query is running:   0%|          |

Downloading: |          |

,service_id,service_description,costs


Day befor in format 2023-01-22

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) ;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2023-01-22


Query returns the previous month's cost and credits by specified label key
This query uses a label key of 'service' as an example  

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  invoice.month AS invoice_month,
  labels.value AS service,
  labels.key AS key,
  SUM(cost) AS costs,
  ROUND(SUM(credits.amount), 2) AS credits
FROM
`bqutil.billing.billing_dashboard_export`
LEFT JOIN UNNEST(credits) AS credits
LEFT JOIN UNNEST(labels) AS labels
WHERE invoice.month = FORMAT_DATE('%Y%m', DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
AND labels.key = 'service'
-- for all invoice months, uncomment below (and comment above)
-- WHERE invoice.month IS NOT null
GROUP BY
  invoice_month,
  service,
  key
;

Query is running:   0%|          |

Downloading: |          |

,invoice_month,service,key,costs,credits


Query returns the previous month's costs and credits by service 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  invoice.month AS invoice_month,
  service.id AS service_id,
  service.description AS service_description,
  ROUND(SUM(cost), 2) AS costs,
  ROUND(SUM(credits.amount), 2) AS credits 
FROM `bqutil.billing.billing_dashboard_export`
LEFT JOIN UNNEST(credits) AS credits
WHERE invoice.month = FORMAT_DATE('%Y%m', DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
GROUP BY invoice_month, service_id, service_description
ORDER BY costs DESC
;

Query is running:   0%|          |

Downloading: |          |

,invoice_month,service_id,service_description,costs,credits


Month befor current 202212

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT FORMAT_DATE('%Y%m', DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)) ;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,202212


Query returns the previous month's costs and credits by invoice month,
label value, and service
This example uses a label key of 'cost_center' and label value of 'sales'

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  invoice.month AS invoice_month,
  labels.value AS label_value,
  service.description AS description,
  SUM(cost) AS costs,
  ROUND(SUM(credits.amount), 2) AS credits 
FROM `bqutil.billing.billing_dashboard_export`
LEFT JOIN UNNEST(labels) AS labels
LEFT JOIN UNNEST(credits) AS credits
WHERE
  invoice.month = FORMAT_DATE('%Y%m', DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
  AND labels.value = 'sales'
  AND labels.key = 'cost_center'
GROUP BY
  invoice_month,
  description,
  label_value
ORDER BY
  invoice_month,
  costs DESC

Query returns total costs by label values for a specific key
This example uses a label key of 'environment' as an example

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  labels.value AS environment,
  SUM(cost) AS costs
FROM `bqutil.billing.billing_dashboard_export`
LEFT JOIN UNNEST(labels) AS labels
WHERE
  labels.key = 'environment'
  AND labels.value IS NOT NULL 
GROUP BY environment

Query returns total costs by sku and all labels

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  sku.description AS sku_description,
  TO_JSON_STRING(labels) AS labels,
  sum(cost) AS cost
FROM `bqutil.billing.billing_dashboard_export`
GROUP BY sku_description, labels

# Billing Views

In [ ]:
%%bigquery --project cf-michelange-dev-sx
 SELECT * FROM `region-eu.INFORMATION_SCHEMA.JOBS`
 WHERE
   job_type = 'QUERY'
   AND statement_type != 'SCRIPT'
   AND user_email like '%@metro-external.digital%'
   AND creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 8 DAY) AND CURRENT_TIMESTAMP()
   AND end_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) AND CURRENT_TIMESTAMP()
   /*
   AND project_id = ''
   AND project_number = 
   AND job_id = 
   AND priority = ''
   AND start_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) AND CURRENT_TIMESTAMP()
   AND query = ''
   AND state = '' 
   AND reservation_id = ''

   AND total_bytes_processed =      --	INTEGER	                              Total bytes processed by the job.
   AND total_slot_ms = 	           -- INTEGER	                              Slot milliseconds for the job over its entire duration.
   AND error_result = 	             -- RECORD	                              Details of error, if any, as an ErrorProto.
AND cache_hit = 	               -- BOOLEAN	                              Whether the query results of this job were from a cache.
AND destination_table = 	       -- RECORD	                              Destination table for results, if any.
AND referenced_tables = 	       -- RECORD	                              Array of tables referenced by the job. Only populated for query jobs.
AND labels = 	                   -- RECORD	                              Array of labels applied to the job as key, value strings.
AND timeline = 	                 -- RECORD	                              Query timeline of the job. Contains snapshots of query execution.
AND job_stages = 	               -- RECORD	                              Query stages of the job.
AND total_bytes_billed = 	       -- INTEGER	                              If the project is configured to use on-demand pricing, then this field contains the total bytes billed for the job. If the project is configured to use flat-rate pricing, then you are not billed for bytes and this field is informational only.

AND parent_job_id = 	          -- STRING	                              ID of the parent job, if any.
AND transaction_id = 	          -- STRING	                              ID of the transaction in which this job ran, if any. (Preview)
AND session_info = 	            -- RECORD	                              Details about the session in which this job ran, if any. (Preview)
AND dml_statistics = 	          -- RECORD	                              If the job is a query with a DML statement, the value is a record with the following fields:

   */
   LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,creation_time,project_id,project_number,user_email,job_id,job_type,statement_type,priority,start_time,end_time,...,labels,timeline,job_stages,total_bytes_billed,transaction_id,parent_job_id,session_info,dml_statistics,total_modified_partitions,bi_engine_statistics
0,2023-01-23 15:37:50.853000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_393a932d8a69a02b170fa721c8951c19_272,QUERY,SELECT,INTERACTIVE,2023-01-23 15:37:50.868000+00:00,2023-01-23 15:37:50.972000+00:00,...,[],"[{'elapsed_ms': 62, 'total_slot_ms': 5, 'pendi...","[{'name': 'S00: Output', 'id': 0, 'start_ms': ...",0,None,bquxjob_7e9eab6f_185df42c9bb,None,None,<NA>,None
1,2023-01-23 07:28:08.418000+00:00,cf-michelange-dev-sx,878302537998,vathsalya.mohandas@metro-external.digital,bquxjob_19f6c1b9_185dd86f773,QUERY,CREATE_VIEW,INTERACTIVE,2023-01-23 07:28:08.618000+00:00,2023-01-23 07:28:08.768000+00:00,...,[],[],[],0,None,None,None,None,<NA>,None
2,2023-01-23 15:34:36.628000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_90c9ea07047e50dc13eea66b172120f2_92,QUERY,SELECT,INTERACTIVE,2023-01-23 15:34:36.659000+00:00,2023-01-23 15:34:36.781000+00:00,...,[],"[{'elapsed_ms': 67, 'total_slot_ms': 12, 'pend...","[{'name': 'S00: Output', 'id': 0, 'start_ms': ...",0,None,bquxjob_7e9eab6f_185df42c9bb,None,None,<NA>,None
3,2023-01-23 15:39:00.162000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_02288d6f061f73cf85fb614fb638a9c6_351,QUERY,SELECT,INTERACTIVE,2023-01-23 15:39:00.183000+00:00,2023-01-23 15:39:00.247000+00:00,...,[],[],[],0,None,bquxjob_7e9eab6f_185df42c9bb,None,None,<NA>,None
4,2023-01-19 15:59:33.971000+00:00,cf-michelange-dev-sx,878302537998,vaibhav.pal@metro-external.digital,bquxjob_88b68dc_185cac18bd6,QUERY,SELECT,INTERACTIVE,2023-01-19 15:59:34.102000+00:00,2023-01-19 15:59:37.971000+00:00,...,[],"[{'elapsed_ms': 621, 'total_slot_ms': 10517, '...","[{'name': 'S00: Input', 'id': 0, 'start_ms': 1...",20265828352,None,None,None,None,<NA>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-01-23 12:29:30.965000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_2a21ec893184f6b77bd398c5b7ef5436_1294,QUERY,SELECT,INTERACTIVE,2023-01-23 12:29:30.987000+00:00,2023-01-23 12:29:31.104000+00:00,...,[],"[{'elapsed_ms': 63, 'total_slot_ms': 18, 'pend...","[{'name': 'S00: Output', 'id': 0, 'start_ms': ...",0,None,bquxjob_7122bfad_185de87d807,None,None,<NA>,None
96,2023-01-23 12:13:36.504000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_d710f7d5a0f47e7822161c8dc94c04e4_316,QUERY,SELECT,INTERACTIVE,2023-01-23 12:13:36.526000+00:00,2023-01-23 12:13:36.598000+00:00,...,[],[],[],0,None,bquxjob_7122bfad_185de87d807,None,None,<NA>,None
97,2023-01-23 12:37:33.949000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_fefcbb3a786dd848402fdf2259d9c579_1756,QUERY,SELECT,INTERACTIVE,2023-01-23 12:37:33.973000+00:00,2023-01-23 12:37:34.051000+00:00,...,[],[],[],0,None,bquxjob_7122bfad_185de87d807,None,None,<NA>,None
98,2023-01-23 14:05:40.297000+00:00,cf-michelange-dev-sx,878302537998,ramakrishnan.r@metro-external.digital,script_job_277384c1abadf4009edb4b5d843422b2_658,QUERY,CREATE_TABLE,INTERACTIVE,2023-01-23 14:05:40.455000+00:00,2023-01-23 14:05:42.209000+00:00,...,[],[],[],0,None,bquxjob_755ca6e2_185dee8abe9,None,None,<NA>,None


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  SUM(total_slot_ms) / (1000 * 60 * 60 * 24 * 7) AS avg_slots
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS
WHERE
  -- Filter by the partition column first to limit the amount of data scanned.
  -- Eight days allows for jobs created before the 7 day end_time filter.
  creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 8 DAY) AND CURRENT_TIMESTAMP()
  AND job_type = 'QUERY'
  AND end_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) AND CURRENT_TIMESTAMP();

Query is running:   0%|          |

Downloading:   0%|          |

,avg_slots
0,2.098891


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  DISTINCT(user_email) AS user
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS
WHERE
  job_type = 'LOAD';

Query is running:   0%|          |

Downloading:   0%|          |

,user
0,usha.k@metro-external.digital
1,ramakrishnan.r@metro-external.digital
2,atley.psaju@metro-external.digital
3,rohan.tiple@metro-external.digital
4,manchala.vardhan@metro-external.digital
5,metro-dbt-sa@cf-michelange-dev-sx.iam.gservice...
6,metro-iig-sa@cf-michelange-dev-sx.iam.gservice...
7,remmi.jose@metro-external.digital
8,a.pillai@metro-external.digital
9,nagasurekha.kolanti@metro-external.digital


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  user_email,
  SUM(total_bytes_billed) AS bytes_billed
FROM
  `region-us.INFORMATION_SCHEMA.JOBS`
WHERE
  job_type = 'QUERY'
  AND statement_type != 'SCRIPT'
GROUP BY
  user_email;

Query is running:   0%|          |

Downloading:   0%|          |

,user_email,bytes_billed
0,abhijith.kt@metro-external.digital,0
1,metro-dbt-sa@cf-michelange-dev-sx.iam.gservice...,0
2,yogesh.mapari@metro-external.digital,0
3,ramakrishnan.r@metro-external.digital,1220542464
4,khushal.walokar@metro-external.digital,0
5,benith.bj@metro-external.digital,0
6,vitthal.birangal@metro-external.digital,0
7,abhijeet.nene@metro-external.digital,1148190720
8,subramony.r@metro-external.digital,0
9,banala.devayani@metro-external.digital,0


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  TIMESTAMP_TRUNC(end_time, HOUR) AS time_window,
  SUM(total_bytes_billed) AS bytes_billed
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS
WHERE
  job_type = 'QUERY'
  AND statement_type != 'SCRIPT'
GROUP BY
  time_window
ORDER BY
  time_window DESC
;

Query is running:   0%|          |

Downloading:   0%|          |

,time_window,bytes_billed
0,2023-01-23 20:00:00+00:00,45088768
1,2023-01-23 18:00:00+00:00,11885608960
2,2023-01-23 17:00:00+00:00,2811232256
3,2023-01-23 10:00:00+00:00,5047844864
4,2023-01-23 07:00:00+00:00,90177536
...,...,...
424,2022-07-29 12:00:00+00:00,0
425,2022-07-29 05:00:00+00:00,<NA>
426,2022-07-28 16:00:00+00:00,10485760
427,2022-07-28 07:00:00+00:00,0


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  job_id,
  creation_time,
  query
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE
  state != 'DONE';

Query is running:   0%|          |

Downloading: |          |

,job_id,creation_time,query


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
   job_id,
   creation_time,
   state
FROM
   `region-us`.INFORMATION_SCHEMA.JOBS_BY_FOLDER
WHERE
   priority = 'INTERACTIVE';

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
   job_id,
   user_email,
   total_bytes_billed
FROM
   `region-eu`.INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION
WHERE
   EXTRACT(DATE FROM  creation_time) = current_date()
 ORDER BY
   total_bytes_billed DESC
 LIMIT 5;

Executing query with job ID: f79428e0-a020-4868-9fbe-2d488324fc7c
Query executing: 1.38s


ERROR:
 403 Access Denied: Table cf-michelange-dev-sx:region-eu.INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION: User does not have permission to query table cf-michelange-dev-sx:region-eu.INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION, or perhaps it does not exist in location EU.

Location: EU
Job ID: f79428e0-a020-4868-9fbe-2d488324fc7c



-- Query Cost by User Over the Last 30 days

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  user_email,
  ROUND(SUM(total_bytes_billed)/1e12*5, 2) costInDollars
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
  creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
  AND CURRENT_TIMESTAMP()
  AND job_type = "QUERY"
GROUP BY
  1
ORDER BY
  2 DESC

Query is running:   0%|          |

Downloading:   0%|          |

,user_email,costInDollars
0,cf-michelange-dev-sx@appspot.gserviceaccount.com,3.18
1,srdjan.grbic@metro.rs,0.28
2,mitesh.borkar@metro-external.digital,0.05
3,anshika.agarwal@metro-external.digital,0.02
4,metro-dbt-sa@cf-michelange-dev-sx.iam.gservice...,0.01
5,ramakrishnan.r@metro-external.digital,0.01
6,vathsalya.mohandas@metro-external.digital,0.00
7,utkarsh.singh@metro-external.digital,0.00
8,twinkle.thomas@metro-external.digital,0.00
9,manchala.vardhan@metro-external.digital,0.00


-- Queries that ran the Longest

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
   query,
   TIMESTAMP_DIFF(end_time, start_time, SECOND) AS run_time_in_seconds
 FROM
   `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
 WHERE
   creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
   AND CURRENT_TIMESTAMP()
   AND job_type = "QUERY"
 ORDER BY
   run_time DESC
 LIMIT
   5
  ;

Executing query with job ID: df931d95-aa41-4a67-b279-c43b12541ff4
Query executing: 0.88s


ERROR:
 400 Unrecognized name: run_time; Did you mean end_time? at [11:4]

Location: US
Job ID: df931d95-aa41-4a67-b279-c43b12541ff4



In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- Type	        View Reference
-- Datasets
SELECT * FROM INFORMATION_SCHEMA.SCHEMATA ;
SELECT * FROM INFORMATION_SCHEMA.SCHEMATA_OPTIONS ;
-- Jobs
SELECT * FROM INFORMATION_SCHEMA.JOBS_BY_USER ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_BY_FOLDER ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION ;
-- Jobs Timeline	
SELECT * FROM INFORMATION_SCHEMA.JOBS_TIMELINE_BY_USER ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_TIMELINE_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_TIMELINE_BY_FOLDER ;
SELECT * FROM INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION ;
-- Reservations	
SELECT * FROM INFORMATION_SCHEMA.RESERVATION_CHANGES_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.RESERVATIONS_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.CAPACITY_COMMITMENT_CHANGES_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.CAPACITY_COMMITMENTS_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.ASSIGNMENT_CHANGES_BY_PROJECT ;
SELECT * FROM INFORMATION_SCHEMA.ASSIGNMENTS_BY_PROJECT ;
-- Routines	    
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.ROUTINES ;
-- Streaming	    
/*
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.STREAMING_TIMELINE_BY_PROJECT ;
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.STREAMING_TIMELINE_BY_FOLDER ;
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.STREAMING_TIMELINE_BY_ORGANIZATION ;
*/
-- Tables	      
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.TABLES ;
-- Views	        
SELECT * FROM `region-eu`.INFORMATION_SCHEMA.VIEWS ;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
   period_start,
   SUM(period_slot_ms) AS total_slot_ms,
FROM
   `reservation-admin-project.region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE
WHERE
   period_start BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY) AND CURRENT_TIMESTAMP()
GROUP BY
  period_start
ORDER BY
  period_start DESC;

Executing query with job ID: 9305237f-b32a-407e-9725-23e64bd372f1
Query executing: 1.35s


ERROR:
 403 Access Denied: Table reservation-admin-project:region-eu.INFORMATION_SCHEMA.JOBS_TIMELINE: User does not have permission to query table reservation-admin-project:region-eu.INFORMATION_SCHEMA.JOBS_TIMELINE, or perhaps it does not exist in location EU.

Location: EU
Job ID: 9305237f-b32a-407e-9725-23e64bd372f1



# Stored procedure

stored_procedure

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/*
 * Copyright 2021 Google LLC
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

-- Adjust p values using the Benjamini-Hochberg multipletests method, additional details in doi:10.1098/rsta.2009.0127
-- the implementation can be compared with the python function 'statsmodels.stats.multitest.multipletests' (method='fdr_bh')

-- @param STRING pvalue_table_name : the name of the table with the p values that need to be adjusted
-- @param STRING pvalue_column_name : the name of the column with p values.
-- @param INT Nrows : Number of tests (equal to number of rows of the input table)
CREATE OR REPLACE PROCEDURE bqutil.procedure.bh_multiple_tests (pvalue_table_name STRING, pvalue_column_name STRING, n_rows INT64, temp_table_name STRING )
BEGIN
   EXECUTE IMMEDIATE format("""
   CREATE TEMP TABLE %s AS
   WITH padjusted_data AS (
       WITH ranked_data AS (
           SELECT *, ( DENSE_RANK() OVER( ORDER BY %s) ) AS jrank
           FROM %s
   )
       SELECT *,
           MIN( %d * %s / jrank )
           OVER (
               ORDER BY jrank DESC
               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
           ) AS p_adj
       FROM ranked_data
   )
   SELECT * EXCEPT (p_adj, jrank), IF( p_adj > 1.0 , 1.0, p_adj) AS p_adj
   FROM padjusted_data
   ORDER BY jrank""", temp_table_name, pvalue_column_name, pvalue_table_name, n_rows, pvalue_column_name );

   EXECUTE IMMEDIATE format("""SELECT * FROM %s""", temp_table_name);
END;

-- a unit test of bh_multiple_tests
BEGIN

   CREATE TEMP TABLE Pvalues AS
      SELECT 0.001 as pval
      UNION ALL SELECT 0.008
      UNION ALL SELECT 0.039
      UNION ALL SELECT 0.041
      UNION ALL SELECT 0.042
      UNION ALL SELECT 0.06
      UNION ALL SELECT 0.074
      UNION ALL SELECT 0.205;

   CALL bqutil.procedure.bh_multiple_tests('Pvalues','pval',8, 'bh_multiple_tests_results');

   # Table Output
   # pval   p_adj
   # 0.001  0.008
   # 0.008  0.032
   # 0.039  0.06720000000000001
   # 0.041  0.06720000000000001
   # 0.042  0.06720000000000001
   # 0.06  0.08
   # 0.074  0.08457142857142856
   # 0.205  0.205

   ASSERT(
       SELECT COUNTIF(
           (pval = 0.001 AND p_adj = 0.008)
           OR (pval = 0.008 AND p_adj = 0.032)
            OR (pval = 0.039 AND p_adj = 0.06720000000000001)
            OR (pval = 0.041 AND p_adj = 0.06720000000000001)
            OR (pval = 0.042 AND p_adj = 0.06720000000000001)
            OR (pval = 0.06 AND p_adj = 0.08)
            OR (pval = 0.074 AND p_adj = 0.08457142857142856)
            OR (pval = 0.205 AND p_adj = 0.205)
       )
       FROM bh_multiple_tests_results
   ) = 8;

END;

Executing query with job ID: ec39d1a1-2640-4d5a-9c50-7bb96901e462
Query executing: 1.29s


ERROR:
 400 Access Denied: Dataset bqutil:procedure: Permission bigquery.routines.create denied on dataset bqutil:procedure (or it may not exist). at [23:1]

Location: US
Job ID: ec39d1a1-2640-4d5a-9c50-7bb96901e462



In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  dataset_id,
  table_id,
  -- convert bytes to GB
  ROUND(size_bytes/pow(10,9),2) as size_gb,
  row_count,
  -- Convert UNIX EPOCH to a timestamp.
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) as last_modified_time,
  CASE
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
  ELSE NULL
  END AS type
FROM
  -- XXX: replace bigquery-public-data.github_repos with myproject.mydataset
  `bigquery-public-data.github_repos.__TABLES__`
ORDER BY
  size_gb DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,dataset_id,table_id,size_gb,row_count,creation_time,last_modified_time,type
0,github_repos,contents,2682.12,281191977,2016-03-12 04:18:08.643000+00:00,2022-11-27 08:13:56.444000+00:00,table
1,github_repos,commits,910.56,265419190,2016-03-23 04:39:50.914000+00:00,2022-11-26 23:02:46.843000+00:00,table
2,github_repos,files,345.07,2309424945,2016-03-12 03:48:57.708000+00:00,2022-11-26 22:35:08.666000+00:00,table
3,github_repos,sample_contents,25.81,2905870,2016-05-24 18:50:29.492000+00:00,2016-06-28 14:43:11.593000+00:00,table
4,github_repos,sample_files,10.72,72879442,2016-05-24 18:30:30.865000+00:00,2016-06-28 14:40:40.730000+00:00,table
5,github_repos,sample_commits,2.68,672309,2016-06-23 16:11:21.593000+00:00,2016-06-24 00:52:21.519000+00:00,table
6,github_repos,languages,0.20,3325634,2016-03-16 22:43:10.173000+00:00,2022-11-26 22:22:14.046000+00:00,table
7,github_repos,licenses,0.11,3325634,2016-03-13 04:34:04.507000+00:00,2022-11-26 22:22:11.236000+00:00,table
8,github_repos,sample_repos,0.01,400000,2016-05-24 18:28:58.759000+00:00,2016-06-28 14:39:08.481000+00:00,table


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  s.*,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), creation_time, DAY) AS days_live,
  option_value AS dataset_description
FROM
  -- XXX: can prefix INFORMATION_SCHEMA with your '<project_id>.' to select from another project, but defaulting to the current project is more convenient than editing this
  `INFORMATION_SCHEMA.SCHEMATA` AS s
  LEFT JOIN `INFORMATION_SCHEMA.SCHEMATA_OPTIONS` AS so
  USING (schema_name)
-- will miss all the datasets without descriptions like this
--WHERE
--  so.option_name = 'description'
ORDER BY
  last_modified_time DESC
LIMIT 15;

Query is running:   0%|          |

Downloading: |          |

,catalog_name,schema_name,schema_owner,creation_time,last_modified_time,location,ddl,default_collation_name,days_live,dataset_description


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  *
FROM
  -- XXX: replace bigquery-public-data.github_repos with myproject.mydataset
 `bigquery-public-data.github_repos.INFORMATION_SCHEMA.COLUMNS`
WHERE
  is_partitioning_column = 'YES'
    OR
  clustering_ordinal_position IS NOT NULL;

Query is running:   0%|          |

Downloading: |          |

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  table_catalog,
  table_schema,
  table_name,
  column_name,
  ordinal_position,
  is_nullable,
  data_type,
  is_generated,
  generation_expression,
  is_stored,
  is_hidden,
  is_updatable,
  is_system_defined,
  is_partitioning_column,
  clustering_ordinal_position
FROM
  -- XXX: replace bigquery-public-data.github_repos with myproject.mydataset
 `bigquery-public-data.github_repos.INFORMATION_SCHEMA.COLUMNS`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,bigquery-public-data,github_repos,sample_commits,commit,1,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>
1,bigquery-public-data,github_repos,sample_commits,tree,2,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>
2,bigquery-public-data,github_repos,sample_commits,parent,3,NO,ARRAY<STRING>,NEVER,None,None,NO,None,NO,NO,<NA>
3,bigquery-public-data,github_repos,sample_commits,author,4,YES,"STRUCT<name STRING, email STRING, time_sec INT...",NEVER,None,None,NO,None,NO,NO,<NA>
4,bigquery-public-data,github_repos,sample_commits,committer,5,YES,"STRUCT<name STRING, email STRING, time_sec INT...",NEVER,None,None,NO,None,NO,NO,<NA>
5,bigquery-public-data,github_repos,sample_commits,subject,6,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>
6,bigquery-public-data,github_repos,sample_commits,message,7,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>
7,bigquery-public-data,github_repos,sample_commits,trailer,8,NO,"ARRAY<STRUCT<key STRING, value STRING, email S...",NEVER,None,None,NO,None,NO,NO,<NA>
8,bigquery-public-data,github_repos,sample_commits,difference,9,NO,"ARRAY<STRUCT<old_mode INT64, new_mode INT64, o...",NEVER,None,None,NO,None,NO,NO,<NA>
9,bigquery-public-data,github_repos,sample_commits,difference_truncated,10,YES,BOOL,NEVER,None,None,NO,None,NO,NO,<NA>


Lists Top 10 tables from a list of public projects by their row count

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH ALL__TABLES__ AS (
  SELECT * FROM `bigquery-public-data.baseball.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.bls.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.census_bureau_usa.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.cloud_storage_geo_index.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.cms_codes.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.fec.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.genomics_cannabis.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.ghcn_d.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.ghcn_m.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.github_repos.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.hacker_news.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.irs_990.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.medicare.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.new_york.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.nlm_rxnorm.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.noaa_gsod.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.open_images.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.samples.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.san_francisco.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.stackoverflow.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.usa_names.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.utility_us.__TABLES__`
)
SELECT
  project_id,
  dataset_id,
  table_id,
  row_count,
  ROUND(size_bytes/pow(10,9),2) as size_gb,
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) as last_modified_time,
  CASE
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
  ELSE NULL
  END AS type
FROM
  ALL__TABLES__
ORDER BY
  row_count DESC,
  size_gb DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,project_id,dataset_id,table_id,row_count,size_gb,creation_time,last_modified_time,type
0,bigquery-public-data,github_repos,files,2309424945,345.07,2016-03-12 03:48:57.708000+00:00,2022-11-26 22:35:08.666000+00:00,table
1,bigquery-public-data,samples,wikipedia,313797035,38.32,2016-03-14 17:16:47.908000+00:00,2016-03-14 17:16:47.908000+00:00,table
2,bigquery-public-data,github_repos,contents,281191977,2682.12,2016-03-12 04:18:08.643000+00:00,2022-11-27 08:13:56.444000+00:00,table
3,bigquery-public-data,github_repos,commits,265419190,910.56,2016-03-23 04:39:50.914000+00:00,2022-11-26 23:02:46.843000+00:00,table
4,bigquery-public-data,stackoverflow,votes,236452885,7.57,2016-10-26 13:34:58.509000+00:00,2022-11-24 14:24:18.476000+00:00,table
5,bigquery-public-data,fec,individuals_ingest_2020,195489278,36.26,2022-09-28 02:47:51.284000+00:00,2023-01-23 03:46:19.329000+00:00,table
6,bigquery-public-data,fec,indiv20,195482945,36.26,2019-07-12 03:42:19.555000+00:00,2022-10-22 05:19:27.199000+00:00,table
7,bigquery-public-data,new_york,tlc_yellow_trips_2012,178544324,23.66,2016-12-01 20:04:11.914000+00:00,2016-12-01 20:04:11.914000+00:00,table
8,bigquery-public-data,new_york,tlc_yellow_trips_2011,176897199,23.44,2016-12-01 20:20:10.324000+00:00,2016-12-01 20:20:10.324000+00:00,table
9,bigquery-public-data,genomics_cannabis,cs3k_vcf_cs10_dv090,174116980,36.15,2020-02-10 09:43:27.028000+00:00,2020-02-10 09:43:27.028000+00:00,table


-- BigQuery Billing Query
-- products with most billing records

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  COUNT(*) AS billing_records
FROM
  `myproject.mydataset.imported_billing_data`
GROUP BY
  product
ORDER BY
  billing_records DESC;

Executing query with job ID: b0e36806-ab99-4ce7-b982-89080a74be9c
Query executing: 1.06s


ERROR:
 403 Access Denied: Table myproject:mydataset.imported_billing_data: User does not have permission to query table myproject:mydataset.imported_billing_data, or perhaps it does not exist in location US.

Location: US
Job ID: b0e36806-ab99-4ce7-b982-89080a74be9c



BigQuery Billing Query
products with highest aggregate cost

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  ROUND(SUM(cost), 2) AS total_cost
FROM
  `myproject.mydataset.imported_billing_data`
GROUP BY
  product
ORDER BY
  total_cost DESC;

BigQuery Billing Query
most frequently used products costing more than 1 dollar

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  COUNT(*) AS billing_records
FROM
  `myproject.mydataset.imported_billing_data`
WHERE
  cost > 1
GROUP BY
  product
ORDER BY
  billing_records DESC;

BigQuery Billing Query
latest 100 charges

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  resource_type,
  start_time,
  end_time,
  cost,
  project_id,
  project_name,
  project_labels_key,
  currency,
  currency_conversion_rate,
  usage_amount,
  usage_unit
FROM
`bigquery-public-data.new_york.imported_billing_data`
  /*`myproject.mydataset.imported_billing_data`*/
WHERE
  cost > 0
ORDER BY
  end_time DESC
LIMIT 100;

Executing query with job ID: 8d00e5b6-a954-4023-a975-3984f74fb561
Query executing: 0.83s


ERROR:
 404 Not found: Table bigquery-public-data:new_york.imported_billing_data was not found in location US

Location: US
Job ID: 8d00e5b6-a954-4023-a975-3984f74fb561



BigQuery Billing Query
commonly charged units of measure

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  usage_unit,
  COUNT(*) AS billing_records
FROM
  `myproject.mydata.imported_billing_data`
WHERE
  cost > 0
GROUP BY
  usage_unit
ORDER BY
  billing_records DESC;

Create table - external

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE EXTERNAL TABLE cloudtrail_logs (
    eventVersion STRING,
    userIdentity STRUCT<
        type: STRING,
        principalId: STRING,
        arn: STRING,
        accountId: STRING,
        invokedBy: STRING,
        accessKeyId: STRING,
        userName: STRING,
        sessionContext: STRUCT<
            attributes: STRUCT<
                mfaAuthenticated: STRING,
                creationDate: STRING>,
            sessionIssuer: STRUCT<
                type: STRING,
                principalId: STRING,
                arn: STRING,
                accountId: STRING,
                userName: STRING>>>,
    eventTime STRING,
    eventSource STRING,
    eventName STRING,
    awsRegion STRING,
    sourceIpAddress STRING,
    userAgent STRING,
    errorCode STRING,
    errorMessage STRING,
    requestParameters STRING,
    responseElements STRING,
    additionalEventData STRING,
    requestId STRING,
    eventId STRING,
    resources ARRAY<STRUCT<
        arn: STRING,
        accountId: STRING,
        type: STRING>>,
    eventType STRING,
    apiVersion STRING,
    readOnly STRING,
    recipientAccountId STRING,
    serviceEventDetails STRING,
    sharedEventID STRING,
    vpcEndpointId STRING
)
COMMENT 'CloudTrail table for <MY_BUCKET> bucket'
ROW FORMAT SERDE 'com.amazon.emr.hive.serde.CloudTrailSerde'
STORED AS INPUTFORMAT 'com.amazon.emr.cloudtrail.CloudTrailInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://<MY_BUCKET>/AWSLogs/<MY_ACCOUNT_NUMBER>/'
TBLPROPERTIES ('classification'='cloudtrail');

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
   job_id,
   user_email,
   total_bytes_billed
FROM
   `region-en`.INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION
 WHERE
   EXTRACT(DATE FROM  creation_time) = current_date()
 ORDER BY
   total_bytes_billed DESC
LIMIT 5;

Executing query with job ID: 1cf7a2c0-d503-4577-88f6-ded2e0090b07
Query executing: 1.41s


ERROR:
 400 Unrecognized region: en

Location: US
Job ID: 1cf7a2c0-d503-4577-88f6-ded2e0090b07



Query Cost by User Over the Last 30 days

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  user_email,
  ROUND(SUM(total_bytes_billed)/1e12*5, 2) costInDollars
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
  creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
  AND CURRENT_TIMESTAMP()
  AND job_type = "QUERY"
GROUP BY
  1
ORDER BY
  2 DESC
;

Query is running:   0%|          |

Downloading:   0%|          |

,user_email,costInDollars
0,cf-michelange-dev-sx@appspot.gserviceaccount.com,3.18
1,srdjan.grbic@metro.rs,0.28
2,mitesh.borkar@metro-external.digital,0.05
3,anshika.agarwal@metro-external.digital,0.02
4,ramakrishnan.r@metro-external.digital,0.01
5,metro-dbt-sa@cf-michelange-dev-sx.iam.gservice...,0.01
6,twinkle.thomas@metro-external.digital,0.00
7,vathsalya.mohandas@metro-external.digital,0.00
8,mrudula.sonewane@metro-external.digital,0.00
9,manchala.vardhan@metro-external.digital,0.00


Query Cost by User Over the Last 30 days

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  user_email,
  ROUND(SUM(total_bytes_billed)/1e12*5, 2) costInDollars
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
  creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
  AND CURRENT_TIMESTAMP()
  AND job_type = "QUERY"
GROUP BY
  1
ORDER BY
  2 DESC
;

Query is running:   0%|          |

Downloading:   0%|          |

,user_email,costInDollars
0,cf-michelange-dev-sx@appspot.gserviceaccount.com,3.18
1,srdjan.grbic@metro.rs,0.28
2,mitesh.borkar@metro-external.digital,0.05
3,anshika.agarwal@metro-external.digital,0.02
4,metro-dbt-sa@cf-michelange-dev-sx.iam.gservice...,0.01
5,ramakrishnan.r@metro-external.digital,0.01
6,twinkle.thomas@metro-external.digital,0.00
7,vathsalya.mohandas@metro-external.digital,0.00
8,manchala.vardhan@metro-external.digital,0.00
9,mrudula.sonewane@metro-external.digital,0.00


Queries that ran the Longest

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  query,
  TIMESTAMP_DIFF(end_time, start_time, SECOND) AS run_time_in_seconds
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
  creation_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
  AND CURRENT_TIMESTAMP()
  AND job_type = "QUERY"
ORDER BY
   run_time DESC
 LIMIT
   5
;

Executing query with job ID: 0febe37a-7648-4885-bf84-509d1081c6c8
Query executing: 1.30s


ERROR:
 400 Unrecognized name: run_time; Did you mean end_time? at [11:4]

Location: US
Job ID: 0febe37a-7648-4885-bf84-509d1081c6c8



In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  period_start,
  SUM(period_slot_ms) AS total_slot_ms,
FROM
  `reservation-admin-project.region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE
WHERE
   period_start BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY) AND CURRENT_TIMESTAMP()
GROUP BY
  period_start
ORDER BY
  period_start DESC;

Executing query with job ID: 8dd7f6d2-83e4-4e74-be46-5b7125e74ff3
Query executing: 1.89s


ERROR:
 403 Access Denied: Table reservation-admin-project:region-eu.INFORMATION_SCHEMA.JOBS_TIMELINE: User does not have permission to query table reservation-admin-project:region-eu.INFORMATION_SCHEMA.JOBS_TIMELINE, or perhaps it does not exist in location EU.

Location: EU
Job ID: 8dd7f6d2-83e4-4e74-be46-5b7125e74ff3



In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  period_start,
  SUM(period_slot_ms) AS total_slot_ms,
FROM
  `reservation-admin-project.region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE
WHERE
  period_start BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY) AND CURRENT_TIMESTAMP()
GROUP BY
  period_start
ORDER BY
  period_start DESC;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  period_start,
  SUM(IF(state = "PENDING", 1, 0)) as PENDING,
  SUM(IF(state = "RUNNING", 1, 0)) as RUNNING
FROM
  `reservation-admin-project.region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE
WHERE
  period_start BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY) AND CURRENT_TIMESTAMP()
GROUP BY
  period_start;


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  job_id,
  period_slot_ms
FROM
  `reservation-admin-project.region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_PROJECT
WHERE
  period_start = '2020-07-29 03:52:14'
  AND statement_type != 'SCRIPT';

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  period_start,
  SUM(period_slot_ms) AS total_period_slot_ms
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_USER
GROUP BY
  period_start
ORDER BY
  period_start DESC;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  TIMESTAMP_TRUNC(period_start, MINUTE) AS per_start,
  COUNT(DISTINCT job_id) AS unique_jobs
FROM
  `region-eu`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_FOLDER
GROUP BY
  per_start
ORDER BY
  per_start DESC;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  res.period_start,
  SUM(jobs.period_slot_ms) / 1000 / 60 AS period_slot_minutes,
  ANY_VALUE(res.slots_assigned) AS slot_assigned,
  ANY_VALUE(res.slots_max_assigned) AS slots_max_assigned
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION jobs
JOIN
  `region-us`.INFORMATION_SCHEMA.RESERVATIONS_TIMELINE res
  ON
    TIMESTAMP_TRUNC(jobs.period_start, MINUTE) = res.period_start
    AND jobs.reservation_id = res.reservation_id
WHERE
  jobs.job_creation_time
    BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
    AND CURRENT_TIMESTAMP()
  AND res.reservation_id = 'YOUR_RESERVATION_ID'
  AND (jobs.statement_type != "SCRIPT" OR jobs.statement_type IS NULL)  -- Avoid duplicate byte counting in parent and children jobs.
GROUP BY
  period_start
ORDER BY
  period_start DESC;

Executing query with job ID: 50b93637-335e-4e93-910a-f6221afe66a2
Query executing: 0.90s


ERROR:
 403 Access Denied: Table cf-michelange-dev-sx:region-us.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION: User does not have permission to query table cf-michelange-dev-sx:region-us.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION, or perhaps it does not exist in location US.

Location: US
Job ID: 50b93637-335e-4e93-910a-f6221afe66a2



In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  res.period_start,
  res.reservation_id,
  SUM(jobs.period_slot_ms) / 1000 / 60 AS period_slot_minutes,
  ANY_VALUE(res.slots_assigned) AS slots_assigned,
  ANY_VALUE(res.slots_max_assigned) AS slots_max_assigned,
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION jobs
JOIN
  `region-us`.INFORMATION_SCHEMA.RESERVATIONS_TIMELINE res
  ON TIMESTAMP_TRUNC(jobs.period_start, MINUTE) = res.period_start
     AND jobs.reservation_id = res.reservation_id
WHERE
  jobs.job_creation_time
  BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
  AND CURRENT_TIMESTAMP()
  AND (jobs.statement_type != "SCRIPT" OR jobs.statement_type IS NULL)  -- Avoid duplicate byte counting in parent and children jobs.
GROUP BY
  period_start,
  reservation_id
ORDER BY
  period_start DESC,
  reservation_id;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  res.period_start,
  SUM(jobs.period_slot_ms) / 1000 / 60 AS period_slot_minutes,
  ANY_VALUE(res.slots_assigned) AS slot_assigned,
  ANY_VALUE(res.slots_max_assigned) AS slots_max_assigned
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION jobs
JOIN
  `region-us`.INFORMATION_SCHEMA.RESERVATIONS_TIMELINE res
  ON
    TIMESTAMP_TRUNC(jobs.period_start, MINUTE) = res.period_start
    AND jobs.reservation_id = res.reservation_id
WHERE
  jobs.job_creation_time
    BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
    AND CURRENT_TIMESTAMP()
  AND res.reservation_id = 'YOUR_RESERVATION_ID'
  AND (jobs.statement_type != "SCRIPT" OR jobs.statement_type IS NULL)  -- Avoid duplicate byte counting in parent and children jobs.
GROUP BY
  period_start
ORDER BY
  period_start DESC;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  res.period_start,
  res.reservation_id,
  SUM(jobs.period_slot_ms) / 1000 / 60 AS period_slot_minutes,
  ANY_VALUE(res.slots_assigned) AS slots_assigned,
  ANY_VALUE(res.slots_max_assigned) AS slots_max_assigned,
FROM
  `region-us`.INFORMATION_SCHEMA.JOBS_TIMELINE_BY_ORGANIZATION jobs
JOIN
  `region-us`.INFORMATION_SCHEMA.RESERVATIONS_TIMELINE res
  ON TIMESTAMP_TRUNC(jobs.period_start, MINUTE) = res.period_start
     AND jobs.reservation_id = res.reservation_id
WHERE
  jobs.job_creation_time
  BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
  AND CURRENT_TIMESTAMP()
  AND (jobs.statement_type != "SCRIPT" OR jobs.statement_type IS NULL)  -- Avoid duplicate byte counting in parent and children jobs.
GROUP BY
  period_start,
  reservation_id
ORDER BY
  period_start DESC,
  reservation_id;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  DISTINCT(user_email) AS user
FROM
  `region-eu`.INFORMATION_SCHEMA.SESSIONS_BY_PROJECT
WHERE
  is_active = true
  AND creation_time >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY);


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  session_id,
  creation_time
FROM
  `region-us`.INFORMATION_SCHEMA.SESSIONS_BY_USER
WHERE
  creation_time >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
ORDER BY
  creation_time DESC;

Returns metadata for the access control bindings for mydataset.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM myproject.`region-us`.INFORMATION_SCHEMA.OBJECT_PRIVILEGES
WHERE object_name = "mydataset";

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT *
FROM mycompany.`region-us`.INFORMATION_SCHEMA.OBJECT_PRIVILEGES
WHERE object_name = "mydataset";

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT *
FROM mycompany.`region-us`.INFORMATION_SCHEMA.OBJECT_PRIVILEGES
WHERE object_schema = "mydataset" AND object_name = "testdata" ;


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  reservation.reservation_name,
  reservation.slot_capacity
FROM
  `RESERVATION_ADMIN_PROJECT.region-REGION_NAME`.
  INFORMATION_SCHEMA.ASSIGNMENTS_BY_PROJECT assignment
INNER JOIN
  `RESERVATION_ADMIN_PROJECT.region-REGION_NAME`.
  INFORMATION_SCHEMA.RESERVATIONS_BY_PROJECT AS reservation
ON
  (assignment.reservation_name = reservation.reservation_name)
WHERE
   assignment.assignee_id = "PROJECT_ID"
  AND job_type = "QUERY";

# Misc SQLs

Neki komentar


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT DIV(created_data, 1000000000), street_number
FROM `bigquery-public-data.austin_311.311_requests`
where  MOD(street_number, 1000) > 100
LIMIT 10
;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH double_entry_book AS (
   -- debits
   SELECT array_to_sting(input.addresses, ",") as address, inputs.type, -inputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.input` as inputs
   UNION ALL
   -- credits
   SELECT array_to_string(outputs.addresses, ",") as address, outputs.type, outputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.output` as outputs
)
SELECT address, type, sum(value) as balance,  TO_BASE32(address) as byte_address, TO_BASE64(address) as byte_address
FROM double_entry_book
GROUP BY address
ORDER BY balance DESC
LIMIT 1000


In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    select to_address as address, value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)

    union all

    select from_address as address, -value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)

    union all

    select mine as address, sum(float(receipt_gas_used) * float(gas_price)) as value
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.block` as blocks on blocks.number = transactions.block_number
    group by blocks.miner
    union all
    select from_address as address, -(float(receipt_gas_used) * float(gas_price)) as value
    from `bigquery-public-data.crypto_ethereum.transaction`
)
select IFNULL(address, Pow(10, (SELECT max(LENGTH(address)) from double_entry_book))), sum(value) as balance
from double_entry_book
group by address
order by balance desc
limit 1000
;

In [ ]:
%%bigquery --project cf-michelange-dev-sx

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT max(value$)
FROM `bigquery-public-data.crypto_ethereum.transactions`
group bye hash
LIMIT 10;

# User Activity Statistics

# Jobs check

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH data AS (
/*
Based on the idea of
https://mac-blog.org.ua/bigquery-sessions/
*/
SELECT
    creation_time
   ,start_time
   ,end_time
   ,TIMESTAMP_DIFF(start_time,creation_time,MILLISECOND) as sql_prepare_time_millisec
   ,TIMESTAMP_DIFF(end_time,start_time,MILLISECOND) as sql_runtime_millisec
   ,instr(query, "SELECT * FROM (") as start_with_sel_star
   ,user_email
   ,job_id
   ,referenced_tables
   ,ARRAY_LENGTH( REGEXP_EXTRACT_ALL(ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(query, '(\nSELECT.*clmn[1,2].+?FROM \\(\nSELECT \\*)'),''), '(clmn.+?_)') ) num_datastudio_column
   ,ARRAY_TO_STRING ( REGEXP_EXTRACT_ALL(ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(query, '(\nSELECT.*clmn[1,2].+?FROM \\(\nSELECT \\*)'),''), '(clmn.+?_)'),',' ) datastudio_column_list
   ,REGEXP_EXTRACT_ALL(query, '(\n\\) WHERE.+?\n\\) GROUP BY)') where_clause
   ,total_bytes_processed
   ,total_slot_ms
   ,query
   ,cache_hit
   ,round((total_bytes_billed/pow(1024,4)),2) terab_2dec
   ,round((total_bytes_billed/pow(1024,4)),2) * 5 cost_usd
   ,total_slot_ms / TIMESTAMP_DIFF(end_time,start_time,MILLISECOND) as num_slot
 --,a.*
FROM `metro-bi-wb-mag-sci-s00`.`region-EU`.INFORMATION_SCHEMA.JOBS_BY_PROJECT a
WHERE
  DATE(creation_time) BETWEEN DATE(DATE_ADD(CURRENT_DATE(), INTERVAL -2 DAY)) AND DATE(DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY))
  AND job_type = "QUERY"
  AND instr(query, "clmn")>0 
  AND Instr(query, "LIMIT 20000000")>1 
), data_with_previous_timestamp AS (
  SELECT
  -- POI: previous timestamp if any
  LAG(creation_time, 1) OVER (PARTITION BY user_email ORDER BY creation_time) AS previous_creation_time,
  *
  FROM data
), data_with_is_new_session AS (
  SELECT
  -- POI: 5 min window example
  -- Here the basic trick: Also long as the time difference between two SQL execution is less then 5 minutes
  --   they will fall into the same session. 5 minutes can be a long time and must be adapted in every analysis.
  TIMESTAMP_DIFF(creation_time, previous_creation_time, MINUTE),
  CASE WHEN TIMESTAMP_DIFF(creation_time, previous_creation_time, MINUTE) >= 5 OR previous_creation_time IS NULL THEN 1 ELSE 0 END AS is_new_session,
  *
  FROM data_with_previous_timestamp
 ), data_with_sessions AS (
  SELECT
  -- POI: will be 1, 2, 3, 4, 5 - e.g. overall sessions
  SUM(is_new_session) OVER (ORDER BY user_email, creation_time) AS global_session_id,
  -- POI: will be 1, 2, 3 for user 1 and 1, 2 for user 2 - e.g. sessions per user
  SUM(is_new_session) OVER (PARTITION BY user_email ORDER BY creation_time) AS user_session_id,
  *
  FROM data_with_is_new_session
)
select 
DATE(creation_time) date_of_day
,global_session_id
,user_email
,round(sum(cost_usd),2) as session_costs_doller
,min(creation_time) session_start_timestamp
,max(creation_time) session_end_timestamp
,count(job_id) num_jobs
,round(sum(cost_usd)*100/count(job_id),2) as job_costs_cent
from data_with_sessions
group by 1,2,3
order by 4 desc
limit 10
;


Query is running:   0%|          |

Downloading: |          |

,date_of_day,global_session_id,user_email,session_costs_doller,session_start_timestamp,session_end_timestamp,num_jobs,job_costs_cent


In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- use standard sql
-- change country in project metro-bi-dl-cro-prod. It will display only for Croatia prod
select a.databasename, a.tablename, a.bq_insert_date,a.mdw_last_update_timestamp_id
from `metro-bi-dl-cro-prod.mdw.dw_mdw_to_bq_corsscheck_detailed_info` a join  (
select max(cast(concat(replace(cast(bq_insert_date as string),'-','') , SUBSTR(replace(cast(bq_insert_time as string),':',''),0,6)) as INT64) ) max_bq_timestamp ,tablename
from `metro-bi-dl-cro-prod.mdw.dw_mdw_to_bq_corsscheck_detailed_info`
group by tablename) as b on a.tablename=b.tablename
where  cast(concat(replace(cast(bq_insert_date as string),'-','') , SUBSTR(replace(cast(bq_insert_time as string),':',''),0,6)) as INT64) = max_bq_timestamp
and (  date_diff(current_date(), bq_insert_date, day)  >2) -- it wull display tables not updated in the last 2 days
and a.tablename  in ('dw_art_var_tu_suppl_combi_co_c','dw_cust_contact_hist','dw_art_var_tu_co_price_new') -- only for tables in list
group by a.databasename, a.tablename, a.bq_insert_date,a.mdw_last_update_timestamp_id order by  a.tablename
limit 100
;


Change country in project metro-bi-dl-cro-prod. It will display only for Croatia prod

In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- use standard sql
-- change country in project metro-bi-dl-cro-prod. It will display only for Croatia prod
select a.databasename, a.tablename, a.bq_insert_date,a.mdw_last_update_timestamp_id
from `metro-bi-dl-cro-prod.mdw.dw_mdw_to_bq_corsscheck_detailed_info` a join  (
select max(cast(concat(replace(cast(bq_insert_date as string),'-','') , SUBSTR(replace(cast(bq_insert_time as string),':',''),0,6)) as INT64) ) max_bq_timestamp ,tablename
from `metro-bi-dl-cro-prod.mdw.dw_mdw_to_bq_corsscheck_detailed_info`
group by tablename) as b on a.tablename=b.tablename
where  cast(concat(replace(cast(bq_insert_date as string),'-','') , SUBSTR(replace(cast(bq_insert_time as string),':',''),0,6)) as INT64) = max_bq_timestamp
and (  date_diff(current_date(), bq_insert_date, day)  >2) -- it wull display tables not updated in the last 2 days
and a.tablename  in ('dw_art_var_tu_suppl_combi_co_c','dw_cust_contact_hist','dw_art_var_tu_co_price_new') -- only for tables in list
group by a.databasename, a.tablename, a.bq_insert_date,a.mdw_last_update_timestamp_id order by  a.tablename
;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT COUNT(1) FROM
(
SELECT customerid, STRING_AGG(device_type)
FROM `metro-bi-dl-ger-prod.ingest_mapp.customer_status`
WHERE device_type in ("iOS","Android")
AND dana_ingestion_timestamp between "2022-05-01" and "2022-08-31"
GROUP BY customerid
HAVING (STRING_AGG(device_type) LIKE "%iOS%" AND STRING_AGG(device_type) LIKE "%Android%")
)
;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,5356


In [ ]:
%%bigquery --project cf-michelange-dev-sx
select count(1) from (  
  SELECT customerid, count(distinct device_type) 
  FROM `metro-bi-dl-fra-prod.ingest_mapp.customer_status` 
  where device_type like "%Android%" or device_type like "%iOS%" 
  group by 1
  having count(distinct device_type)  = 2
)
;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,28958


# New Section

# sCore Dashboard

score_tdc

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select    

custinvln.client_cd,
dtm.fy_year_id fy_year_id,
custinvln.month_id month_id,

sbdvint.fnf_cd fnf_int_cd, --sbdvint.fnf_desc fnf_int_desc,
sbdmint.stratbuy_division_int_id, --sbdvint.stratbuy_division_int_desc,
mmcsbint.stratbuy_domain_int_id, --sbdmint.stratbuy_domain_int_desc,


art.ART_IS_PRIVATE_LABEL_IND private_label_ind, 

bdmintbr.own_brand_ind bdm_own_brand_ind, 
bdmobcl.ob_main_cluster_cd bdm_ob_main_cluster_cd,  --bdmobmcl.ob_main_cluster_desc bdm_ob_main_cluster_desc,
--bdmobscl.ob_cluster_cd bdm_ob_cluster_cd,  bdmobcl.ob_cluster_desc bdm_ob_cluster_desc,
--bdmintbr.ob_sub_cluster_cd bdm_ob_sub_cluster_cd,  bdmobscl.ob_sub_cluster_desc bdm_ob_sub_cluster_desc,

cust_hts.cust_hts_id unique_hts_id, --cust_hts.cust_hts_desc unique_hts_desc,

--custinvln.fulfillment_model_cd, custfm.fulfillment_model_desc,
custfm.cust_sales_channel_cd, --custsch.cust_sales_channel_cd_desc cust_sales_channel_desc,

custinvln.high_vol_sales_ind,

sum(custinvln.cu_sell_val_nsp)  CILSales
    from    
    `metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_invline` as custinvln
    
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_customer` as customer 
      on	(custinvln.cust_no = customer.cust_no and custinvln.home_store_id = customer.home_store_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_customer_unique` as customer_unique
      on 	(customer.unique_cust_no = customer_unique.unique_cust_no 
            and customer.unique_home_store_id = customer_unique.unique_home_store_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_CUST_BRANCH` as cust_branch
      on 	(customer_unique.unique_branch_id = cust_branch.branch_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_CUST_BRANCH_GROUP` as cust_branch_group
      on 	(cust_branch.branch_group_id = cust_branch_group.branch_group_id)
		left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_branch_family` as cust_branch_family
      on 	(cust_branch.branch_family_id = cust_branch_family.branch_family_id)
		left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_CUST_BRANCH_MAIN_GROUP`	as cust_branch_main_group	
      on 	(cust_branch_family.branch_main_group_id = cust_branch_main_group.branch_main_group_id)
		left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_assort_section` as	cust_assort_section
      on 	(cust_branch.cust_assort_section_id = cust_assort_section.cust_assort_section_id)
		left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_hts` as cust_hts
      on 	(cust_assort_section.cust_hts_id = cust_hts.cust_hts_id)   

    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_ARTICLE` as art
      on (custinvln.art_no = art.art_no)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_CAT_MGE_MAIN_CAT`	as mmc
      on (art.mge_main_cat_id = mmc.mge_main_cat_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_CAT_MMC_STRATBUY_INT` as mmcsbint
      on (art.mge_main_cat_id = mmcsbint.mge_main_cat_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_STRATBUY_DOMAIN_INT` as sbdmint
      on (mmcsbint.stratbuy_domain_int_id = sbdmint.stratbuy_domain_int_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_STRATBUY_DIVISION_INT` as	sbdvint
      on (sbdmint.stratbuy_division_int_id = sbdvint.stratbuy_division_int_id)		
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_RT_PRIVATE_LABEL` as pl
      on (art.ART_PRIVATE_LABEL_CD = pl.private_label_cd)
    
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_bdm_brand_int_ref` as bdmint
      on	(art.brand_id = bdmint.brand_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_bdm_int_brand` as bdmintbr
      on	(bdmint.brand_int_id = bdmintbr.brand_int_id)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_V_BDM_OB_SUB_CLUSTER` as bdmobscl
      on	(bdmintbr.ob_sub_cluster_cd = bdmobscl.ob_sub_cluster_cd)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_V_BDM_OB_CLUSTER` as bdmobcl
      on	(bdmobscl.ob_cluster_cd = bdmobcl.ob_cluster_cd)
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_V_BDM_OB_MAIN_CLUSTER` as bdmobmcl
      on	(bdmobcl.ob_main_cluster_cd = bdmobmcl.ob_main_cluster_cd)
    
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_fulfillment_model`	as custfm
      on	(custinvln.fulfillment_model_cd = custfm.fulfillment_model_cd)
    -- left join	`metro-bi-dl-ger-prod.cc_dwh_views.dw_v_rt_cust_sales_channel` as custsch on	(custfm.cust_sales_channel_cd = custsch.cust_sales_channel_cd)
    
    left join	`metro-bi-dl-ger-prod.cc_dwh_views.DW_V_TIME_MONTH` as	DTM
      on DTM.month_id = custinvln.month_id

where DTM.month_id in(202110)
	
group by    1,2,3,4,5,6,7,8,9,10,11,12
limit 100
;

Query is running:   0%|          |

Downloading:   0%|          |

,client_cd,fy_year_id,month_id,fnf_int_cd,stratbuy_division_int_id,stratbuy_domain_int_id,private_label_ind,bdm_own_brand_ind,bdm_ob_main_cluster_cd,unique_hts_id,cust_sales_channel_cd,high_vol_sales_ind,CILSales
0,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,0,1848638.090000000
1,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,1,2,1,560838.880000000
2,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,3,1,1,997036.670000000
3,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,2,1,1,10333614.120000000
4,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,3,2,0,223591.720000000
5,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,1,3,0,384.040000000
6,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,2,1,0,76226713.980000000
7,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,0,111425704.490000000
8,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,3,2,1,9196.200000000
9,2760,20212022,202110,None,<NA>,<NA>,<NA>,<NA>,<NA>,1,2,0,42273305.710000000


Thorsten_Notes_FSD_Invoice_Lines

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select	
	coalesce(pa11.client_cd, pa12.client_cd)  client_cd,
	coalesce(pa11.month_id, pa12.month_id)  month_id,
	a14.MONTH_DESC  MONTH_DESC,
	pa11.Num_Total_Inv_Lines  Num_Total_Inv_Lines,
	pa12.num_ord_lines_123  num_ord_lines_123,
	pa12.num_inv_lines_5  num_inv_lines_5
from	(
	select	a13.month_id  month_id,
		a12.client_cd  client_cd,
		sum(a11.num_inv_lines_ind)  Num_Total_Inv_Lines
	from	`metro-bi-dl-ger-prod.mdw.dw_cod_fillrate_cluster`	a11
		join	`metro-bi-dl-ger-prod.mdw.dw_co_store_ha`	a12
		  on 	(a11.store_id = a12.store_id)
		join	`metro-bi-dl-ger-prod.mdw.dw_time_day`	a13
		  on 	(a11.date_of_day = a13.date_of_day)
	where	a13.month_id in (202212, 202211, 202210, 202209, 202208, 202207, 202206, 202205, 202204, 202203, 202202, 202201, 202112, 202111, 202110)
	group by	a13.month_id,
		a12.client_cd
	)	pa11
	full outer join	(select	a14.month_id  month_id,
		a12.client_cd  client_cd,
		sum((Case when a13.main_ffm_cluster_desc in ('1 Fulfilled/Substituted', '4 No Delivery', '3 Partial') then a11.num_ord_lines_ind else NULL end))  num_ord_lines_123,
		sum((Case when a13.main_ffm_cluster_desc in ('5 Returned') then a11.num_inv_lines_ind else NULL end))  num_inv_lines_5
	from	`metro-bi-dl-ger-prod.mdw.dw_cod_fillrate_cluster`	a11
		join	`metro-bi-dl-ger-prod.mdw.dw_co_store_ha`	a12
		  on 	(a11.store_id = a12.store_id)
		join	`metro-bi-dl-ger-prod.mdw.dw_cod_cluster_hist`	a13
		  on 	(a11.ffm_cluster_id = a13.ffm_cluster_id)
		join	`metro-bi-dl-ger-prod.mdw.dw_time_day`	a14
		  on 	(a11.date_of_day = a14.date_of_day)
	where	(a14.month_id in (202212, 202211, 202210, 202209, 202208, 202207, 202206, 202205, 202204, 202203, 202202, 202201, 202112, 202111, 202110)
	 and (a13.main_ffm_cluster_desc in ('1 Fulfilled/Substituted', '4 No Delivery', '3 Partial')
	 or a13.main_ffm_cluster_desc in ('5 Returned')))
	group by	a14.month_id,
		a12.client_cd
	)	pa12
	  on 	(pa11.client_cd = pa12.client_cd and 
	pa11.month_id = pa12.month_id)
	join	`metro-bi-dl-ger-prod.mdw.dw_time_month`	a14
	  on 	(coalesce(pa11.month_id, pa12.month_id) = a14.month_id)
;

Stock Availability Md

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT --S.CLIENT_CD,
dtm.month_id,
SUM(S.STOCK_AVAIL_IND) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 100.00 / sum(s.stock_avail_incl_ind)) AS BIGDECIMAL ) AS stock_availability
FROM `metro-bi-dl-ger-prod.mdw.dw_gm_stock_availability` AS S
LEFT JOIN `metro-bi-dl-ger-prod.mdw.dw_article` AS ART
    ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-ger-prod.mdw.dw_cat_mmc_stratbuy_int` AS MCAT
    ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_stratbuy_domain_int` AS DOMINT
    ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
-- Below table is not a view like in TeraData it is an actual table
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` AS ST
    ON S.company_hier_id = ST.company_hier_id
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` AS TM
    ON TM.DATE_OF_DAY = S.DATE_OF_DAY
join `metro-bi-dl-ger-prod.mdw.dw_time_month` AS DTM 
	ON DTM.month_id = tm.month_id 

WHERE S.ASSORTMENT_CLUSTER_CD IN (0,1)
AND S.ART_BLOCK_CD = 0
AND S.STOCK_AVAIL_INCL_IND = 1
AND DTM.month_id >= 202110 and DTM.month_id <= 202209
GROUP BY 1
order by 1;

SELECT S.CLIENT_CD,
DTM.fy_year_id,
DTM.month_id,
SUM(S.STOCK_AVAIL_IND) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 10000 / sum(s.stock_avail_incl_ind)) AS BIGDECIMAL ) / 10000 AS stock_availability_per
FROM `metro-bi-dl-ger-prod.mdw.dw_gm_stock_availability` AS S
LEFT JOIN `metro-bi-dl-ger-prod.mdw.dw_article` AS ART
    ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-ger-prod.mdw.dw_cat_mmc_stratbuy_int` AS MCAT
    ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_stratbuy_domain_int` AS DOMINT
    ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` AS ST
    ON S.company_hier_id = ST.company_hier_id
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` AS TM
    ON TM.DATE_OF_DAY = S.DATE_OF_DAY
join `metro-bi-dl-ger-prod.mdw.dw_time_month` AS DTM 
	ON DTM.month_id = tm.month_id 

WHERE S.ASSORTMENT_CLUSTER_CD IN (0,1)
AND S.ART_BLOCK_CD = 0
AND S.STOCK_AVAIL_INCL_IND = 1
AND DTM.month_id >= 202010 and DTM.month_id <= 202209
GROUP BY 1,2,3;


Stock Availability Pct_TY

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT --S.CLIENT_CD,
dtm.month_id,
SUM(S.STOCK_AVAIL_IND) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 100.00 / sum(s.stock_avail_incl_ind)) AS BIGDECIMAL ) AS stock_availability
FROM `metro-bi-dl-ger-prod.mdw.dw_gm_stock_availability` AS S
LEFT JOIN `metro-bi-dl-ger-prod.mdw.dw_article` AS ART
    ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-ger-prod.mdw.dw_cat_mmc_stratbuy_int` AS MCAT
    ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_stratbuy_domain_int` AS DOMINT
    ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
-- Below table is not like TeraData
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` AS ST
    ON S.company_hier_id = ST.company_hier_id
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` AS TM
    ON TM.DATE_OF_DAY = S.DATE_OF_DAY
join `metro-bi-dl-ger-prod.mdw.dw_time_month` AS DTM 
	ON DTM.month_id = tm.month_id 

WHERE S.ASSORTMENT_CLUSTER_CD IN (0,1)
AND S.ART_BLOCK_CD = 0
AND S.STOCK_AVAIL_INCL_IND = 1
AND DTM.month_id >= 202110 and DTM.month_id <= 202209
GROUP BY 1
order by 1;

Stock Avaliability Per

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT S.CLIENT_CD,
dtm.month_id,
SUM(S.STOCK_AVAIL_IND) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 10000 / sum(s.stock_avail_incl_ind)) AS BIGDECIMAL ) / 10000 AS stock_availability_per
FROM `metro-bi-dl-ger-prod.mdw.dw_gm_stock_availability` AS S
LEFT JOIN `metro-bi-dl-ger-prod.mdw.dw_article` AS ART
    ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-ger-prod.mdw.dw_cat_mmc_stratbuy_int` AS MCAT
    ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_stratbuy_domain_int` AS DOMINT
    ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
-- Below table is not like TeraData
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` AS ST
    ON S.company_hier_id = ST.company_hier_id
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` AS TM
    ON TM.DATE_OF_DAY = S.DATE_OF_DAY
join `metro-bi-dl-ger-prod.mdw.dw_time_month` AS DTM 
	ON DTM.month_id = tm.month_id 

WHERE S.ASSORTMENT_CLUSTER_CD IN (0,1)
AND S.ART_BLOCK_CD = 0
AND S.STOCK_AVAIL_INCL_IND = 1
AND DTM.month_id >= 202010 and DTM.month_id <= 202209
GROUP BY 1,2;


Stock Availability in Percentage Metro Digital

In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- All Countries Union SQL
--Countries Added: ger,srb,fra,cze,esp,bel,bul,cro,hun,ind,ita,kaz,mda,ned,pak,pol,por,rom,rus,svk,tur,ukr
--Countries Pending: aut,jpn :due to some view are not created for these countries
SELECT S.CLIENT_CD,
DTM.fy_year_id,
SUM(S.STOCK_AVAIL_IND ) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 1000 / sum(s.stock_avail_incl_ind)) / 1000 AS BIGDECIMAL ) AS stock_availability_per
FROM `metro-bi-dl-ger-prod.mdw.dw_gm_stock_availability` S
LEFT JOIN `metro-bi-dl-ger-prod.mdw.dw_article` ART
  ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-ger-prod.mdw.dw_cat_mmc_stratbuy_int` MCAT
  ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_stratbuy_domain_int` domint
  ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` ST
  ON S.COMPANY_HIER_ID = ST.COMPANY_HIER_ID
 JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` TM
   ON TM.DATE_OF_DAY = S.DATE_OF_DAY
 JOIN `metro-bi-dl-ger-prod.mdw.dw_time_month` dtm
   ON DTM.month_id = tm.month_id
WHERE S.ASSORTMENT_CLUSTER_CD IN (0,1)
    AND S.ART_BLOCK_CD = 0
    AND S.STOCK_AVAIL_INCL_IND = 1
GROUP BY 1,2

Md_Notes_FSD_Invoice_Lines

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT	pa12.client_cd  client_cd,
	pa12.month_id  month_id,
	a13.MONTH_DESC  MONTH_DESC,
	pa12.num_ord_lines_123 + pa12.num_inv_lines_5  AS FSD_Num_Order_Lines
FROM	(SELECT	a14.month_id  month_id,
		a12.client_cd  client_cd,
		Sum((CASE WHEN a13.main_ffm_cluster_desc IN ('1 Fulfilled/Substituted', '4 No Delivery', '3 Partial') THEN a11.num_ord_lines_ind ELSE NULL end))  num_ord_lines_123,
		Sum((CASE WHEN a13.main_ffm_cluster_desc IN ('5 Returned') THEN a11.num_inv_lines_ind ELSE NULL end))  num_inv_lines_5
	FROM	`metro-bi-dl-ger-prod.mdw.dw_cod_fillrate_cluster`	a11
		JOIN	`metro-bi-dl-ger-prod.mdw.dw_co_store_ha`	a12
		  ON 	(a11.store_id = a12.store_id)
		JOIN	`metro-bi-dl-ger-prod.mdw.dw_cod_cluster`	a13
		  ON 	(a11.ffm_cluster_id = a13.ffm_cluster_id)
		JOIN	`metro-bi-dl-ger-prod.mdw.dw_time_day`	a14
		  ON 	(a11.date_of_day = a14.date_of_day)
	WHERE	(a14.month_id between 202110 and 202209
	 AND (a13.main_ffm_cluster_desc IN ('1 Fulfilled/Substituted', '4 No Delivery', '3 Partial')
	 OR a13.main_ffm_cluster_desc IN ('5 Returned')))
	GROUP BY	a14.month_id,
		a12.client_cd
	)	pa12
	JOIN	`metro-bi-dl-ger-prod.mdw.dw_time_month`	a13
	  ON 	(pa12.month_id = a13.month_id);

FSD Invoice Lines - Ger Only

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  A13.fy_year_id,
  A14.month_id,
  A12.client_cd,
  SUM(A11.Cu_num_invoice_lines) NumInvoiceLines
FROM
  `metro-bi-dl-ger-prod.cc_dwh_views.dw_v_cust_invoice_orig_nn` As A11
JOIN `metro-bi-dl-ger-prod.cc_dwh_views.dw_v_co_store` As A12
  on A11.store_id = A12.store_id
JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_time_day` AS A13
  ON A13.date_of_day = A11.date_of_day
JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_time_month` AS A14 
  ON A14.month_id = A13.month_id
WHERE 
     A11.FULFILLMENT_MODEL_CD IN (2, 5, 6) 
     and A13.month_id >= 202010 
     and A13.month_id <= 202209
GROUP BY 1, 2, 3

FSD Invoice Lines

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  A13.fy_year_id,
  A14.month_id,
  A12.client_cd,
  SUM(A11.num_invoice_lines) NumInvoiceLines
FROM
  `metro-bi-dl-ger-prod.mdw.dw_cust_invoice_orig` A11
JOIN `metro-bi-dl-ger-prod.mdw.dw_co_store_ha` A12
  on A11.store_id = A12.store_id
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_day` AS A13
  ON A13.date_of_day = A11.date_of_day
JOIN `metro-bi-dl-ger-prod.mdw.dw_time_month` AS A14 
  ON A14.month_id = A13.month_id
WHERE 
     A11.FULFILLMENT_MODEL_CD IN (2, 5, 6) 
     and A13.month_id >= 202010 
     and A13.month_id <= 202209
GROUP BY 1, 2, 3

04_Stock Availability

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT DTM.month_id,

SUM(S.STOCK_AVAIL_IND) AS CONSIDERED_LINES_WITH_STOCK,
SUM (S.STOCK_AVAIL_INCL_IND) AS CONSIDERED_LINES,
CAST((sum(s.stock_avail_ind) * 100.00 / sum(s.stock_avail_incl_ind)) AS BIGDECIMAL ) AS stock_availability

FROM `metro-bi-dl-srb-prod.mdw.dw_gm_stock_availability` AS S
LEFT JOIN `metro-bi-dl-srb-prod.mdw.dw_article` AS ART
  ON S.ART_NO = ART.ART_NO
JOIN `metro-bi-dl-srb-prod.mdw.dw_cat_mmc_stratbuy_int` AS MCAT
  ON ART.MGE_MAIN_CAT_ID = MCAT.MGE_MAIN_CAT_ID
JOIN `metro-bi-dl-srb-prod.mdw.dw_stratbuy_domain_int` AS DOMINT
  ON MCAT.STRATBUY_DOMAIN_INT_ID = DOMINT.STRATBUY_DOMAIN_INT_ID
JOIN `metro-bi-dl-srb-prod.mdw.dw_co_store_ha` AS ST
  ON S.company_hier_id = ST.company_hier_id
JOIN `metro-bi-dl-srb-prod.mdw.dw_time_day` AS TM
  ON TM.DATE_OF_DAY = S.DATE_OF_DAY
JOIN `metro-bi-dl-srb-prod.mdw.dw_time_month` AS DTM 
  ON DTM.month_id = tm.month_id
WHERE TM.MONTH_ID between 202110 and 202212 --(202204,202205,202206)
	AND S.ASSORTMENT_CLUSTER_CD IN (0,1)
	AND S.ART_BLOCK_CD = 0
	AND S.STOCK_AVAIL_INCL_IND = 1
GROUP BY 1
ORDER BY 1

In [ ]:
%%bigquery --project cf-michelange-dev-sx


# metro-bi-wb-srb-im-s00

**ARRAYS**

In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


**BQ_Introduction_script**

**GitHub_SQL-script-master**

**Learn GBQ**

**PoC ITA Route** 

**SQL Translator**

**Table sampling**

**bg_dvm_20**

**bigquery-oreilly-book-master**

**bigquery-oreilly-book-srdjan**

**cloud_google_bigquery**

**count_co_sql-resources**

**dana_sqls**

**google_bigquery_create_use_table**

**google_online**

**my_leture_notes**

**start**

**stored_procedure**

In [ ]:
%%bigquery --project cf-michelange-dev-sx
DECLARE columns ARRAY<STRUCT<column_name STRING, date DATE>>;
SET columns = (
  WITH all_date_columns AS (
    SELECT column_name, parse_date('_%m_%d_%y', column_name) AS date
    FROM `bigquery-public-data`.covid19_jhu_csse.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = 'confirmed_cases' AND STARTS_WITH(column_name, '_')
  )
  SELECT ARRAY_AGG(STRUCT(column_name, date) ORDER BY date DESC LIMIT 3) AS columns
  FROM all_date_columns
);
EXECUTE IMMEDIATE format("""
  SELECT 
     country_region, province_state, 
     %s AS cases_day0, '%t' AS date_day0,
     %s AS cases_day1, '%t' AS date_day1,
     %s AS cases_day2, '%t' AS date_day2
  FROM `bigquery-public-data`.covid19_jhu_csse.confirmed_cases
  WHERE country_region LIKE 'Canada'
  ORDER BY cases_day0 DESC
""", 
columns[OFFSET(0)].column_name, columns[OFFSET(0)].date,
columns[OFFSET(1)].column_name, columns[OFFSET(1)].date,
columns[OFFSET(2)].column_name, columns[OFFSET(2)].date
);

Executing query with job ID: 4cfef831-2190-449d-a212-5ef44ea9e92b
Query executing: 3.31s


ERROR:
 400 Query error: Unrecognized name: country_region; Did you mean country_or_region? at [8:9]

Location: US
Job ID: 4cfef831-2190-449d-a212-5ef44ea9e92b



# Transformation views

In [ ]:
-- https://cloud.google.com/bigquery/docs/materialized-views-create
%%bigquery --project cf-michelange-dev-sx
SELECT TIMESTAMP_TRUNC(ts, HOUR) AS ts_hour, COUNT(*) / 10 AS cnt
FROM mydataset.mytable
GROUP BY ts_hour;

SyntaxError: ignored

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH tmp AS (
  SELECT TIMESTAMP_TRUNC(ts, HOUR) AS ts_hour, *
  FROM mydataset.mytable
)
SELECT ts_hour, COUNT(*) AS cnt
FROM tmp
GROUP BY ts_hour

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH tmp AS (
  SELECT city, COUNT(*) AS population
  FROM mydataset.mytable
  GROUP BY city
)
SELECT population, COUNT(*) AS cnt
GROUP BY population;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE TABLE my_project.my_dataset.my_base_table(
  employee_id INT64,
  transaction_time TIMESTAMP)
  PARTITION BY DATE(transaction_time)
  OPTIONS (partition_expiration_days = 2);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE MATERIALIZED VIEW my_project.my_dataset.my_mv_table
  PARTITION BY DATE(transaction_time)
  CLUSTER BY employee_id
AS (
  SELECT
    employee_id,
    transaction_time,
    COUNT(employee_id) AS cnt
  FROM
    my_dataset.my_base_table
  GROUP BY
    employee_id, transaction_time
);


In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE MATERIALIZED VIEW my_project.my_dataset.my_mv_table
  PARTITION BY date
  CLUSTER BY employee_id
AS (
  SELECT
    employee_id,
    _PARTITIONDATE AS date,
    COUNT(1) AS count
  FROM
    my_dataset.my_base_table
  GROUP BY
    employee_id,
    date
);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE TABLE my_project.my_dataset.my_base_table (
  employee_id INT64,
  transaction_time TIMESTAMP)
  PARTITION BY DATE(transaction_time);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE MATERIALIZED VIEW my_project.my_dataset.my_mv_table
  PARTITION BY DATE(transaction_hour)
AS (
  SELECT
    employee_id,
    TIMESTAMP_TRUNC(transaction_time, HOUR) AS transaction_hour,
    COUNT(employee_id) AS cnt
  FROM
    my_dataset.my_base_table
  GROUP BY
    employee_id,
    transaction_hour
);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
CREATE MATERIALIZED VIEW dataset.mv
  CLUSTER BY s_market_id
AS (
  SELECT
    s_market_id,
    s_country,
    SUM(ss_net_paid) AS sum_sales,
    COUNT(*) AS cnt_sales
  FROM dataset.store_sales
  INNER JOIN dataset.store
    ON ss_store_sk = s_store_sk
  GROUP BY s_market_id, s_country
);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  SUM(ss_net_paid)
FROM dataset.store_sales
INNER JOIN dataset.store
ON ss_store_sk = s_store_sk
WHERE s_country = 'Germany';

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  store_id,
  CAST(sold_datetime AS DATE) AS sold_date
  SUM(net_profit) AS sum_profit
FROM dataset.store_sales
WHERE
  CAST(sold_datetime AS DATE) >= '2021-01-01' AND
  promo_id IS NOT NULL
GROUP BY 1, 2;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  store_id,
  CAST(sold_datetime AS DATE) AS sold_date
  SUM(net_profit) AS sum_profit
FROM dataset.store_sales
WHERE
  CAST(sold_datetime AS DATE) >= '2021-01-01' AND
  promo_id IS NOT NULL
GROUP BY 1, 2

In [ ]:
CREATE MATERIALIZED VIEW my_dataset.my_mv AS
SELECT date, customer_id, region, SUM(net_paid) as total_paid
FROM my_dataset.sales
GROUP BY 1, 2, 3;

In [ ]:
CREATE TABLE my_dataset.my_materialized_table AS
SELECT date, customer_id, region, SUM(net_paid) as total_paid
FROM my_dataset.sales
GROUP BY 1, 2, 3;

In [ ]:

  SELECT * FROM my_dataset.my_mv LIMIT 10
    SELECT * FROM my_dataset.my_materialized_table LIMIT 10
      SELECT SUM(total_paid) FROM my_dataset.my_mv WHERE date > '2020-12-01'
        SELECT SUM(total_paid) FROM my_dataset.my_materialized_table WHERE date > '2020-12-01'

In [ ]:
CREATE MATERIALIZED VIEW project-id.my_dataset.my_mv_table
PARTITION BY RANGE_BUCKET(column, buckets)
OPTIONS (enable_refresh = false)
AS SELECT ...


In [ ]:
ALTER MATERIALIZED VIEW project-id.my_dataset.my_mv_table
SET OPTIONS (enable_refresh = true)

In [ ]:
CREATE MATERIALIZED VIEW project-id.my_dataset.my_mv_table
OPTIONS (enable_refresh = true, refresh_interval_minutes = 60)
AS SELECT ...

In [ ]:
ALTER MATERIALIZED VIEW project-id.my_dataset.my_mv_table
SET OPTIONS (refresh_interval_minutes = 60)

In [ ]:
CALL BQ.REFRESH_MATERIALIZED_VIEW('project-id.my_dataset.my_mv_table')

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT job_id, total_slot_ms, total_bytes_processed
FROM region-EU.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE job_id LIKE "%materialized_view_refresh_%"
LIMIT 10
;

Query is running:   0%|          |

Downloading:   0%|          |

,job_id,total_slot_ms,total_bytes_processed
0,materialized_view_refresh_RfW0UBh5u96uzBZV3Jsp...,14535,620889
1,materialized_view_refresh_h66Sjaa2QLVhMPhPCvAt...,13689,1724823
2,materialized_view_refresh_YKVNHPV6WD4oeiZW3Nug...,14392,616869
3,materialized_view_refresh_uwMCSO10KLhx8RIEeyhJ...,11067,621693
4,materialized_view_refresh_P04YnW1DZNwc2DGo8q1I...,5860,5382024
5,materialized_view_refresh_UIGkC8zoBxgMguZiCtd2...,18736,716565
6,materialized_view_refresh_83bL5w60BcWgCKgUsCef...,18770,517173
7,materialized_view_refresh_3TVeUris1EkcNFgzlhZX...,15520,517173
8,materialized_view_refresh_sh7n4AfYiH7tQ3xrlus4...,16945,996
9,materialized_view_refresh_tLihgpdmwLR9id0BwQVg...,13093,26643


In [ ]:
%%bigquery --project cf-michelange-dev-sx


# Arrays

Working with arrays 
In BigQuery, an array is an ordered list consisting of zero or more values of the same data type. You can construct arrays of simple data types, such as INT64, and complex data types, such as STRUCTs. The current exception to this is the ARRAY data type because arrays of arrays are not supported. To learn more about the ARRAY data type, including NULL handling, see Array type.

With BigQuery, you can construct array literals, build arrays from subqueries using the ARRAY function, and aggregate values into an array using the ARRAY_AGG function.

You can combine arrays using functions like ARRAY_CONCAT(), and convert arrays to strings using ARRAY_TO_STRING().

Constructing arrays
Using array literals
You can build an array literal in BigQuery using brackets ([ and ]). Each element in an array is separated by a comma.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT [1, 2, 3] as numbers;

Query is running:   0%|          |

Downloading:   0%|          |

,numbers
0,"[1, 2, 3]"


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT ["apple", "pear", "orange"] as fruit;

Query is running:   0%|          |

Downloading:   0%|          |

,fruit
0,"[apple, pear, orange]"


In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT [true, false, true] as booleans;

-- You can also create arrays from any expressions that have compatible types. For example:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT [a, b, c]
FROM
  (SELECT 5 AS a,
          37 AS b,
          406 AS c);

SELECT [a, b, c]
FROM
  (SELECT CAST(5 AS INT64) AS a,
          CAST(37 AS FLOAT64) AS b,
          406 AS c);

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,"[5.0, 37.0, 406.0]"


-- Notice that the second example contains three expressions: one that returns an INT64, one that returns a FLOAT64, and one that declares a literal. This expression works because all three expressions share FLOAT64 as a supertype.

-- To declare a specific data type for an array, use angle brackets (< and >). For example:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT ARRAY<FLOAT64>[1, 2, 3] as floats;

-- Arrays of most data types, such as INT64 or STRING, don't require that you declare them first.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
-- SELECT [1, 2, 3] as numbers;
-- SELECT ["1", "2", "3"] as numbers;
SELECT ARRAY<STRING>["1", "2", "3"] as numbers;

Query is running:   0%|          |

Downloading:   0%|          |

,numbers
0,"[1, 2, 3]"


Generate numbers in defined range with step.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT GENERATE_ARRAY(11, 33, 2) AS odds;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT GENERATE_ARRAY(21, 14, -1) AS countdown;

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT GENERATE_DATE_ARRAY('2017-11-21', '2017-12-31', INTERVAL 1 WEEK) AS date_array;

Query is running:   0%|          |

Downloading:   0%|          |

,date_array
0,"[2017-11-21, 2017-11-28, 2017-12-05, 2017-12-1..."


Offset - Ordinal

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT [5, 10] AS some_numbers)
SELECT some_numbers,
       some_numbers[OFFSET(1)] AS offset_1,
       some_numbers[ORDINAL(1)] AS ordinal_1
FROM sequences;

Query is running:   0%|          |

Downloading:   0%|          |

,some_numbers,offset_1,ordinal_1
0,"[0, 1, 1, 2, 3, 5]",1,0
1,"[2, 4, 8, 16, 32]",4,2
2,"[5, 10]",10,5


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT [5, 10] AS some_numbers)
SELECT some_numbers,
       ARRAY_LENGTH(some_numbers) AS len
FROM sequences;

Query is running:   0%|          |

Downloading:   0%|          |

,some_numbers,len
0,"[0, 1, 1, 2, 3, 5]",6
1,"[2, 4, 8, 16, 32]",5
2,"[5, 10]",2


-- Converting elements in an array to rows in a table
-- To convert an ARRAY into a set of rows, also known as "flattening," use the UNNEST operator. UNNEST takes an ARRAY and returns a table with a single row for each element in the ARRAY.

-- Because UNNEST destroys the order of the ARRAY elements, you may wish to restore order to the table. To do so, use the optional WITH OFFSET clause to return an additional column with the offset for each array element, then use the ORDER BY clause to order the rows by their offset.

-- Example

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT *
FROM UNNEST(['foo', 'bar', 'baz', 'qux', 'corge', 'garply', 'waldo', 'fred']) AS element
WITH OFFSET AS offset
ORDER BY offset;

Query is running:   0%|          |

Downloading:   0%|          |

,element,offset
0,foo,0
1,bar,1
2,baz,2
3,qux,3
4,corge,4
5,garply,5
6,waldo,6
7,fred,7


-- To flatten an entire column of ARRAYs while preserving the values of the other columns in each row, use a correlated cross join to join the table containing the ARRAY column to the UNNEST output of that ARRAY column.

-- With a correlated join, the UNNEST operator references the ARRAY typed column from each row in the source table, which appears previously in the FROM clause. For each row N in the source table, UNNEST flattens the ARRAY from row N into a set of rows containing the ARRAY elements, and then the cross join joins this new set of rows with the single row N from the source table.

-- Examples

-- The following example uses UNNEST to return a row for each element in the array column. Because of the CROSS JOIN, the id column contains the id values for the row in sequences that contains each number.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT 1 AS id, [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT 2 AS id, [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT 3 AS id, [5, 10] AS some_numbers)
SELECT id, flattened_numbers
FROM sequences
CROSS JOIN UNNEST(sequences.some_numbers) AS flattened_numbers;


Query is running:   0%|          |

Downloading:   0%|          |

,id,flattened_numbers
0,1,0
1,1,1
2,1,1
3,1,2
4,1,3
5,1,5
6,2,2
7,2,4
8,2,8
9,2,16


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT 1 AS id, [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT 2 AS id, [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT 3 AS id, [5, 10] AS some_numbers)
SELECT id, flattened_numbers
FROM sequences, sequences.some_numbers AS flattened_numbers;


-- Querying nested arrays
-- If a table contains an ARRAY of STRUCTs, you can flatten the ARRAY to query the fields of the STRUCT. You can also flatten ARRAY type fields of STRUCT values.

-- Querying STRUCT elements in an ARRAY
-- The following example uses UNNEST with CROSS JOIN to flatten an ARRAY of STRUCTs.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH races AS (
  SELECT "800M" AS race,
    [STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as laps),
     STRUCT("Makhloufi" as name, [24.5, 25.4, 26.6, 26.1] as laps),
     STRUCT("Murphy" as name, [23.9, 26.0, 27.0, 26.0] as laps),
     STRUCT("Bosse" as name, [23.6, 26.2, 26.5, 27.1] as laps),
     STRUCT("Rotich" as name, [24.7, 25.6, 26.9, 26.4] as laps),
     STRUCT("Lewandowski" as name, [25.0, 25.7, 26.3, 27.2] as laps),
     STRUCT("Kipketer" as name, [23.2, 26.1, 27.3, 29.4] as laps),
     STRUCT("Berian" as name, [23.7, 26.1, 27.0, 29.3] as laps)]
       AS participants)
SELECT
  race,
  participant
FROM races r
CROSS JOIN UNNEST(r.participants) as participant;

Query is running:   0%|          |

Downloading:   0%|          |

,race,participant
0,800M,"{'name': 'Rudisha', 'laps': [23.4, 26.3, 26.4,..."
1,800M,"{'name': 'Makhloufi', 'laps': [24.5, 25.4, 26...."
2,800M,"{'name': 'Murphy', 'laps': [23.9, 26.0, 27.0, ..."
3,800M,"{'name': 'Bosse', 'laps': [23.6, 26.2, 26.5, 2..."
4,800M,"{'name': 'Rotich', 'laps': [24.7, 25.6, 26.9, ..."
5,800M,"{'name': 'Lewandowski', 'laps': [25.0, 25.7, 2..."
6,800M,"{'name': 'Kipketer', 'laps': [23.2, 26.1, 27.3..."
7,800M,"{'name': 'Berian', 'laps': [23.7, 26.1, 27.0, ..."


-- You can find specific information from repeated fields. For example, the following query returns the fastest racer in an 800M race.

-- This example does not involve flattening an array, but does represent a common way to get information from a repeated field.

-- Example

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH races AS (
  SELECT "800M" AS race,
    [STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as laps),
     STRUCT("Makhloufi" as name, [24.5, 25.4, 26.6, 26.1] as laps),
     STRUCT("Murphy" as name, [23.9, 26.0, 27.0, 26.0] as laps),
     STRUCT("Bosse" as name, [23.6, 26.2, 26.5, 27.1] as laps),
     STRUCT("Rotich" as name, [24.7, 25.6, 26.9, 26.4] as laps),
     STRUCT("Lewandowski" as name, [25.0, 25.7, 26.3, 27.2] as laps),
     STRUCT("Kipketer" as name, [23.2, 26.1, 27.3, 29.4] as laps),
     STRUCT("Berian" as name, [23.7, 26.1, 27.0, 29.3] as laps)]
       AS participants)
SELECT
  race,
  (SELECT name
   FROM UNNEST(participants)
   ORDER BY (
     SELECT SUM(duration)
     FROM UNNEST(laps) AS duration) ASC
   LIMIT 1) AS fastest_racer
FROM races;

-- Querying ARRAY-type fields in a STRUCT
-- You can also get information from nested repeated fields. For example, the following statement returns the runner who had the fastest lap in an 800M race.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH races AS (
 SELECT "800M" AS race,
   [STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as laps),
    STRUCT("Makhloufi" as name, [24.5, 25.4, 26.6, 26.1] as laps),
    STRUCT("Murphy" as name, [23.9, 26.0, 27.0, 26.0] as laps),
    STRUCT("Bosse" as name, [23.6, 26.2, 26.5, 27.1] as laps),
    STRUCT("Rotich" as name, [24.7, 25.6, 26.9, 26.4] as laps),
    STRUCT("Lewandowski" as name, [25.0, 25.7, 26.3, 27.2] as laps),
    STRUCT("Kipketer" as name, [23.2, 26.1, 27.3, 29.4] as laps),
    STRUCT("Berian" as name, [23.7, 26.1, 27.0, 29.3] as laps)]
    AS participants)
SELECT
race,
(SELECT name
 FROM UNNEST(participants),
   UNNEST(laps) AS duration
 ORDER BY duration ASC LIMIT 1) AS runner_with_fastest_lap
FROM races;

-- Notice that the preceding query uses the comma operator (,) to perform an implicit CROSS JOIN. It is equivalent to the following example, which uses an explicit CROSS JOIN.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH races AS (
 SELECT "800M" AS race,
   [STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as laps),
    STRUCT("Makhloufi" as name, [24.5, 25.4, 26.6, 26.1] as laps),
    STRUCT("Murphy" as name, [23.9, 26.0, 27.0, 26.0] as laps),
    STRUCT("Bosse" as name, [23.6, 26.2, 26.5, 27.1] as laps),
    STRUCT("Rotich" as name, [24.7, 25.6, 26.9, 26.4] as laps),
    STRUCT("Lewandowski" as name, [25.0, 25.7, 26.3, 27.2] as laps),
    STRUCT("Kipketer" as name, [23.2, 26.1, 27.3, 29.4] as laps),
    STRUCT("Berian" as name, [23.7, 26.1, 27.0, 29.3] as laps)]
    AS participants)
SELECT
race,
(SELECT name
 FROM UNNEST(participants)
 CROSS JOIN UNNEST(laps) AS duration
 ORDER BY duration ASC LIMIT 1) AS runner_with_fastest_lap
FROM races;

-- Flattening arrays with a CROSS JOIN excludes rows that have empty or NULL arrays. If you want to include these rows, use a LEFT JOIN.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH races AS (
 SELECT "800M" AS race,
   [STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as laps),
    STRUCT("Makhloufi" as name, [24.5, 25.4, 26.6, 26.1] as laps),
    STRUCT("Murphy" as name, [23.9, 26.0, 27.0, 26.0] as laps),
    STRUCT("Bosse" as name, [23.6, 26.2, 26.5, 27.1] as laps),
    STRUCT("Rotich" as name, [24.7, 25.6, 26.9, 26.4] as laps),
    STRUCT("Lewandowski" as name, [25.0, 25.7, 26.3, 27.2] as laps),
    STRUCT("Kipketer" as name, [23.2, 26.1, 27.3, 29.4] as laps),
    STRUCT("Berian" as name, [23.7, 26.1, 27.0, 29.3] as laps),
    STRUCT("Nathan" as name, ARRAY<FLOAT64>[] as laps),
    STRUCT("David" as name, NULL as laps)]
    AS participants)
SELECT
  name, sum(duration) AS finish_time
FROM races, races.participants LEFT JOIN participants.laps duration
GROUP BY name;

-- Creating arrays from subqueries
-- A common task when working with arrays is turning a subquery result into an array. In BigQuery, you can accomplish this using the ARRAY() function.

-- For example, consider the following operation on the sequences table:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
  UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
  UNION ALL SELECT [5, 10] AS some_numbers)
SELECT some_numbers,
  ARRAY(SELECT x * 2
        FROM UNNEST(some_numbers) AS x) AS doubled
FROM sequences;

-- This example starts with a table named sequences. This table contains a column, some_numbers, of type ARRAY<INT64>.

-- The query itself contains a subquery. This subquery selects each row in the some_numbers column and uses UNNEST to return the array as a set of rows. Next, it multiplies each value by two, and then recombines the rows back into an array using the ARRAY() operator.

-- Filtering arrays
-- The following example uses a WHERE clause in the ARRAY() operator's subquery to filter the returned rows.

-- Note: In the following examples, the resulting rows are not ordered.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT [5, 10] AS some_numbers)
SELECT
  ARRAY(SELECT x * 2
        FROM UNNEST(some_numbers) AS x
        WHERE x < 5) AS doubled_less_than_five
FROM sequences;

-- Notice that the third row contains an empty array, because the elements in the corresponding original row ([5, 10]) did not meet the filter requirement of x < 5.

-- You can also filter arrays by using SELECT DISTINCT to return only unique elements within an array.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers)
SELECT ARRAY(SELECT DISTINCT x
             FROM UNNEST(some_numbers) AS x) AS unique_numbers
FROM sequences;

-- You can also filter rows of arrays by using the IN keyword. This keyword filters rows containing arrays by determining if a specific value matches an element in the array.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT [5, 10] AS some_numbers)
SELECT
   ARRAY(SELECT x
         FROM UNNEST(some_numbers) AS x
         WHERE 2 IN UNNEST(some_numbers)) AS contains_two
FROM sequences;

-- Notice again that the third row contains an empty array, because the array in the corresponding original row ([5, 10]) did not contain 2.

-- Scanning arrays
-- To check if an array contains a specific value, use the IN operator with UNNEST. To check if an array contains a value matching a condition, use the EXISTS operator with UNNEST.

-- Scanning for specific values
-- To scan an array for a specific value, use the IN operator with UNNEST.

-- Example

-- The following example returns true if the array contains the number 2.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT 2 IN UNNEST([0, 1, 1, 2, 3, 5]) AS contains_value;

Query is running:   0%|          |

Downloading:   0%|          |

,contains_value
0,True


-- To return the rows of a table where the array column contains a specific value, filter the results of IN UNNEST using the WHERE clause.
-- Example
-- The following example returns the id value for the rows where the array column contains the value 2.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT 1 AS id, [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT 2 AS id, [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT 3 AS id, [5, 10] AS some_numbers)
SELECT id AS matching_rows
FROM sequences
WHERE 2 IN UNNEST(sequences.some_numbers)
ORDER BY matching_rows;


-- Scanning for values that satisfy a condition
-- To scan an array for values that match a condition, use UNNEST to return a table of the elements in the array, use WHERE to filter the resulting table in a subquery, and use EXISTS to check if the filtered table contains any rows.

-- Example
-- The following example returns the id value for the rows where the array column contains values greater than 5.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
  Sequences AS (
    SELECT 1 AS id, [0, 1, 1, 2, 3, 5] AS some_numbers
    UNION ALL
    SELECT 2 AS id, [2, 4, 8, 16, 32] AS some_numbers
    UNION ALL
    SELECT 3 AS id, [5, 10] AS some_numbers
  )
SELECT id AS matching_rows
FROM Sequences
WHERE EXISTS(SELECT * FROM UNNEST(some_numbers) AS x WHERE x > 5);

Query is running:   0%|          |

Downloading:   0%|          |

,matching_rows
0,2
1,3


-- Scanning for STRUCT field values that satisfy a condition
-- To search an array of STRUCTs for a field whose value matches a condition, use UNNEST to return a table with a column for each STRUCT field, then filter non-matching rows from the table using WHERE EXISTS.

-- Example

-- The following example returns the rows where the array column contains a STRUCT whose field b has a value greater than 3.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
  Sequences AS (
    SELECT 1 AS id, [STRUCT(0 AS a, 1 AS b)] AS some_numbers
    UNION ALL
    SELECT 2 AS id, [STRUCT(2 AS a, 4 AS b)] AS some_numbers
    UNION ALL
    SELECT 3 AS id, [STRUCT(5 AS a, 3 AS b), STRUCT(7 AS a, 4 AS b)] AS some_numbers
  )
SELECT id AS matching_rows
FROM Sequences
WHERE EXISTS(SELECT 1 FROM UNNEST(some_numbers) WHERE b > 3);

Query is running:   0%|          |

Downloading:   0%|          |

,matching_rows
0,2
1,3


-- Arrays and aggregation
-- With BigQuery, you can aggregate values into an array using ARRAY_AGG().

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH fruits AS
  (SELECT "apple" AS fruit
   UNION ALL SELECT "pear" AS fruit
   UNION ALL SELECT "banana" AS fruit)
SELECT ARRAY_AGG(fruit) AS fruit_basket
FROM fruits;

Query is running:   0%|          |

Downloading:   0%|          |

,fruit_basket
0,"[apple, pear, banana]"


-- The array returned by ARRAY_AGG() is in an arbitrary order, since the order in which the function concatenates values is not guaranteed. To order the array elements, use ORDER BY. For example:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH fruits AS
  (SELECT "apple" AS fruit
   UNION ALL SELECT "pear" AS fruit
   UNION ALL SELECT "banana" AS fruit)
SELECT ARRAY_AGG(fruit ORDER BY fruit) AS fruit_basket
FROM fruits;

Query is running:   0%|          |

Downloading:   0%|          |

,fruit_basket
0,"[apple, banana, pear]"


-- You can also apply aggregate functions such as SUM() to the elements in an array. For example, the following query returns the sum of array elements for each row of the sequences table.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH sequences AS
  (SELECT [0, 1, 1, 2, 3, 5] AS some_numbers
   UNION ALL SELECT [2, 4, 8, 16, 32] AS some_numbers
   UNION ALL SELECT [5, 10] AS some_numbers)
SELECT some_numbers,
  (SELECT SUM(x)
   FROM UNNEST(s.some_numbers) x) AS sums
FROM sequences s;

Query is running:   0%|          |

Downloading:   0%|          |

,some_numbers,sums
0,"[0, 1, 1, 2, 3, 5]",12
1,"[2, 4, 8, 16, 32]",62
2,"[5, 10]",15


-- BigQuery also supports an aggregate function, ARRAY_CONCAT_AGG(), which concatenates the elements of an array column across rows.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH aggregate_example AS
  (SELECT [1,2] AS numbers
   UNION ALL SELECT [3,4] AS numbers
   UNION ALL SELECT [5, 6] AS numbers)
SELECT ARRAY_CONCAT_AGG(numbers) AS count_to_six_agg
FROM aggregate_example;

Query is running:   0%|          |

Downloading:   0%|          |

,count_to_six_agg
0,"[1, 2, 3, 4, 5, 6]"


-- Note: The array returned by ARRAY_CONCAT_AGG() is non-deterministic, since the order in which the function concatenates values is not guaranteed.

-- Converting arrays to strings
-- The ARRAY_TO_STRING() function allows you to convert an ARRAY<STRING> to a single STRING value or an ARRAY<BYTES> to a single BYTES value where the resulting value is the ordered concatenation of the array elements.

-- The second argument is the separator that the function will insert between inputs to produce the output; this second argument must be of the same type as the elements of the first argument.

-- Example:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH greetings AS
  (SELECT ["Hello", "World"] AS greeting)
SELECT ARRAY_TO_STRING(greeting, " ") AS greetings
FROM greetings;

Query is running:   0%|          |

Downloading:   0%|          |

,greetings
0,Hello World


-- The optional third argument takes the place of NULL values in the input array.
-- If you omit this argument, then the function ignores NULL array elements.
-- If you provide an empty string, the function inserts a separator for NULL array elements.
-- Example:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  ARRAY_TO_STRING(arr, ".", "N") AS non_empty_string,
  ARRAY_TO_STRING(arr, ".", "") AS empty_string,
  ARRAY_TO_STRING(arr, ".") AS omitted
FROM (SELECT ["a", NULL, "b", NULL, "c", NULL] AS arr);

Query is running:   0%|          |

Downloading:   0%|          |

,non_empty_string,empty_string,omitted
0,a.N.b.N.c.N,a..b..c.,a.b.c


-- Combining arrays
-- In some cases, you might want to combine multiple arrays into a single array. You can accomplish this using the ARRAY_CONCAT() function.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT ARRAY_CONCAT([1, 2], [3, 4], [5, 6]) as count_to_six;

Query is running:   0%|          |

Downloading:   0%|          |

,count_to_six
0,"[1, 2, 3, 4, 5, 6]"


-- Zipping arrays
-- Given two arrays of equal size, you can merge them into a single array consisting of pairs of elements from input arrays, taken from their corresponding positions. This operation is sometimes called zipping.

-- You can zip arrays with UNNEST and WITH OFFSET. In this example, each value pair is stored as a STRUCT in an array.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
  combinations AS (
    SELECT
      ['a', 'b'] AS letters,
      [1, 2, 3] AS numbers
  )
SELECT
  ARRAY(
    SELECT AS STRUCT
      letters[SAFE_OFFSET(index)] AS letter,
      numbers[SAFE_OFFSET(index)] AS number
    FROM combinations
    CROSS JOIN
      UNNEST(
        GENERATE_ARRAY(
          0,
          LEAST(ARRAY_LENGTH(letters), ARRAY_LENGTH(numbers)) - 1)) AS index
    ORDER BY index
  );

-- You can use input arrays of different lengths as long as the first array is equal to or less than the length of the second array. The zipped array will be the length of the shortest input array.

-- To get a zipped array that includes all the elements even when the input arrays are different lengths, change LEAST to GREATEST. Elements of either array that have no associated element in the other array will be paired with NULL.

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
  combinations AS (
    SELECT
      ['a', 'b'] AS letters,
      [1, 2, 3] AS numbers
  )
SELECT
  ARRAY(
    SELECT AS STRUCT
      letters[SAFE_OFFSET(index)] AS letter,
      numbers[SAFE_OFFSET(index)] AS number
    FROM combinations
    CROSS JOIN
      UNNEST(
        GENERATE_ARRAY(
          0,
          GREATEST(ARRAY_LENGTH(letters), ARRAY_LENGTH(numbers)) - 1)) AS index
    ORDER BY index
  );


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,"[{'letter': 'a', 'number': 1}, {'letter': 'b',..."


-- You can use input arrays of different lengths as long as the first array is equal to or less than the length of the second array. The zipped array will be the length of the shortest input array.

-- To get a zipped array that includes all the elements even when the input arrays are different lengths, change LEAST to GREATEST. Elements of either array that have no associated element in the other array will be paired with NULL.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
  combinations AS (
    SELECT
      ['a', 'b'] AS letters,
      [1, 2, 3] AS numbers
  )
SELECT
  ARRAY(
    SELECT AS STRUCT
      letters[SAFE_OFFSET(index)] AS letter,
      numbers[SAFE_OFFSET(index)] AS number
    FROM combinations
    CROSS JOIN
      UNNEST(
        GENERATE_ARRAY(
          0,
          GREATEST(ARRAY_LENGTH(letters), ARRAY_LENGTH(numbers)) - 1)) AS index
    ORDER BY index
  );

-- Building arrays of arrays
-- BigQuery does not support building arrays of arrays directly. Instead, you must create an array of structs, with each struct containing a field of type ARRAY. To illustrate this, consider the following points table:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH points AS
  (SELECT [1, 5] as point
   UNION ALL SELECT [2, 8] as point
   UNION ALL SELECT [3, 7] as point
   UNION ALL SELECT [4, 1] as point
   UNION ALL SELECT [5, 7] as point)
SELECT ARRAY(
  SELECT STRUCT(point)
  FROM points)
  AS coordinates;

# BQ_Introduction_script

!!!!!!!!! Your billing project will be metro-bi-di-training-s00
!!!!!!!!! When you create your dataset in metro-bi-di-training-s00 be sure the name is unique 
!!!!!!!!! In this script replace training_cs  with {your_dataset}

0. GCP console link 
   console.cloud.google.com

In command shell:

1. Setup the current project:
	gcloud config set project metro-bi-di-training-s00
   To check current project 
   gcloud config get-value project

2. 	Create dataset 
	- in current project:
		bq --location=EU mk -d --description “Dataset used to load forecast" training_cs

	- in the project named metro-bi-di-training-s00
		bq --location=EU mk -d --description "Dataset used to load forecast" --dataset metro-bi-di-training-s00:training_cs

3. 	Create table
	- defining schema 
		bq mk -t --description “Forecast Sales for 2022" --label training:BQ_Introduction training_cs.forecast_2022 forecast_date:DATE, forecast_value:NUMERIC
	- reading schema from a json file
		bq mk --table --expiration 3600 --description "Forecast Sales for 2022, read schema from json file" --label training:BQ_Introduction training_cs. forecast_2022_with_schema  $HOME/forecast_sales_2022.json

4. 	View schema of an existing table in JSON format :
		bq show --schema --format=prettyjson [PROJECT_ID]:[DATASET].[TABLE] 
	Export schema into a json file
		bq show --schema --format=prettyjson [PROJECT_ID]:[DATASET].[TABLE] > [SCHEMA_FILE]
		bq show --schema --format=prettyjson metro-bi-di-training-s00:training_cs.forecast_2022 > forecast_2022_export.json
		
5. 	Copy data from $Home folder to GCS bucket named training_cs
		gsutil cp forecast_sales_2022.csv gs://training_cs/
		gsutil cp forecast_2022.json      gs://training_cs/
	List objects from GCS bucket named training_cs 
		gsutil ls -l gs://training_cs/
		
6.	Create table in BQ based on csv from a storage bucket with schema autodetect
	bq load --autodetect --project_id=metro-bi-di-training-s00 --field_delimiter="," --source_format=CSV training_cs.forecast_sales_auto_detect gs://training_cs/forecast_sales_2022.csv

7.	Create table in BQ based on csv from a storage bucket with schema specified in a json file
	bq load --project_id=metro-bi-di-training-s00 --skip_leading_rows=1 --replace=true --field_delimiter="," --source_format=CSV training_cs.forecast_sales gs://training_cs/forecast_sales_2022.csv forecast_2022.json

8. 	Check your data in forecast table = one single entry for each line
	select forecast_date, count(1) 
	from `metro-bi-di-training-s00.training_cs.forecast_sales`
	group by forecast_date
	having count(1) >1;

9.	In forecast all days have values\

	select Forecast.forecast_date,  A.example_date
	from 
			(
			SELECT
					example_date
			FROM 
			UNNEST(GENERATE_DATE_ARRAY('2022-01-01', '2022-01-31', INTERVAL 1 DAY)) AS example_date
			) A left join  `metro-bi-di-training-s00.training_cs.forecast_sales` Forecast
	on A.example_date = Forecast.forecast_date
	where Forecast.forecast_date is null 

10. Exclude Sunday from sales calculation
	select  
			invoice_date,
			FORMAT_DATE('%u',  invoice_date),
			sum(invoice_gross_total_amount) as Sales_Gross
	from `metro-bi-di-training-s02.BQ_Introduction_datasets.invoice_slice_sample` Header
	where FORMAT_DATE('%u',  invoice_date) <> '7' 
	group by 1, 2
	order by 3 desc;

11. DataStudio link
	datastudio.google.com
	
12. Datasource definition
	select  Header.invoice_date,
			Header.Sales_Gross,
			Forecast.forecast_sales
	from 
	(select  
			invoice_date,
			sum(invoice_gross_total_amount) as Sales_Gross
	from `metro-bi-di-training-s02.BQ_Introduction_datasets.invoice_slice_sample` Header
	where FORMAT_DATE('%u',  invoice_date) <> '7' 
	group by 1 ) as Header
	inner join `metro-bi-di-training-s00.training_cs.forecast_sales` as Forecast
			on Header.invoice_date = Forecast.forecast_date	

-- BQ2_script1
-------------------------------------------------------------------
--BigQuery: Arrays
-------------------------------------------------------------------
--Create an array
-----------------
SELECT ['raspberry', 'blackberry', 'strawberry', 'cherry'] AS fruit_array;

--Count the elements in an array with ARRAY_LENGTH:
------------------------------------------------
WITH fruits AS (
     SELECT ['raspberry', 'blackberry', 'strawberry', 'cherry'] AS fruit_array
)
SELECT ARRAY_LENGTH(fruit_array) AS array_size
FROM fruits;

-- Flatten Arrays:
------------------------------------
SELECT ['apple', 'pear', 'plum'] AS item,
        'Jacob' AS customer;

--UNNEST operator transforms the array into a denormalized format:

SELECT
   items,
   customer_name
FROM
UNNEST(['apple', 'pear', 'peach']) AS items
CROSS JOIN
(SELECT 'Jacob' AS customer_name);

--Aggregate into an Array with ARRAY_AGG:
---------------------------------------
WITH fruits AS
(SELECT 1 as basket_no, "apple" AS fruit
	UNION ALL
 SELECT 1 as basket_no, "pear" AS fruit
	UNION ALL
 SELECT 1 as basket_no, "banana" AS fruit)

SELECT basket_no, ARRAY_AGG(fruit) AS fruit_basket
FROM fruits
GROUP BY basket_no;

--Sort Array Output with ORDER BY:
--------------------------------
WITH fruits AS
(SELECT "apple" AS fruit
	UNION ALL
 SELECT "pear" AS fruit
	UNION ALL
 SELECT "banana" AS fruit)

SELECT ARRAY_AGG(fruit ORDER BY fruit) AS fruit_basket
FROM fruits;

--Filter Arrays using WHERE IN:
-----------------------------
WITH groceries AS
(SELECT ['apple', 'pear', 'banana'] AS list
	UNION ALL
SELECT ['carrot', 'apple'] AS list
	UNION ALL
SELECT ['water', 'wine'] AS list)

SELECT
ARRAY(
	SELECT items 
	FROM UNNEST(list) AS items
	WHERE 'apple' IN UNNEST(list)
) AS contains_apple
FROM groceries;

--Practice on real tables:
SELECT 	
	visitId,
	visitStartTime,
	date,
	ARRAY_LENGTH(hits_pagepath) AS array_size
FROM `metro-bi-di-training-s00.BQ2_alex.website_tracking_array`; 

SELECT
	visitNumber,
	visitId,
	visitStartTime,
	date,
	pages
FROM `metro-bi-di-training-s00.BQ2_alex.website_tracking_array` ,
UNNEST (hits_pagepath) AS pages;

SELECT visitId, hits_pagepath
FROM  `metro-bi-di-training-s00.BQ2_alex.website_tracking_array`
WHERE 'shop.metro.fr/shop/category/alimentaire/epicerie-sucrée/chocolat' in UNNEST(hits_pagepath);

-------------------------------------------------------------------
--BigQuery: Struct
-------------------------------------------------------------------

--Create a simple struct element
---------------------------------------
SELECT STRUCT(35 AS age, 'Jacob' AS name) AS customers;

--Create a complex struct element
---------------------------------------
SELECT
	STRUCT(35 AS age, 'Jacob' AS name,['apple', 'pear', 'peach'] AS
	items) AS customers;

--De-normalize the data
---------------------------------------
SELECT visitorId, visitId, device.browser as device_browser, device.language as device_language
FROM `metro-bi-di-training-s00.BQ2_alex.website_tracking_struct`
ORDER by visitId;

--Filter a struct value
---------------------------------------
--Find customers who are using American English on their devices: 

SELECT visitorId, visitId, device.browser as device_browser, device.language as device_language
FROM `metro-bi-di-training-s00.BQ2_alex.website_tracking_struct`
where device.language = 'en-us';

-------------------------------------------------------------------
--BigQuery: Partition and Clusters
-------------------------------------------------------------------

select *
from  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` 
where date_of_day ='2021-06-10';
--14.9GB

-- drop table  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition`;
-- Create table `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition` 
-- partition by date_of_day
--  AS
-- SELECT * FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` ;

select *
from  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition` 
where  date_of_day ='2021-06-10';
--40.6 MB

select *
from  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` 
where date_of_day ='2021-06-10'
and   channelGrouping = 'Email';
--14.9GB

select *
from  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition` 
where  date_of_day ='2021-06-10' 
and    channelGrouping = 'Email';
--40.6 MB

-- drop table  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition_cluster` ;
-- Create table `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition_cluster` 
-- partition by date_of_day
-- cluster by channelGrouping
-- AS
-- SELECT * 
-- FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger`;

select *
from  `metro-bi-di-training-s00.BQ2_alex.ga_sessions_partition_cluster` 
where  date_of_day ='2021-06-10' 
and    channelGrouping = 'Email';
--40.5 MB
--Bytes processed	23.51 MB

-- BQ2_script2_MV
-------------------------------------------------------------------
--BigQuery: Materialized Views
-------------------------------------------------------------------

SELECT  date,count(visitNumber) num_visits
FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` 
group by 1;
--17.2MB
--6.4Kib

-- drop materialized view `metro-bi-di-training-s00.BQ2_alex.ga_sessions_num_visits_mv`;
-- create or replace materialized view `metro-bi-di-training-s00.BQ2_alex.ga_sessions_num_visits_mv`
-- AS
-- SELECT  date,count(visitNumber) num_visits
-- FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` 
-- group by 1;

select * from `metro-bi-di-training-s00.BQ2_alex.ga_sessions_num_visits_mv`;
--6.4Kib

----------
select date_of_day,device.isMobile,avg(totals.timeOnSite) avg_timeOnSite,sum(totals.hits) sum_hits
from `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger`
group by 1,2;

-- create materialized view `metro-bi-di-training-s00.BQ2_alex.ga_sessions_mv`
-- AS 
-- select date_of_day,device.isMobile,avg(totals.timeOnSite) avg_timeOnSite,sum(totals.hits) sum_hits
-- from `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger`
-- group by 1,2;

select * from `metro-bi-di-training-s00.BQ2_alex.ga_sessions_mv`;

select date_of_day,device.isMobile,sum(totals.hits) sum_hits
from `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger`
group by 1,2;

-- BQ2_script3

CREATE TABLE `metro-bi-di-training-s00.BQ2_alex.customers`
(customer_id STRING);

DECLARE id STRING;
SET id = GENERATE_UUID();

INSERT INTO `metro-bi-di-training-s00.BQ2_alex.customers`(customer_id)
VALUES(id);

SELECT FORMAT("Created customer ID %s",id);

---

CREATE OR REPLACE PROCEDURE `metro-bi-di-training-s00.BQ2_alex.SP_customers`()
BEGIN
    DECLARE id STRING;
    SET id = GENERATE_UUID();

    INSERT INTO `metro-bi-di-training-s00.BQ2_alex.customers`(customer_id)
    VALUES(id);

    SELECT FORMAT("Created customer ID %s",id);
END

CALL `metro-bi-di-training-s00.BQ2_alex.SP_customers`();
select * from `metro-bi-di-training-s00.BQ2_alex.customers`;

--slide variables

DECLARE x int64;
BEGIN
DECLARE y int64;
--Here you can reference x and y
END
--Here you can reference x, but not y
----------------------------------------

DECLARE device_categ STRING DEFAULT 'mobile';
DECLARE numVisits,distinctNumVisits INT64;

SET(numVisits,distinctNumVisits) =
    (SELECT AS STRUCT count(fullVisitorId),count(distinct fullVisitorId)
    FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` 
    where device.deviceCategory =device_categ);

SELECT numVisits,distinctNumVisits;

--slide Scripting and Stored procedures

ASSERT
    EXISTS(
        SELECT X
        FROM UNNEST([1,2,3,4,5,6,7]) AS X
        WHERE X =2
    )
AS 'Column must contain value 2';

----------------------error handling
CREATE OR REPLACE PROCEDURE BQ2_alex.sp_error1()
BEGIN 
    SELECT 1/0;
END;

CREATE OR REPLACE PROCEDURE BQ2_alex.sp_error2()
BEGIN 
    CALL BQ2_alex.sp_error1();
END;

BEGIN 
    CALL  BQ2_alex.sp_error2();
EXCEPTION WHEN ERROR THEN
    SELECT
        @@error.message,
        @@error.stack_trace,
        @@error.statement_text,
        @@error.formatted_stack_trace;
END

---slide statements
DECLARE x INT64 DEFAULT 1;
IF x = 1 THEN
  SELECT 'Statement 1';
  SET x = x + 2;
ELSEIF x = 2 THEN 
  SELECT 'Statement 2';
ELSE
  SELECT 'Statement n';
END IF;
SELECT x;

--------------
DECLARE x int64 default 0;
LOOP 
    SET x = x + 1;
    IF x >= 10 THEN
        LEAVE;
    END IF;
END LOOP;

SELECT x;

--slide 

select count(*) from `metro-bi-di-training-s00.BQ2_alex.customers`;

BEGIN

    BEGIN TRANSACTION;
    INSERT INTO `metro-bi-di-training-s00.BQ2_alex.customers`
    VALUES('123');
    --Trigger an error
    SELECT 1/0;
    COMMIT TRANSACTION ;

    EXCEPTION WHEN ERROR THEN
    --Rollback transaction inside the exception handler
    SELECT @@error.message;
    ROLLBACK TRANSACTION ;
END;

---------------------

BEGIN
CREATE TEMP TABLE top_products_tmp
AS
SELECT  p.v2ProductName, count(*) num_views
FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger` s,
unnest (hits)h,
unnest (h.product) p
group by 1
order by 2 desc
LIMIT 5;

SELECT * FROM top_products_tmp;

END;

SELECT * FROM top_products_tmp;


-- BQ2_script4_ML
CREATE OR REPLACE MODEL BQ2_alex.ML_LR_website_tracking
TRANSFORM (
CONCAT(CAST(fullvisitorid as string),CAST(visitStartTime as string)) as sessionId,
EXTRACT(hour from TIMESTAMP_SECONDS(visitStartTime)) visitStartTime,
fullvisitorid,
isMobile,
deviceCategory,
operatingSystem,
browser,
country,
subContinent,
campaign,
medium,
traffic_source,
visits,
hits,
pageviews,
timeOnSite,
date,
isBuyer)
OPTIONS(
model_type='logistic_reg',
INPUT_LABEL_COLS=["isBuyer"]
) AS
select * from `metro-bi-di-training-s00.BQ2_alex.website_tracking_data`;


SELECT
*
FROM
ML.EVALUATE(MODEL `BQ2_alex.ML_LR_website_tracking`);

SELECT *
FROM ML.PREDICT(MODEL `BQ2_alex.ML_LR_website_tracking`, (
    select CONCAT(CAST(fullvisitorid as string),CAST(visitStartTime as string)) as sessionId,
    EXTRACT(hour from TIMESTAMP_SECONDS(visitStartTime)) visitStartTime,
    fullvisitorid,
    isMobile,
    deviceCategory,
    operatingSystem,
    browser,
    country,
    subContinent,
    campaign,
    medium,
    traffic_source,
    visits,
    hits,
    pageviews,
    timeOnSite,
    date
from `metro-bi-di-training-s00.BQ2_alex.website_tracking_data_fresh`
))


-- BQ2_script4_functions
--Find the first 3 visits who accessed the website on each day:
select * except (rn)
from (SELECT date_of_day,visitId, ROW_NUMBER() OVER (partition by date_of_day order by visitStartTime) as rn
  FROM `metro-bi-di-training-s00.BQ2_alex.ga_sessions_fra_ger`)x
where rn <=3
order by 1;

--Create temporary function to implement a specific formula
CREATE TEMP FUNCTION calc_formula(x INT64, y INT64)
  RETURNS FLOAT64
  AS ((x + 4) / y);


SELECT val, calc_formula(val, 2)
  FROM UNNEST([2,3,5,8]) AS val;
  
  SELECT val, calc_formula(val, 2)
  FROM UNNEST([4,6,10,16]) AS val;
  
  -----
  --Create a regular function with the same formula
  --DROP FUNCTION BQ2_alex.calc_formula
  CREATE FUNCTION BQ2_alex.calc_formula(x INT64, y INT64)
  RETURNS FLOAT64
  AS ((x + 4) / y);
  
  SELECT val, BQ2_alex.calc_formula(val, 2)
  FROM UNNEST([2,3,5,8,12]) AS val;
  
  
---Create a temporary function to return the last element of an array of any type:

CREATE TEMP FUNCTION lastArrayElement(arr ANY TYPE) AS (
  arr[ORDINAL(ARRAY_LENGTH(arr))]
);

SELECT lastArrayElement(x) as last_element
  FROM (SELECT [2,3,5,8,13,10,7] as x);
  






In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


# GitHub_SQL-scripts-master

'GitHub'
'C:\Users\srdjan.grbic\OneDrive - METRO\Knowledge\008_Google\GitHub\SQL-scripts-master'

'  https://github.com/harisekhon/sql
'  https://www.linkedin.com/in/harisekhon
' BigQuery Billing Query

'/*	bigquery_billing_commonly_charged_units_of_measure.sql */
'`myproject.mydata.imported_billing_data`'

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  usage_unit,
  COUNT(*) AS billing_records
FROM
  `metro-bi-wb-srb-im-s00.drop_address_id.imported_billing_data`  
WHERE
  cost > 0
GROUP BY
  usage_unit
ORDER BY
  billing_records DESC;

Executing query with job ID: af6211a5-df99-44e8-8c1e-24829b9b9458
Query executing: 0.56s


ERROR:
 404 Not found: Dataset metro-bi-wb-srb-im-s00:drop_address_id was not found in location US

Location: US
Job ID: af6211a5-df99-44e8-8c1e-24829b9b9458



' BigQuery Billing Query
' latest 100 charges
'/*	bigquery_billing_latest_100_charges.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  resource_type,
  start_time,
  end_time,
  cost,
  project_id,
  project_name,
  project_labels_key,
  currency,
  currency_conversion_rate,
  usage_amount,
  usage_unit
FROM
  `bigquery-public-data.311_service_request.imported_billing_data`
WHERE
  cost > 0
ORDER BY
  end_time DESC
LIMIT 100;



' BigQuery Billing Query
' most frequently used products costing more than 1 dollar
'/*	bigquery_billing_products_most_frequently_used.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  COUNT(*) AS billing_records
FROM
  `myproject.mydataset.imported_billing_data`
WHERE
  cost > 1
GROUP BY
  product
ORDER BY
  billing_records DESC;

' BigQuery Billing Query
' products with highest aggregate cost
/*	bigquery_billing_products_with_highest_aggregate_cost.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  product,
  ROUND(SUM(cost), 2) AS total_cost
FROM
  `myproject.mydataset.imported_billing_data`
GROUP BY
  product
ORDER BY
  total_cost DESC;

' BigQuery Billing Query
' products with most billing records
/*	bigquery_billing_products_with_most_billing_records.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  /*product,*/
  COUNT(*) AS billing_records
FROM
  /*`myproject.mydataset.imported_billing_data`*/
  `metro-bi-wb-mag-sci-s00.FinanceBI_Dataset.MTsp_DAN_Tour`
GROUP BY
  product
ORDER BY
  billing_records DESC;

Executing query with job ID: 1bb42f7d-db76-4a5b-b5a9-ea90620945cc
Query executing: 1.05s


ERROR:
 400 Unrecognized name: product at [2:3]

Location: EU
Job ID: 1bb42f7d-db76-4a5b-b5a9-ea90620945cc



' BigQuery Information Schema
'
' Lists Top 10 tables from a list of public projects by their row count

' For a script that auto-generates this SQL for all datasets in the current project, see
'
'     bigquery_generate_query_biggest_tables_across_datasets_by_row_count.sh
'     bigquery_generate_query_biggest_tables_across_datasets_by_size.sh
'
' in the DevOps Bash tools repo:
'
'     https://github.com/HariSekhon/devops-bash-tools

' Unfortunately we can't do this trick for the bigquery-public-data project because of this error:
'
' bq ls -j bigquery-public-data
'
' BigQuery error in ls operation: Access Denied: Project bigquery-public-data: User does not have bigquery.jobs.list permission in project bigquery-public-data.


In [ ]:
%%bigquery --project cf-michelange-dev-sx
'/*	bigquery_info_biggest_public_tables_by_row_count.sql	*/
WITH ALL__TABLES__ AS (
  SELECT * FROM `bigquery-public-data.baseball.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.bls.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.census_bureau_usa.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.cloud_storage_geo_index.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.cms_codes.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.fec.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.genomics_cannabis.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.ghcn_d.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.ghcn_m.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.github_repos.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.hacker_news.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.irs_990.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.medicare.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.new_york.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.nlm_rxnorm.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.noaa_gsod.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.open_images.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.samples.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.san_francisco.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.stackoverflow.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.usa_names.__TABLES__` UNION ALL
  SELECT * FROM `bigquery-public-data.utility_us.__TABLES__`
)
SELECT
  project_id,
  dataset_id,
  table_id,
  row_count,
  ROUND(size_bytes/pow(10,9),2) as size_gb,
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) as last_modified_time,
  CASE
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
  ELSE NULL
  END AS type
FROM
  ALL__TABLES__
ORDER BY
  row_count DESC,
  size_gb DESC
LIMIT 10;


Executing query with job ID: 1f6cde57-6cb9-4416-9178-d91aff463503
Query executing: 0.35s


ERROR:
 400 Syntax error: Unclosed string literal at [1:1]

Location: US
Job ID: 1f6cde57-6cb9-4416-9178-d91aff463503



' BigQuery Information Schema
'
' Lists columns and their details in a given dataset
'
' Processes 10MB of data when run

' https://cloud.google.com/bigquery/docs/tables#columns_view

/*	bigquery_info_columns.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  table_catalog,
  table_schema,
  table_name,
  column_name,
  ordinal_position,
  is_nullable,
  data_type,
  is_generated,
  generation_expression,
  is_stored,
  is_hidden,
  is_updatable,
  is_system_defined,
  is_partitioning_column,
  clustering_ordinal_position
FROM
  ' XXX: replace bigquery-public-data.github_repos with myproject.mydataset
 `bigquery-public-data.github_repos.INFORMATION_SCHEMA.COLUMNS`;


' BigQuery Information Schema
' Lists partitioned and/or clustered columns in a dataset
' Processes 10MB of data when run

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  *
FROM
  ' XXX: replace bigquery-public-data.github_repos with myproject.mydataset
 `bigquery-public-data.github_repos.INFORMATION_SCHEMA.COLUMNS`
WHERE
  is_partitioning_column = 'YES'
    OR
  clustering_ordinal_position IS NOT NULL;

' BigQuery Information Schema
' Lists datasets in the current project along with their descriptions, sorted by last modified time
' Processes 20MB
'/*	bigquery_info_datasets.sql	*/
' XXX: can prefix INFORMATION_SCHEMA with your '<project_id>.' to select from another project, but defaulting to the current project is more convenient than editing this

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  s.*,
  TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), creation_time, DAY) AS days_live,
  option_value AS dataset_description
FROM
  `INFORMATION_SCHEMA.SCHEMATA` AS s
  LEFT JOIN `INFORMATION_SCHEMA.SCHEMATA_OPTIONS` AS so
  USING (schema_name)
' will miss all the datasets without descriptions like this
'WHERE
'  so.option_name = 'description'
ORDER BY
  last_modified_time DESC
LIMIT 15;

' BigQuery Information Schema
' Lists tables and their details in a given dataset
' Processes zero bytes for billing purposes since this is just using the Information Schema
' https://cloud.google.com/bigquery/docs/information-schema-tables
' https://cloud.google.com/bigquery/docs/tables#getting_information_about_tables
' Returns:
' dataset id
' table id
' size in GB
' row counts
' created human readable timestamp
' last modified human readable timestamp
' type - table or view
' number of rows is the number of tables in the dataset
'/*	bigquery_info_tables.sql	*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  dataset_id,
  table_id,
  ' convert bytes to GB
  ROUND(size_bytes/pow(10,9),2) as size_gb,
  row_count,
  ' Convert UNIX EPOCH to a timestamp.
  TIMESTAMP_MILLIS(creation_time) AS creation_time,
  TIMESTAMP_MILLIS(last_modified_time) as last_modified_time,
  CASE
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
  ELSE NULL
  END AS type
FROM
  ' XXX: replace bigquery-public-data.github_repos with myproject.mydataset'
  `bigquery-public-data.github_repos.__TABLES__`
ORDER BY
  size_gb DESC;

'C:\Users\srdjan.grbic\OneDrive - METRO\Knowledge\008_Google\GitHub\bigquery-data-lineage-master'
'# Awesome BigQuery Views

'- [Top Ethereum Balances](#top-ethereum-balances)
'- [Every Ethereum Balance on Every Day](#every-ethereum-balance-on-every-day)
'- [Ethereum Address Number Growth](#ethereum-address-number-growth)
'- [Ether Supply](#ether-supply)
'- [Top Bitcoin Balances](#top-bitcoin-balances)
'- [Bitcoin Gini Index](#bitcoin-gini-index)
'- [Transaction Throughput Comparison](#transaction-throughput-comparison)
'- [Shortest Path Between Addresses](#shortest-path-between-addresses)
'- [More Queries](#more-queries)

'## Top Ethereum Balances

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    -- debits
    select to_address as address, value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.number, blocks.miner
    union all
    -- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum.transactions`
)
select address, sum(value) as balance
from double_entry_book
group by address
order by balance desc
limit 1000

Query is running:   0%|          |

Downloading:   0%|          |

,address,balance
0,0x00000000219ab540356cbb839cbe05303d7705fa,16308887106802428144139100.000000000
1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3938493922551695312588446.000000000
2,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,1996008289413268335198349.000000000
3,0xda9dfa130df4de4673b89022ee50ff26f6ea73cf,1780839979118769395003802.000000000
4,0x0716a17fbaee714f1e6ab0f9d59edbc5f09815c0,1746037393895017987539780.000000000
...,...,...
995,0x95bfd1ba0404e1f382448d0d5370b3dd775db364,12000000000000000000000.000000000
996,0x8707d70c64007a1f095bcda3fbd3f6fbd97089e8,11999996325000000000000.000000000
997,0x36a009f589ff6df59b0db6919be12daf6aa6615a,11996259159845076355260.000000000
998,0xd4fcc07a8da7d55599167991d4ab47f976d0a306,11962557200000100000000.000000000


Related article: https://medium.com/google-cloud/how-to-query-balances-for-all-ethereum-addresses-in-bigquery-fb594e4034a7   


In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    '-- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    '-- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    '-- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value,
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.number, blocks.miner, block_timestamp
    union all
    '-- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value,
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions`
),
double_entry_book_grouped_by_date as (
    select address, sum(value) as balance_increment, date(block_timestamp) as date
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select address, date, sum(balance_increment) over (partition by address order by date) as balance,
    lead(date, 1, current_date()) over (partition by address order by date) as next_date
    from double_entry_book_grouped_by_date
),
calendar AS (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
)
select address, date, balance
from daily_balances

Executing query with job ID: 3dc8e65b-ce37-4e5d-8c2b-7121d17e6484
Query executing: 0.37s


ERROR:
 400 Syntax error: Unclosed string literal at [2:5]

Location: US
Job ID: 3dc8e65b-ce37-4e5d-8c2b-7121d17e6484



'Related article: https://medium.com/google-cloud/plotting-ethereum-address-growth-chart-55cc0e7207b2       
'## Ethereum Address Number Growth


In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    -- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value, 
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner, block_timestamp
    union all
    -- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value, 
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions`
),
double_entry_book_grouped_by_date as (
    select address, sum(value) as balance_increment, date(block_timestamp) as date
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select address, date, sum(balance_increment) over (partition by address order by date) as balance,
    lead(date, 1, current_date()) over (partition by address order by date) as next_date
    from double_entry_book_grouped_by_date
),
calendar AS (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
)
select date, count(*) as address_count
from daily_balances
where balance > 0
group by date

Executing query with job ID: 464d46d6-635b-4b6f-a8a8-0d9cffc1975e
Query executing: 36.80s

KeyboardInterrupt: ignored

'Related article: https://medium.com/google-cloud/plotting-ethereum-address-growth-chart-55cc0e7207b2    
'## Ether Supply

'```sql 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with ether_emitted_by_date  as (
  select date(block_timestamp) as date, sum(value) as value
  from `bigquery-public-data.crypto_ethereum.traces`
  where trace_type in ('genesis', 'reward')
  group by date(block_timestamp)
)
select date, sum(value) OVER (ORDER BY date) / power(10, 18) AS supply
from ether_emitted_by_date

Query is running:   0%|          |

Downloading:   0%|          |

,date,supply
0,1970-01-01,7.200999e+07
1,2015-07-30,7.204930e+07
2,2015-07-31,7.208549e+07
3,2015-08-01,7.211320e+07
4,2015-08-02,7.214142e+07
...,...,...
2736,2023-01-24,1.223739e+08
2737,2023-01-25,1.223739e+08
2738,2023-01-26,1.223739e+08
2739,2023-01-27,1.223739e+08


'Related article: https://medium.com/google-cloud/how-to-query-ether-supply-in-bigquery-90f8ae795a8     
'## Top Bitcoin Balances
'```sql 

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH double_entry_book AS (
   -- debits
   SELECT array_to_string(inputs.addresses, ",") as address, inputs.type, -inputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.inputs` as inputs
   UNION ALL
   -- credits
   SELECT array_to_string(outputs.addresses, ",") as address, outputs.type, outputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.outputs` as outputs
)
SELECT address, type, sum(value) as balance
FROM double_entry_book
GROUP BY address, type
ORDER BY balance DESC
LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

,address,type,balance
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,scripthash,24859733578256.000000000
1,bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24...,witness_v0_scripthash,16800998776163.000000000
2,3JJmF63ifcamPLiAmLgG96RA599yNtY3EQ,scripthash,12735105708043.000000000
3,1LQoWist8KkaUXSPKZHNvEyfrEkPHzSsCd,pubkeyhash,12434740305401.000000000
4,bc1qazcm763858nkj2dj986etajv6wquslv8uxwczt,witness_v0_keyhash,9464330605085.000000000
...,...,...,...
995,18SkThdkvbegRcZc9cw5GKS87cidmLUmdD,pubkeyhash,173123434364.000000000
996,15H8vDVWZvySPnYYTd4FmRUXnMAqykKTN3,pubkeyhash,173123431080.000000000
997,1BesmNnogi6iS2YnSWmzniUnuUYWvpDsgt,pubkeyhash,173123431080.000000000
998,1NTWJbJwSwGxfArW8iGibWNH8t374uL4YZ,pubkeyhash,173123431080.000000000


'## Bitcoin Gini Index
'```sql

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with 
double_entry_book as (
    select 
        array_to_string(outputs.addresses,',') as address,
        value, block_timestamp
    from `crypto-etl-bitcoin-prod.bitcoin_blockchain.transactions` join unnest(outputs) as outputs
    union all
    select 
        array_to_string(inputs.addresses,',') as address,
        -value as value, block_timestamp
    from `crypto-etl-bitcoin-prod.bitcoin_blockchain.transactions` join unnest(inputs) as inputs
),
double_entry_book_by_date as (
    select 
        date(block_timestamp) as date, 
        address, 
        sum(value * 0.00000001) as value
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select 
        address, 
        date,
        sum(value) over (partition by address order by date) as balance,
        lead(date, 1, current_date()) over (partition by address order by date) as next_date
        from double_entry_book_by_date
),
calendar as (
    select date from unnest(generate_date_array('2009-01-03', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
    where balance > 1
),
address_counts as (
    select
        date,
        count(*) as address_count
    from
        daily_balances
    group by date
),
daily_balances_sampled as (
    select address, daily_balances.date, balance
    from daily_balances
    join address_counts on daily_balances.date = address_counts.date
    where mod(abs(farm_fingerprint(address)), 100000000)/100000000 <= safe_divide(10000, address_count) 
),
ranked_daily_balances as (
    select 
        date,
        balance,
        row_number() over (partition by date order by balance desc) as rank
    from daily_balances_sampled
)
select 
    date, 
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * sum((balance * (rank - 1) + balance / 2)) / count(*) / sum(balance) as gini
from ranked_daily_balances
group by date
having sum(balance) > 0
order by date asc

Executing query with job ID: 60412152-67dc-4f5d-85ae-41b27e675706
Query executing: 0.65s


ERROR:
 403 Access Denied: Table crypto-etl-bitcoin-prod:bitcoin_blockchain.transactions: User does not have permission to query table crypto-etl-bitcoin-prod:bitcoin_blockchain.transactions, or perhaps it does not exist in location US.

Location: US
Job ID: 60412152-67dc-4f5d-85ae-41b27e675706



'Related article: 
'https://cloud.google.com/blog/products/data-analytics/introducing-six-new-cryptocurrencies-in-bigquery-public-datasets-and-how-to-analyze-them

'## Shortest Path Between Two Ethereum Addresses
''
'```sql

In [ ]:
%%bigquery --project cf-michelange-dev-sx
DECLARE start_address STRING DEFAULT LOWER('0x47068105c5feff69e44520b251b9666d4b512a70');
DECLARE end_address STRING DEFAULT LOWER('0x2604afb5a64992e5abbf25865c9d3387ade92bad');

with traces_0 as (
  select *
  from `bigquery-public-data.crypto_ethereum.traces`
  where from_address = start_address
),
traces_1_hop as (
  SELECT
      1 as hops,
      traces_1.from_address,
      traces_1.to_address,
      traces_1.trace_address,
      traces_1.block_timestamp,
      concat(traces_0.from_address, ' -> ', traces_0.to_address, ' -> ', traces_1.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_1
  INNER JOIN traces_0
  ON traces_0.to_address = traces_1.from_address
  AND traces_0.block_timestamp <= traces_1.block_timestamp 
),
traces_2_hops as (
  SELECT
      2 as hops,
      traces_2.from_address,
      traces_2.to_address,
      traces_2.trace_address,
      traces_2.block_timestamp,
      concat(path, ' -> ', traces_2.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_2
  INNER JOIN traces_1_hop
  ON traces_1_hop.to_address = traces_2.from_address
  AND traces_1_hop.block_timestamp <= traces_2.block_timestamp 
),
traces_3_hops as (
  SELECT
      3 as hops,
      traces_3.from_address,
      traces_3.to_address,
      traces_3.trace_address,
      traces_2_hops.block_timestamp,
      concat(path, ' -> ', traces_3.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_3
  INNER JOIN traces_2_hops
  ON traces_2_hops.to_address = traces_3.from_address
  AND traces_2_hops.block_timestamp <= traces_3.block_timestamp 
  where traces_3.to_address = end_address
),
traces_all_hops AS (
    select * from traces_1_hop
    UNION ALL
    select * from traces_2_hops
    UNION ALL
    select * from traces_3_hops
)
select *
from traces_all_hops
where hops = 3
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

,hops,from_address,to_address,trace_address,block_timestamp,path
0,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-28 08:43:51+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
1,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-28 08:43:51+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
2,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-28 08:43:51+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
3,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-28 08:43:51+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
4,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-28 08:43:51+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
...,...,...,...,...,...,...
95,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-29 10:24:08+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
96,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-29 10:24:08+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
97,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-29 10:24:08+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
98,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2020-11-29 10:24:08+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...


'The query above will fail if there are too many traces. There is a workaround here 
'https://console.cloud.google.com/bigquery?sq=749871510730:082d50190dc04d79aaf7ee4d5c6f9d00
'## Transaction Throughput Comparison

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with bitcoin_throughput as (
    -- takes transactions count in every block and divides it by average block time on that day
    select 'bitcoin' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_bitcoin.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
bitcoin_cash_throughput as (
    select 'bitcoin_cash' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_bitcoin_cash.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
ethereum_throughput as (
    select 'ethereum' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
ethereum_classic_throughput as (
    select 'ethereum_classic' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_ethereum_classic.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
dogecoin_throughput as (
    select 'dogecoin' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_dogecoin.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
litecoin_throughput as (
    select 'litecoin' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_litecoin.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
dash_throughput as (
    select 'dash' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_dash.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
),
zcash_throughput as (
    select 'zcash' as chain, count(*) / (24 * 60 * 60 / count(*) over (partition by date(block_timestamp))) as throughput, block_timestamp as time
    from `bigquery-public-data.crypto_zcash.transactions` as transactions
    group by transactions.block_number, transactions.block_timestamp
    order by throughput desc
    limit 1
)
select * from bitcoin_throughput
union all
select * from bitcoin_cash_throughput
union all
select * from ethereum_throughput
union all
select * from ethereum_classic_throughput
union all
select * from dogecoin_throughput
union all
select * from litecoin_throughput
union all
select * from dash_throughput
union all
select * from zcash_throughput
order by throughput desc

Query is running:   0%|          |

Downloading:   0%|          |

,chain,throughput,time
0,bitcoin_cash,281.999676,2018-11-10 14:34:35+00:00
1,ethereum,116.875451,2022-12-23 20:18:59+00:00
2,dash,67.076319,2020-03-16 00:05:13+00:00
3,dogecoin,54.789282,2014-08-05 02:53:06+00:00
4,litecoin,49.419167,2019-11-02 18:43:59+00:00
5,ethereum_classic,23.739896,2019-01-13 08:09:56+00:00
6,bitcoin,21.956539,2015-08-01 01:06:41+00:00
7,zcash,7.876435,2018-08-03 02:08:54+00:00


'Related article: 
'https://medium.com/@medvedev1088/comparing-transaction-throughputs-for-8-blockchains-in-google-bigquery-with-google-data-studio-edbabb75b7f1
'## More Queries
'Check out this awesome repository: https://github.com/RokoMijic/awesome-bigquery-views

'https://evgemedvedev.medium.com/comparing-transaction-throughputs-for-8-blockchains-in-google-bigquery-with-google-data-studio-edbabb75b7f1'

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with daily_transactions as (
    select date(block_timestamp) as date, count(*) as count
    from `bigquery-public-data.crypto_bitcoin.transactions`
    group by date
)
select date_trunc(date, WEEK) as week, cast(avg(count) as INT64) as count
from daily_transactions
group by week
order by week




Query is running:   0%|          |

Downloading:   0%|          |

,week,count
0,2008-12-28,1
1,2009-01-04,38
2,2009-01-11,114
3,2009-01-18,118
4,2009-01-25,125
...,...,...
730,2022-12-25,248632
731,2023-01-01,252714
732,2023-01-08,292650
733,2023-01-15,292322


# Awesome BigQuery Views

- [Top Ethereum Balances](#top-ethereum-balances)
- [Every Ethereum Balance on Every Day](#every-ethereum-balance-on-every-day)
- [Ethereum Address Number Growth](#ethereum-address-number-growth)
- [Ether Supply](#ether-supply)
- [Top Bitcoin Balances](#top-bitcoin-balances)
- [Bitcoin Gini Index](#bitcoin-gini-index)
- [Transaction Throughput Comparison](#transaction-throughput-comparison)
- [Shortest Path Between Addresses](#shortest-path-between-addresses)
- [More Queries](#more-queries)


## Top Ethereum Balances

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    -- debits
    select to_address as address, value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.number, blocks.miner
    union all
    -- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum.transactions`
)
select address, sum(value) as balance
from double_entry_book
group by address
order by balance desc
limit 1000


Query is running:   0%|          |

Downloading:   0%|          |

,address,balance
0,0x00000000219ab540356cbb839cbe05303d7705fa,16308887106802428144139100.000000000
1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3946949324689032126825094.000000000
2,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,1996008289413268335198349.000000000
3,0xda9dfa130df4de4673b89022ee50ff26f6ea73cf,1780839979118769395003802.000000000
4,0x0716a17fbaee714f1e6ab0f9d59edbc5f09815c0,1746037393895017987539780.000000000
...,...,...
995,0x95bfd1ba0404e1f382448d0d5370b3dd775db364,12000000000000000000000.000000000
996,0x8707d70c64007a1f095bcda3fbd3f6fbd97089e8,11999996325000000000000.000000000
997,0x36a009f589ff6df59b0db6919be12daf6aa6615a,11996259159845076355260.000000000
998,0xd4fcc07a8da7d55599167991d4ab47f976d0a306,11962557200000100000000.000000000


## Every Ethereum Balance on Every Day

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    -- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value,
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.number, blocks.miner, block_timestamp
    union all
    -- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value,
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions`
),
double_entry_book_grouped_by_date as (
    select address, sum(value) as balance_increment, date(block_timestamp) as date
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select address, date, sum(balance_increment) over (partition by address order by date) as balance,
    lead(date, 1, current_date()) over (partition by address order by date) as next_date
    from double_entry_book_grouped_by_date
),
calendar AS (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
)
select address, date, balance
from daily_balances

## Ethereum Address Number Growth

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with double_entry_book as (
    -- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select 
        miner as address, 
        sum(cast(receipt_gas_used as numeric) * cast((receipt_effective_gas_price - coalesce(base_fee_per_gas, 0)) as numeric)) as value, 
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner, block_timestamp
    union all
    -- transaction fees credits
    select 
        from_address as address, 
        -(cast(receipt_gas_used as numeric) * cast(receipt_effective_gas_price as numeric)) as value, 
        block_timestamp
    from `bigquery-public-data.crypto_ethereum.transactions`
),
double_entry_book_grouped_by_date as (
    select address, sum(value) as balance_increment, date(block_timestamp) as date
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select address, date, sum(balance_increment) over (partition by address order by date) as balance,
    lead(date, 1, current_date()) over (partition by address order by date) as next_date
    from double_entry_book_grouped_by_date
),
calendar AS (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
)
select date, count(*) as address_count
from daily_balances
where balance > 0
group by date

## Ether Supply

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with ether_emitted_by_date  as (
  select date(block_timestamp) as date, sum(value) as value
  from `bigquery-public-data.crypto_ethereum.traces`
  where trace_type in ('genesis', 'reward')
  group by date(block_timestamp)
)
select date, sum(value) OVER (ORDER BY date) / power(10, 18) AS supply
from ether_emitted_by_date

## Top Bitcoin Balances

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH double_entry_book AS (
   -- debits
   SELECT array_to_string(inputs.addresses, ",") as address, inputs.type, -inputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.inputs` as inputs
   UNION ALL
   -- credits
   SELECT array_to_string(outputs.addresses, ",") as address, outputs.type, outputs.value as value
   FROM `bigquery-public-data.crypto_bitcoin.outputs` as outputs
)
SELECT address, type, sum(value) as balance
FROM double_entry_book
GROUP BY address, type
ORDER BY balance DESC
LIMIT 1000

## Bitcoin Gini Index

In [ ]:
%%bigquery --project cf-michelange-dev-sx
with 
double_entry_book as (
    select 
        array_to_string(outputs.addresses,',') as address,
        value, block_timestamp
    from `crypto-etl-bitcoin-prod.bitcoin_blockchain.transactions` join unnest(outputs) as outputs
    union all
    select 
        array_to_string(inputs.addresses,',') as address,
        -value as value, block_timestamp
    from `crypto-etl-bitcoin-prod.bitcoin_blockchain.transactions` join unnest(inputs) as inputs
),
double_entry_book_by_date as (
    select 
        date(block_timestamp) as date, 
        address, 
        sum(value * 0.00000001) as value
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select 
        address, 
        date,
        sum(value) over (partition by address order by date) as balance,
        lead(date, 1, current_date()) over (partition by address order by date) as next_date
        from double_entry_book_by_date
),
calendar as (
    select date from unnest(generate_date_array('2009-01-03', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
    where balance > 1
),
address_counts as (
    select
        date,
        count(*) as address_count
    from
        daily_balances
    group by date
),
daily_balances_sampled as (
    select address, daily_balances.date, balance
    from daily_balances
    join address_counts on daily_balances.date = address_counts.date
    where mod(abs(farm_fingerprint(address)), 100000000)/100000000 <= safe_divide(10000, address_count) 
),
ranked_daily_balances as (
    select 
        date,
        balance,
        row_number() over (partition by date order by balance desc) as rank
    from daily_balances_sampled
)
select 
    date, 
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * sum((balance * (rank - 1) + balance / 2)) / count(*) / sum(balance) as gini
from ranked_daily_balances
group by date
having sum(balance) > 0
order by date asc

Executing query with job ID: 782d5922-bdb5-4b3f-bcf8-160f02eb38a5
Query executing: 0.65s


ERROR:
 403 Access Denied: Table crypto-etl-bitcoin-prod:bitcoin_blockchain.transactions: User does not have permission to query table crypto-etl-bitcoin-prod:bitcoin_blockchain.transactions, or perhaps it does not exist in location US.

Location: US
Job ID: 782d5922-bdb5-4b3f-bcf8-160f02eb38a5



## Shortest Path Between Two Ethereum Addresses

In [ ]:
%%bigquery --project cf-michelange-dev-sx
DECLARE start_address STRING DEFAULT LOWER('0x47068105c5feff69e44520b251b9666d4b512a70');
DECLARE end_address STRING DEFAULT LOWER('0x2604afb5a64992e5abbf25865c9d3387ade92bad');

with traces_0 as (
  select *
  from `bigquery-public-data.crypto_ethereum.traces`
  where from_address = start_address
),
traces_1_hop as (
  SELECT
      1 as hops,
      traces_1.from_address,
      traces_1.to_address,
      traces_1.trace_address,
      traces_1.block_timestamp,
      concat(traces_0.from_address, ' -> ', traces_0.to_address, ' -> ', traces_1.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_1
  INNER JOIN traces_0
  ON traces_0.to_address = traces_1.from_address
  AND traces_0.block_timestamp <= traces_1.block_timestamp 
),
traces_2_hops as (
  SELECT
      2 as hops,
      traces_2.from_address,
      traces_2.to_address,
      traces_2.trace_address,
      traces_2.block_timestamp,
      concat(path, ' -> ', traces_2.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_2
  INNER JOIN traces_1_hop
  ON traces_1_hop.to_address = traces_2.from_address
  AND traces_1_hop.block_timestamp <= traces_2.block_timestamp 
),
traces_3_hops as (
  SELECT
      3 as hops,
      traces_3.from_address,
      traces_3.to_address,
      traces_3.trace_address,
      traces_2_hops.block_timestamp,
      concat(path, ' -> ', traces_3.to_address) as path
  FROM `bigquery-public-data.crypto_ethereum.traces` AS traces_3
  INNER JOIN traces_2_hops
  ON traces_2_hops.to_address = traces_3.from_address
  AND traces_2_hops.block_timestamp <= traces_3.block_timestamp 
  where traces_3.to_address = end_address
),
traces_all_hops AS (
    select * from traces_1_hop
    UNION ALL
    select * from traces_2_hops
    UNION ALL
    select * from traces_3_hops
)
select *
from traces_all_hops
where hops = 3
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

,hops,from_address,to_address,trace_address,block_timestamp,path
0,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
1,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
2,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
3,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
4,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
...,...,...,...,...,...,...
95,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
96,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
97,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...
98,3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2604afb5a64992e5abbf25865c9d3387ade92bad,4,2021-01-28 05:46:33+00:00,0x47068105c5feff69e44520b251b9666d4b512a70 -> ...


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


In [ ]:
%%bigquery --project cf-michelange-dev-sx


# Optimal performance

/* Problem 1 - The initial query threw an "out of resource" error: */

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
Q1 AS
(SELECT count (distinct A.id) countPersonTotal FROM `metro-bi-dl-srb-prod.alex_prep.person` as A
),
Q2 AS
(select count(distinct A.id) countPersonWithIdam from `metro-bi-dl-srb-prod.alex_prep.person` as A, `metro-bi-dl-srb-prod.alex_prep.mapping` as B , UNNEST(B.entry.keys.targetValues) AS targetId
where B.sourceSystem="idam" and B.targetSystem="cdm" and B.entry.keys.targetKey="person" and targetId=A.id)
SELECT Q1.countPersonTotal, Q2.countPersonWithIdam, 'BUL' AS COUNTRY from Q1, Q2;

Query is running:   0%|          |

Downloading:   0%|          |

,countPersonTotal,countPersonWithIdam,COUNTRY
0,1334102,9585,BUL


/* Problem 1 - Possible solution:  The proposed query first filtered the distinct ids from both tables separately and joined only these smaller tables together, 
avoiding the "out of resource" error:*/

In [ ]:
%%bigquery --project cf-michelange-dev-sx
WITH
T1 AS

       ( SELECT distinct A.id

        FROM `metro-bi-dl-ned-prod.alex_prep.person` as A),
T2 AS
       (SELECT distinct targetId
        FROM`metro-bi-dl-ned-prod.alex_prep.mapping` as B ,
        UNNEST(B.entry.keys.targetValues) AS targetId
        where B.sourceSystem        = "idam"
         and B.targetSystem            = "cdm"
         and B.entry.keys.targetKey = "person" ),
Q1 AS
        (SELECT count (1) countPersonTotal

         FROM T1 as A),
Q2 AS
        (select count(1) countPersonWithIdam
        from T1 as A
        inner join T2 as B
        on targetId=A.id)

SELECT Q1.countPersonTotal, Q2.countPersonWithIdam, 'BUL' AS COUNTRY from Q1, Q2;

Query is running:   0%|          |

Downloading:   0%|          |

,countPersonTotal,countPersonWithIdam,COUNTRY
0,6721194,896075,BUL


# Formating

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
  invoice_time
  ,CAST(invoice_time/10000 AS INT64) AS time_hour
  ,MOD(CAST(FLOOR(invoice_time/100) AS INT64),100) AS time_minute
  ,MOD(invoice_time,100) AS time_second 
FROM
  metro-bi-dl-prep-srb-prod.cc_dwh.dw_cust_invoice_orig
WHERE
  date_of_day = '2022-09-01'

Executing query with job ID: abc769da-3ba5-4e29-a3e8-62007551da2b
Query executing: 0.75s


ERROR:
 400 No matching signature for operator / for argument types: TIME, INT64. Supported signatures: FLOAT64 / FLOAT64; NUMERIC / NUMERIC; BIGNUMERIC / BIGNUMERIC; INTERVAL / INT64 at [3:9]

Location: EU
Job ID: abc769da-3ba5-4e29-a3e8-62007551da2b



# SAP BW

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT 
  F.sap_entity_cd,
  E.sap_entity_desc,
  F.sap_account_cd,
  A.sap_account_desc,
  F.sap_sub_account_cd,
  SA.sap_sub_account_desc,
  F.sap_scenario_cd,
  SCE.sap_scenario_desc,
  F.month_id,
  M.month_desc,
  F.sap_currency_cd,
  SUM(F.sap_value) AS sap_value,
  SUM(ACCU.sap_value_accu) AS sap_value_accu,
  SUM(F.sap_value_bc) AS sap_value_bc,
  SUM(ACCU.sap_value_bc_accu) AS sap_value_bc_accu

FROM `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_ent_acct_m` AS F
  INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity` AS E ON E.sap_entity_cd=F.sap_entity_cd
  INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_account` AS A ON A.sap_account_cd=F.sap_account_cd
  INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_sub_account` AS SA ON SA.sap_sub_account_cd=F.sap_sub_account_cd
  INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_scenario` AS SCE ON SCE.sap_scenario_cd=F.sap_scenario_cd
  INNER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_time_month` AS M ON M.month_id=F.month_id

INNER JOIN 
(
  SELECT
    F.sap_entity_cd,
    F.sap_account_cd,
    F.sap_sub_account_cd,
    F.sap_scenario_cd,
    TIME_FY.month_id,
    F.sap_currency_cd,
    SUM(F.sap_value)    AS sap_value_accu,
    SUM(F.sap_value_bc) AS sap_value_bc_accu
  FROM `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_ent_acct_m` AS F 
  INNER JOIN `metro-bi-dl-ger-prod.cc_dwh_views_msi.DW_V_TIMEFY_YEAR_TO_MONTH` AS TIME_FY ON TIME_FY.fy_month_id_accu_ty=F.month_id 
  GROUP BY    
    F.sap_entity_cd,
    F.sap_account_cd,
    F.sap_sub_account_cd,
    F.sap_scenario_cd,
    TIME_FY.month_id,
    F.sap_currency_cd
) AS accu
  ON  accu.sap_entity_cd      = F.sap_entity_cd
  AND accu.sap_account_cd     = F.sap_account_cd
  AND accu.sap_sub_account_cd = F.sap_sub_account_cd
  AND accu.sap_scenario_cd    = F.sap_scenario_cd
  AND accu.month_id           = F.month_id
  AND accu.sap_currency_cd    = F.sap_currency_cd

-- WHERE F.month_id = 202209 -- here can select a certain month or delete the WHERE clause
WHERE F.sap_entity_cd like'%[+] CCRS'
GROUP BY   
  F.sap_entity_cd,
  E.sap_entity_desc,
  F.sap_account_cd,
  A.sap_account_desc,
  F.sap_sub_account_cd,
  SA.sap_sub_account_desc,
  F.sap_scenario_cd,
  SCE.sap_scenario_desc,
  F.month_id,
  M.month_desc,
  F.sap_currency_cd



Query is running:   0%|          |

Downloading:   0%|          |

,sap_entity_cd,sap_entity_desc,sap_account_cd,sap_account_desc,sap_sub_account_cd,sap_sub_account_desc,sap_scenario_cd,sap_scenario_desc,month_id,month_desc,sap_currency_cd,sap_value,sap_value_accu,sap_value_bc,sap_value_bc_accu
0,[+] CCRS,Serbia,4423233100,IMF Delivery percentage,#,#,2021A00,2021 Actual,202109,2021/09 Sep,-1,0E-9,0E-9,0E-9,0E-9
1,[+] CCRS,Serbia,4423233100,IMF Delivery percentage,#,#,2021A00,2021 Actual,202104,2021/04 Apr,-1,0E-9,0E-9,0E-9,0E-9
2,[+] CCRS,Serbia,4423233100,IMF Delivery percentage,#,#,2021A00,2021 Actual,202106,2021/06 Jun,-1,0E-9,0E-9,0E-9,0E-9
3,[+] CCRS,Serbia,4423233100,IMF Delivery percentage,#,#,2021A00,2021 Actual,202107,2021/07 Jul,-1,0E-9,0E-9,0E-9,0E-9
4,[+] CCRS,Serbia,4423233100,IMF Delivery percentage,#,#,2021A00,2021 Actual,202105,2021/05 Mai,-1,0E-9,0E-9,0E-9,0E-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10290,[+] CCRS,Serbia,4423400000,Total Delivery Sales Own Brand (excl. TAC/PU),XPY,Prior year,2022F30,2022 Forecast III -,202110,2021/10 Okt,-1,0E-9,0E-9,0E-9,0E-9
10291,[+] CCRS,Serbia,4423400000,Total Delivery Sales Own Brand (excl. TAC/PU),XPY,Prior year,2022F30,2022 Forecast III -,202111,2021/11 Nov,-1,0E-9,0E-9,0E-9,0E-9
10292,[+] CCRS,Serbia,4424132300,Sales of SCO customers (unadjusted) Rolling 12...,XPY,Prior year,2022F30,2022 Forecast III -,202110,2021/10 Okt,EUR,0E-9,0E-9,-60942506.000000000,-60942506.000000000
10293,[+] CCRS,Serbia,4424112300,Income SCO unadjusted,XPY,Prior year,2022F30,2022 Forecast III -,202110,2021/10 Okt,EUR,0E-9,0E-9,-948805.000000000,-948805.000000000


# Germany Views

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT
 invl.orig_invoice_id,
 invl.invoice_id,
 invl.home_store_id,
 CASE
 WHEN invl.store_id - invl.home_store_id = 0 THEN 0
 ELSE 1
 END AS home_store_ind,
 invl.invoice_line_id,
 invl.gm_cond_type_cd,
 invl.art_no,
 invl.var_tu_key,
 invl.vat_rate,
 invl.fnf_cd,
 invl.disc_line_ind,
 invl.license_type_cd,
 invl.suppl_no AS suppl_hist_no,
 invl.suppl_prod_no AS suppl_prod_hist_no,
 invl.commercial_suppl_no AS commercial_suppl_hist_no,
 invl.private_label_cd,
 invl.lot_no,
 invl.lot_no_entry_method_cd,
 invl.vat_reverse_charge_ind,
 invl.sell_ean_no AS sell_ean_hist_no,
 invl.high_vol_sales_ind,
 CAST(1 as integer) AS cu_num_invoice_lines,
 invl.vendor_no,
 invl.fulfillment_model_cd,
 -1 * round(invl.sell_val_nsp * invl.prof_bonus_pct, 4)   * security.inv_prof_inc_det_check_ind_cl   AS cu_prof_bonus_val,
 round(invl.hdp_val_nnbp_per_colli * invl.sell_qty_colli, 4)   * security.inv_prof_inc_det_check_ind_cl   AS cu_prof_hidden_profit_val,
 round(invl.sell_val_nnbp * invl.prof_later_income_pct, 4)   * security.inv_prof_inc_det_check_ind_cl   AS cu_prof_later_income_val,
 round(invl.sell_val_nnbp * invl.prof_shrinkage_store_pct, 4)   * security.inv_prof_inc_det_check_ind_cl   AS cu_prof_shrinkage_store_val,
 round(invl.sell_val_nnbp * invl.prof_shrinkage_plat_pct, 4)   * security.inv_prof_inc_det_check_ind_cl   AS cu_prof_shrinkage_plat_val,
 round(invl.prof_delvery_execution_val,4)   * security.inv_prof_cost_det_check_ind_cl   AS cu_prof_delvery_execution_val,
 round(invl.prof_dlvry_trnsp_drop_val * invl.share_nnbp_line_pct,4)   * security.inv_prof_cost_det_check_ind_cl    AS cu_prof_delivery_trnsp_val,
 round(invl.sell_val_nnbp * invl.prof_delivery_hq_pct,4)   * security.inv_prof_cost_det_check_ind_cl    AS cu_prof_delivery_hq_val,
 round(invl.sell_val_nnbp * invl.prof_delivery_oth_pct,4)   * security.inv_prof_cost_det_check_ind_cl   AS cu_prof_delivery_oth_val,
 round(invl.sell_val_nnbp * invl.prof_sl_force_pct,4)   * security.inv_prof_cost_det_check_ind_cl    AS cu_prof_sl_force_val,
 round(invl.sell_val_nnbp * invl.prof_kam_pct,4)   * security.inv_prof_cost_det_check_ind_cl   AS cu_prof_kam_val,

round(invl.sell_val_nnbp * invl.prof_sl_force_hq_pct, 4)   * security.inv_prof_cost_det_check_ind_cl    AS cu_prof_sl_force_hq_val,
 CASE
 WHEN 1 = 1   and (security.inv_prof_tot_check_ind_cl=1 OR security.inv_prof_inc_det_check_ind_cl=1 OR security.inv_prof_cost_det_check_ind_cl=1)
 THEN round(invl.prof_theoretical_nsp_val,4)
 ELSE CAST(0 as NUMERIC)
 END AS cu_prof_theoretical_nsp_val,

CASE
 WHEN 1 = 1   and (security.inv_prof_tot_check_ind_cl=1 OR security.inv_prof_inc_det_check_ind_cl=1 OR security.inv_prof_cost_det_check_ind_cl=1)
 THEN round(invl.sell_val_nnbp,4)
 ELSE CAST(0 as NUMERIC)
 END AS cu_prof_theoretical_nnbp_val,

invl.prof_disc_nsp_ind AS cu_prof_disc_nsp_ind,
 CASE
 WHEN 1 = 1   and (security.inv_prof_tot_check_ind_cl=1 OR security.inv_prof_inc_det_check_ind_cl=1 OR security.inv_prof_cost_det_check_ind_cl=1)
 THEN round(invl.prof_disc_nsp_val,4)
 ELSE CAST(0 as NUMERIC)
 END AS cu_prof_disc_nsp_val,

invl.prof_price_markup_nsp_ind AS cu_prof_price_markup_nsp_ind,
 CASE
 WHEN 1 = 1   and (security.inv_prof_tot_check_ind_cl=1 OR security.inv_prof_inc_det_check_ind_cl=1 OR security.inv_prof_cost_det_check_ind_cl=1)
 THEN round(invl.prof_price_markup_nsp_val,4)
 ELSE CAST(0 as NUMERIC)
 END AS cu_prof_price_markup_nsp_val,

invl.cust_domestic_ind,
 invl.prof_bonus_ind AS cu_prof_bonus_ind,
 round(invl.suppl_funding_unit_val * invl.sell_qty_colli * invl.tunit_qty, 4)   * security.nnbp_check_ind   AS cu_suppl_funding_val,
 invl.sell_qty_colli AS cu_sell_qty_colli,
 round(invl.sell_val_gsp,4) AS cu_sell_val_gsp,
 round(invl.sell_val_nsp,4) AS cu_sell_val_nsp,
 round(invl.sell_val_nnbp,4)   * security.nnbp_check_ind   AS cu_sell_val_nnbp,
 round(invl.sell_val_invbp,4)   * security.nnbp_check_ind   AS cu_sell_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0 THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd = 0 THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0 THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0 THEN  round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0 THEN  round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0 THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd <> 0 THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0 THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0 THEN round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0 THEN round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_val_invbp,
 CASE
 WHEN upper(invl.fnf_cd) <> 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_f_qty_colli,
 CASE
 WHEN upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_f_val_gsp,
 CASE
 WHEN upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_f_val_nsp,
 CASE
 WHEN upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_f_val_nnbp,
 CASE
 WHEN upper(invl.fnf_cd) <> 'N' THEN  round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_f_val_invbp,
 CASE
 WHEN upper(invl.fnf_cd) = 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_nf_qty_colli,
 CASE
 WHEN upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_nf_val_gsp,
 CASE
 WHEN upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_nf_val_nsp,
 CASE
 WHEN upper(invl.fnf_cd) = 'N' THEN  round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_nf_val_nnbp,
 CASE
 WHEN upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_nf_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) <> 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_f_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_f_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_f_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) <> 'N' THEN  round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_f_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_f_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) <> 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_f_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_f_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_f_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_f_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) <> 'N' THEN round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_f_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) = 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_nf_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_nf_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_norm_nf_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) = 'N' THEN  round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_nf_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd = 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_norm_nf_val_invbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) = 'N' THEN invl.sell_qty_colli
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_nf_qty_colli,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_gsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_nf_val_gsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_nsp,4)
 ELSE NUMERIC '0.0000'
 END AS cu_sell_prom_nf_val_nsp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) = 'N' THEN round(invl.sell_val_nnbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_nf_val_nnbp,
 CASE
 WHEN invl.gm_cond_type_cd <> 0
 AND upper(invl.fnf_cd) = 'N' THEN  round(invl.sell_val_invbp,4)
 ELSE NUMERIC '0.0000'
 END   * security.nnbp_check_ind   AS cu_sell_prom_nf_val_invbp,
 substr(CAST(0 as STRING), 1, 2) AS cupr_disc_type_cd,
 round(0.0000,4) AS cu_cupr_disc_val_nsp,
 substr(CAST(0 as STRING), 1, 2) AS cupr_charge_type_cd,
 round(0.0000,4) AS cu_cupr_charge_val_nsp,
 CAST(0 as integer) AS cupr_action_id,
 CAST(0 as integer) AS cupr_action_sequence_id,
 round(invl.disc_base_val_nsp,4) AS cu_disc_base_val_nsp,
 round(invl.disc_val_nsp,4) AS cu_disc_val_nsp,
 round(invl.disc_charge_val_nsp,4) AS cu_disc_charge_val_nsp,
 round(invl.disc_base_val_nsp,4) AS cu_dnr_disc_base_val_nsp,
 round(invl.disc_val_nsp,4) AS cu_dnr_disc_val_nsp,
 round(invl.disc_charge_val_nsp,4) AS cu_dnr_disc_charge_val_nsp,
 d.date_of_day,
 invl.cust_no,
 invl.store_id,
 d.month_id,
 CAST(extract(YEAR from CAST(d.date_of_day as TIMESTAMP)) as integer) AS year_id,
 invl.advert_id,
 invl.prom_id,
 invl.return_reason_cd,
 invl.cod_price_type_cd,
 CAST(0 as integer) AS self_scan_invoice_line_cd,
 invl.disc_line_ind AS cu_disc_ind,
 invl.ean_char_no,
 invl.client_cd
 FROM
 metro-bi-dl-prep-ger-prod.cc_dwh.dw_cust_invoice_line AS invl
 INNER JOIN metro-bi-dl-prep-ger-prod.cc_dwh.dw_time_day AS d ON invl.date_of_day = d.date_of_day
 AND invl.client_cd = d.client_cd

 INNER JOIN metro-bi-dl-prep-ger-prod.security_views.security_access_control as security
 ON SESSION_USER() = security.user,
 UNNEST(access_area) as un_access_area,
 UNNEST(security.home_store_id_rl) as un_home_store_id
 WHERE ('ALL' in (un_access_area) or (un_access_area) IN ('detail_cust_check_ind' ))
 AND invl.home_store_id = un_home_store_id

# New Section

# Ex-rate

# YTM Serbia

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT * FROM `metro-bi-wb-mag-sci-s00.FinanceBI_Dataset.DW_TARGET_CURRENCY_RATE_SNAPSHOT` 
where 1=1
and client_cd = 8910
and month_id = 202209
-- and year_id =

Query is running:   0%|          |

Downloading:   0%|          |

,client_cd,currency_cd_local,currency_cd,conversion_rate,month_id,year_id,currency_Date,dwh_insert_id,exchange_Rate_Type
0,8910,CSD,EUR,0.008508392,202209,2022,2023-01-31,202302060858,XR


In [ ]:
%%bigquery --project cf-michelange-dev-sx
select * from metro-bi-wb-mag-sci-s00.FinanceBI_Dataset.DW_TARGET_CURRENCY_RATE_SNAPSHOT

In [ ]:
%%bigquery --project cf-michelange-dev-sx
insert into `metro-bi-wb-mag-sci-s00.FinanceBI_Dataset.DW_TARGET_CURRENCY_RATE_SNAPSHOT`
with 
cte0 as 
(
SELECT 
client_cd,
extract(year from date_of_day) * 100 + extract(month from date_of_day) as month_id,
extract(year from date_of_day) as year_id, 
currency_cd,  
case when extract(month from date_of_day) - extract(year from date_of_day) * 100 in (10, 11, 12) then extract(year from date_of_day)+1 else extract(year from date_of_day) end as financial_year_id,
case when extract(month from date_of_day) - extract(year from date_of_day) * 100 in (10, 11, 12) then (extract(year from date_of_day)+1)*100+(extract(month from date_of_day) - extract(year from date_of_day) * 100)-9 else (extract(year from date_of_day))*100+(extract(month from date_of_day) - extract(year from date_of_day) * 100)+3  end as financial_month_id,
conversion_rate_bc
FROM `metro-bi-wb-mag-sci-s00.FinanceBI_Dataset.dw_migra_currency_rate_XR`
),
cte1 as 
(
SELECT 
client_cd,
month_id,
year_id, 
currency_cd as currency_cd_local,  
case when month_id - year_id * 100 in (10, 11, 12) then year_id+1 else year_id end as financial_year_id,
case when month_id - year_id * 100 in (10, 11, 12) then (year_id+1)*100+(month_id - year_id * 100)-9 else (year_id)*100+(month_id - year_id * 100)+3  end as financial_month_id,
sum(1/conversion_rate_bc) as sum_conversion_rate_bc,
count(conversion_rate_bc) as cnt_conversion_rate_bc
FROM cte0 
group by client_cd, month_id, year_id, currency_cd
),
cte2 as (
select 
client_cd,
month_id,
year_id, 
currency_cd_local,  
financial_year_id,
financial_month_id,
sum(sum_conversion_rate_bc)
OVER (
    PARTITION BY client_cd, financial_year_id
    ORDER BY financial_month_id 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS suma,
sum(cnt_conversion_rate_bc)
OVER (
    PARTITION BY client_cd, financial_year_id
    ORDER BY financial_month_id 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cnt,
from cte1
)
select 
    client_cd                                                        as client_cd,
    currency_cd_local                                                as currency_cd_local,
    'EUR'                                                            as currency_cd,
    CAST(1/(suma/cnt) AS NUMERIC)                                    as conversion_rate,
    month_id                                                         as month_id,
    year_id                                                          as year_id, 
    DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 DAY)      as currency_Date,
    CAST(
         replace(substring(cast(current_date() as string),0,10), '-', '') 
         || 
         replace(substring(cast(current_time("Europe/Berlin") as string),0,5), ':', '') 
         AS INTEGER
        ) as snapshot_Date,
    'XR' as exchange_Rate_Type
from cte2
order by 
year_id desc , 
month_id desc;

# cf-mdw-int-msi-mid-dev.cc_dwh.DW_TARGET_CURRENCY_RATE_SNAPSHOT:

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select
a.month_id,
a.client_cd,
a.district_cd,
a.unit_id,
a.store_id,
ROUND(a.fsd_store_pnl_sales,2) AS fsd_store_pnl_sales,
ROUND(a.fsd_store_pnl_execution_cost,2) AS fsd_store_pnl_execution_cost,
ROUND(a.fsd_store_pnl_gross_profit,2) AS fsd_store_pnl_gross_profit,
ROUND(a.fsd_store_pnl_later_income,2) AS fsd_store_pnl_later_income,
ROUND(a.fsd_store_pnl_total_income,2) AS fsd_store_pnl_total_income,
ROUND(a.fsd_store_pnl_cost_to_serve,2) AS fsd_store_pnl_cost_to_serve,
ROUND(a.fsd_store_pnl_transport_cost,2) AS fsd_store_pnl_transport_cost,
ROUND(a.fsd_store_pnl_marketing_cost,2) AS fsd_store_pnl_marketing_cost,
ROUND(a.fsd_store_pnl_indirect_cost,2) AS fsd_store_pnl_indirect_cost,
ROUND(a.fsd_store_pnl_all_otr_drct_cost,2) AS fsd_store_pnl_all_otr_drct_cost,
ROUND(a.fsd_store_pnl_depreciation,2) AS fsd_store_pnl_depreciation,
ROUND(a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve,2) as fsd_pnl_store_pnl_incm_afr_cst_to_srv,
ROUND(((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost),2) as fsd_store_pnl_cash_profit,
ROUND((((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost) - fsd_store_pnl_indirect_cost),2) as fsd_store_pnl_ebitda,
ROUND(((((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost) - fsd_store_pnl_indirect_cost) - fsd_store_pnl_depreciation),2) as fsd_store_pnl_ebit,
'LC' as currency_ind,
coalesce(xc.currency_cd_local, 'N/A') as currency_cd,
coalesce(xc.conversion_rate, 1) as conversion_rate
from (
select
month_id,
client_cd,
district_cd,
unit_id,
store_id,
sv_180447 + sv_180448 as  fsd_store_pnl_sales,
sv_180460 as fsd_store_pnl_execution_cost,
sv_180450 + sv_180453 as fsd_store_pnl_gross_profit,
sv_180451 + sv_180454 as fsd_store_pnl_later_income,
sv_180450 + sv_180453 + sv_180451 + sv_180454 as fsd_store_pnl_total_income,
sv_180459 + sv_180460 as fsd_store_pnl_cost_to_serve,
sv_180459 as fsd_store_pnl_transport_cost,
sv_180461 as fsd_store_pnl_marketing_cost,
ROUND(sv_180469 + sv_180470 + ((sv_180447 + sv_180448)*-0.0075) + sv_180478 + sv_180479 + sv_182976 + sv_182977+sv_182978, 2) as fsd_store_pnl_indirect_cost,
sv_180476 + sv_180463 + sv_180462 + sv_182969 as fsd_store_pnl_all_otr_drct_cost,
sv_182969 + sv_182978 as fsd_store_pnl_depreciation
from ( select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi_store 
pivot(sum(selective_value) as sv 
FOR kpi_cd in ('180447', '180448','180460','180450','180453','180451','180454','180459','180461','180476','180463','180462'
, '180469','180470','180478','180479','182976','182977','182978','182969')))
) a
left join cf-mdw-int-msi-mid-dev.cc_dwh.DW_TARGET_CURRENCY_RATE_SNAPSHOT xc
on xc.client_cd = a.client_cd
and xc.month_id = a.month_id
UNION ALL
select
a.month_id,
a.client_cd,
a.district_cd,
a.unit_id,
a.store_id,
ROUND(a.fsd_store_pnl_sales* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_sales,
ROUND(a.fsd_store_pnl_execution_cost* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_execution_cost,
ROUND(a.fsd_store_pnl_gross_profit* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_gross_profit,
ROUND(a.fsd_store_pnl_later_income* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_later_income,
ROUND(a.fsd_store_pnl_total_income* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_total_income,
ROUND(a.fsd_store_pnl_cost_to_serve* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_cost_to_serve,
ROUND(a.fsd_store_pnl_transport_cost* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_transport_cost,
ROUND(a.fsd_store_pnl_marketing_cost* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_marketing_cost,
ROUND(a.fsd_store_pnl_indirect_cost* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_indirect_cost,
ROUND(a.fsd_store_pnl_all_otr_drct_cost* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_all_otr_drct_cost,
ROUND(a.fsd_store_pnl_depreciation* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_depreciation,
ROUND((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve) * coalesce(xc.conversion_rate, 1),2)  as fsd_pnl_store_pnl_incm_afr_cst_to_srv,
ROUND(((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost) * coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_cash_profit,
ROUND((((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost) - fsd_store_pnl_indirect_cost)* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_ebitda,
ROUND(((((a.fsd_store_pnl_total_income - fsd_store_pnl_cost_to_serve ) - fsd_store_pnl_marketing_cost - fsd_store_pnl_all_otr_drct_cost) - fsd_store_pnl_indirect_cost) - fsd_store_pnl_depreciation)* coalesce(xc.conversion_rate, 1),2) as fsd_store_pnl_ebit,
'GC' as currency_ind,
coalesce(xc.currency_cd, 'EUR') as currency_cd,
coalesce(xc.conversion_rate, 1) as conversion_rate
from (
select
month_id,
client_cd,
district_cd,
unit_id,
store_id,
sv_180447 + sv_180448 as  fsd_store_pnl_sales,
sv_180460 as fsd_store_pnl_execution_cost,
sv_180450 + sv_180453 as fsd_store_pnl_gross_profit,
sv_180451 + sv_180454 as fsd_store_pnl_later_income,
sv_180450 + sv_180453 + sv_180451 + sv_180454 as fsd_store_pnl_total_income,
sv_180459 + sv_180460 as fsd_store_pnl_cost_to_serve,
sv_180459 as fsd_store_pnl_transport_cost,
sv_180461 as fsd_store_pnl_marketing_cost,
ROUND(sv_180469 + sv_180470 + ((sv_180447 + sv_180448)*-0.0075) + sv_180478 + sv_180479 + sv_182976 + sv_182977+sv_182978, 2) as fsd_store_pnl_indirect_cost,
sv_180476 + sv_180463 + sv_180462 + sv_182969 as fsd_store_pnl_all_otr_drct_cost,
sv_182969 + sv_182978 as fsd_store_pnl_depreciation
from ( select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi_store 
pivot(sum(selective_value) as sv 
FOR kpi_cd in ('180447', '180448','180460','180450','180453','180451','180454','180459','180461','180476','180463','180462'
, '180469','180470','180478','180479','182976','182977','182978','182969')))
) a
left join cf-mdw-int-msi-mid-dev.cc_dwh.DW_TARGET_CURRENCY_RATE_SNAPSHOT xc
on xc.client_cd = a.client_cd
and xc.month_id = a.month_id


Query is running:   0%|          |

Downloading:   0%|          |

,month_id,client_cd,district_cd,unit_id,store_id,fsd_store_pnl_sales,fsd_store_pnl_execution_cost,fsd_store_pnl_gross_profit,fsd_store_pnl_later_income,fsd_store_pnl_total_income,...,fsd_store_pnl_indirect_cost,fsd_store_pnl_all_otr_drct_cost,fsd_store_pnl_depreciation,fsd_pnl_store_pnl_incm_afr_cst_to_srv,fsd_store_pnl_cash_profit,fsd_store_pnl_ebitda,fsd_store_pnl_ebit,currency_ind,currency_cd,conversion_rate
0,202010,7240,11475,1544,6,3.705075e+05,-3421.34,51030.16,36033.78,87063.94,...,-33812.78,15882.01,1417.88,113911.01,115885.39,149698.17,148280.29,GC,EUR,1.000000000
1,202010,3480,39,1291,12,1.052223e+08,-5563678.05,17735647.97,5488757.62,23224405.59,...,-5806739.88,NaN,NaN,35740684.01,NaN,NaN,NaN,GC,EUR,1.000000000
2,202010,1000,92,1038,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC,EUR,1.000000000
3,202010,3480,11354,1294,22,4.725391e+07,-2674211.53,8380550.65,2528406.85,10908957.50,...,-2513009.14,NaN,NaN,17643809.66,NaN,NaN,NaN,GC,EUR,1.000000000
4,202010,7240,81,1551,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GC,EUR,1.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38131,202208,2760,11479,1247,661,0.000000e+00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,LC,EUR,1.000000000
38132,202209,2760,11479,2819,503,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC,EUR,1.000000000
38133,202209,2760,11479,1218,638,0.000000e+00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,LC,EUR,1.000000000
38134,202209,2760,11479,1244,663,0.000000e+00,0.00,0.00,0.00,0.00,...,40.80,10.43,-2.73,-2530.03,-2544.49,-2585.29,-2582.56,LC,EUR,1.000000000


# Business Glossary - KPI definitions

Hi Yesim, Srdjan,

Forgot to mention to you,
We need some additional fields per “KPI” for visualization and data treatment.
I am willing to maintain these parts – but I am wondering where is the best place to maintain them.
Should we do this in Erwin as part of Business Glossary KPI template? Should we just have a data table in GBQ where we directly maintain such things?

-	KPI ID
-	KPI Numerator Type (Currency, Units)
-	KPI Denominator Type (Currency, Units)
-	KPI Type (Currency, Units)*
-	KPI Format (Currency, %, Index Ratio)
-	KPI Custom Suffix (e.g. Per sqm, Per Day, Per Month, Per Person, KM Per Hour, etc) (it can be left blank)
-	KPI Custom Suffix Short (e.g. watt, psqm, pd, pm, ppr, kph)
-	Flag KPI Display Sign Reverse (Yes/No)#
-	Formula for the KPI ^ 
-	KPI Yearly Target available (Yes/No)
-	KPI Monthly Target available (Yes/No)
-	KPI Display/Hide Flag@

* Note1:
-	A KPI with Currency Numerator and Currency Denominator has to have final type also as Currency. KPI which is a mix of Currency divide by Units is also resulting in Currency format KPI (e.g. Sales per sq m, Stock per store, Sales per day, Sales per Sales-Force Manager. Etc.)
-	KPI which is based on Units Numerator and Units Denominator has a KPI which is Units based (i.e. it cannot be a Currency KPI) and such a Unit KPI can have different formats to display as a Percent %, Index Ratio, etc.

#Note2:
-	In certain cases the KPI actual and target value is inputted with a reverse sign (minus instead of plus) then the flag has to be set to reverse the data sign before displaying on the report.)

^Note3:
-	In some cases the KPI is not exactly Numerator/Denominator. In this case the field can be used to describe the logic. E.g. Productivity = Sales in mn / FTE / per day *1 mn (depending on Period)

@Note4:
-	In some cases the KPI must not be displayed in the sCore dashboard at all. (e.g. when the KPI is defined but the backend-frontend for Actual data is not available or not checked). Therefore, we need a master data flag which will indicates if the KPI should be shown in the front end or hidden.







# Data Modeling - Star schema

In [ ]:
%%bigquery --project cf-michelange-dev-sx



/****************************************************************** CUSTOMER FLATTEN DIM ********************************************************************************************************************/


DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim`  
PARTITION BY PARTITION_DATE
CLUSTER BY client_cd, cust_no, cust_hts_desc, branch_group_desc

AS (
SELECT 
CURRENT_DATE() AS partition_date,
cb.client_cd,
c.cust_no,
cust.unique_cust_no,
c.home_store_id,
/*Level 1*/
cust_hts.cust_hts_id,
cust_hts_desc,
/*Level 2*/
bf.branch_main_group_id,
cmb.branch_main_group_desc,
/*Level 3*/
cbg.branch_group_id,
cbg.branch_group_desc,
/*Level 4*/
cb.branch_family_id,
bf.branch_family_desc
--cust_sales_channel_cd_desc 
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer` c 
--LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer` uc ON uc.cust_no = c.cust_no AND uc.home_store_id = c.home_store_id AND uc.client_cd = c.client_cd
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer_unique` cust ON c.cust_no = cust.unique_cust_no AND 
c.unique_home_store_id = cust.unique_home_store_id AND c.client_cd = cust.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch` cb ON c.branch_id = cb.branch_id AND c.client_cd = cb.client_cd 
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_family` bf ON cb.branch_family_id = bf.branch_family_id AND cb.cust_assort_section_id = bf.cust_assort_section_id AND cb.client_cd = bf.client_cd                     
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_main_group` cmb ON cmb.branch_main_group_id = bf.branch_main_group_id AND cmb.client_cd = bf.client_cd 
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_group` cbg ON cbg.branch_group_id = cb.branch_id AND cbg.client_cd = cb.client_cd 
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_v_cust_assort_section` cas ON bf.cust_assort_section_id = cas.cust_assort_section_id AND bf.client_cd = cas.client_cd                                            
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_v_cust_hts` cust_hts ON (cas.cust_hts_id = cust_hts.cust_hts_id) AND cas.client_cd = cust_hts.client_cd 
);

/*
where 
cust_hts.cust_hts_id=1
and bf.branch_main_group_id=10
and cbg.branch_group_id='101'
-- and cb.branch_family_id=
limit 100
*/


 /****************************************************************** PRODUCT FLATTEN DIM ********************************************************************************************************************/


DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.product_dim`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.product_dim`  

PARTITION BY PARTITION_DATE
CLUSTER BY client_cd, fnf_cd, stratbuy_division_int_id, stratbuy_domain_int_id AS

(SELECT
CURRENT_DATE() AS partition_date
,art.client_cd
,ar.fnf_cd
,ar.fnf_desc
,mmc.assort_section_id
,mmc.assort_section_desc
,sbdvint.stratbuy_division_int_id
,sbdvint.stratbuy_division_int_desc
,sbdmint.stratbuy_domain_int_id
,sbdmint.stratbuy_domain_int_desc
,ar.mge_main_cat_id
,ar.mge_main_cat_desc
,mmc.mge_main_cat_desc_tl
,ar.mge_cat_id
,ar.mge_cat_desc
,ar.art_no
,ar.art_name
,art.var_tu_key

FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_art_var_tu` AS art
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_article` ar ON art.art_no = ar.art_no and art.client_cd = ar.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cat_mge_main_cat` mmc on (ar.mge_main_cat_id = mmc.mge_main_cat_id) and ar.client_cd = mmc.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cat_mmc_stratbuy_int`  mmcsbint    on (ar.mge_main_cat_id = mmcsbint.mge_main_cat_id) and ar.client_cd = mmcsbint.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_stratbuy_domain_int`  sbdmint     on (mmcsbint.stratbuy_domain_int_id = sbdmint.stratbuy_domain_int_id) and sbdmint.client_cd = mmcsbint.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_stratbuy_division_int`  sbdvint     on (sbdmint.stratbuy_division_int_id = sbdvint.stratbuy_division_int_id) and sbdvint.client_cd = sbdmint.client_cd
);




/***************************************************** TEMP TABLES ****************************************************************************************/



DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`  AS
(SELECT DISTINCT cus.cust_no, cus.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON cus.cust_no = inv.cust_no AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`  AS
(SELECT ar.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP` art ON ar.art_no = art.art_no AND ar.client_cd = art.client_cd
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);




/***************************************************** TEMP TABLES ****************************************************************************************/



DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP`  AS
(SELECT DISTINCT ar.art_no,ar.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON ar.art_no = inv.art_no AND ar.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);


DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`  AS
(SELECT ar.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP` art ON ar.art_no = art.art_no AND ar.client_cd = art.client_cd
);




DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`  AS
(SELECT DISTINCT cus.cust_no, cus.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON cus.cust_no = inv.cust_no AND cus.client_cd = inv.client_cd AND cus.home_store_id = inv.home_store_id
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);






/***************************************************** NEW MSI INVOICE TABLE ****************************************************************************************/



DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`

PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS
(
(SELECT
 inv.PARTITION_DATE
,INVOICE_GLOBAL_ID
,inv.client_cd
,inv.cust_no
,inv.cust_hts_desc
,inv.month_id
,inv.year_id
,inv.invoice_id
,inv.store_id
,inv.home_store_id
,inv.store_desc
,inv.mge_main_cat_desc
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.mge_main_cat_desc_tl
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.fsd_ind
,currency_code_ind
,'LC''
,inv.sell_val_nsp
,drops
,visits
,invoice_lines
,avgerage_sales_per_drop
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv

UNION ALL

(SELECT
 inv.PARTITION_DATE
,INVOICE_GLOBAL_ID
,inv.client_cd
,inv.cust_no
,inv.cust_hts_desc
,inv.month_id
,inv.year_id
,inv.invoice_id
,inv.store_id
,inv.home_store_id
,inv.store_desc
,inv.mge_main_cat_desc
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.mge_main_cat_desc_tl
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.fsd_ind
,currency_code_ind
,'EUR'
,CASE WHEN conversion_rate <> 0 AND conversion_rate <> 1 THEN inv.sell_val_nsp ELSE inv.sell_val_nsp * inv.conversion_rate END AS sell_val_nsp
,drops
,visits
,invoice_lines
,avgerage_sales_per_drop
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv
);




/***************************************************** NEW MSI INVOICE TEMP TABLE ****************************************************************************************/
--Invoive Lines (Local Currency)


DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP`
PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS

(
SELECT CURRENT_DATE() AS PARTITION_DATE
,CONCAT(inv.client_cd,'-',inv.invoice_id) AS INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id
,inv.year_id
,inv.invoice_id
,cus.cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,cus.cust_hts_id

,cus.cust_hts_desc

/*Level 2*/

,cus.branch_main_group_id

,cus.branch_main_group_desc

/*Level 3*/

,cus.branch_group_id

,cus.branch_group_desc

/*Level 4*/

,cus.branch_family_id
,cus.branch_family_desc

,ar.fnf_cd
,ar.fnf_desc
,ar.assort_section_id
,ar.assort_section_desc
,ar.stratbuy_division_int_id
,ar.stratbuy_division_int_desc
,ar.stratbuy_domain_int_id
,ar.stratbuy_domain_int_desc
,ar.mge_main_cat_id
,ar.mge_main_cat_desc
,ar.mge_main_cat_desc_tl
,ar.mge_cat_id
,ar.mge_cat_desc
,CASE WHEN inv.fulfillment_model_cd IN (2,5,6) THEN 1 ELSE 0 END AS fsd_ind
,'LC' AS currency_code_ind
,SUM(inv.sell_val_nsp) AS sell_val_nsp
,COUNT(*) AS invoice_lines
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
,SUM(sell_val_nsp - sell_val_nnbp) AS satistical_sales_margin
,SUM(inv.sell_val_nnbp * inv.prof_later_income_pct) /* * sec_u.inv_prof_inc_det_check_ind */ AS cu_prof_later_income_val

FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO` ar ON ar.art_no = inv.art_no 
AND inv.var_tu_key = ar.var_tu_key AND ar.client_cd = inv.client_cd
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no 
AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_co_store` st ON st.store_id = inv.store_id 
AND cus.home_store_id = inv.home_store_id AND st.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31)
;








/***************************************************** NEW MSI INVOICE TABLE ****************************************************************************************/

--Invoive Lines (Local Currency)
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`
PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS

((SELECT 
inv.PARTITION_DATE
,inv.INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id,inv.year_id
,inv.invoice_id
,inv.cust_no
,inv.unique_cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,inv.cust_hts_id

,inv.cust_hts_desc

/*Level 2*/

,inv.branch_main_group_id

,inv.branch_main_group_desc

/*Level 3*/

,inv.branch_group_id

,inv.branch_group_desc

/*Level 4*/

,inv.branch_family_id
,inv.branch_family_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.mge_main_cat_id
,inv.mge_main_cat_desc
,inv.mge_main_cat_desc_tl
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fsd_ind
,'LC' AS currency_code_ind
,invoice_lines
,drops
,satistical_sales_margin
,inv.sell_val_nsp

FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv)

UNION ALL

(SELECT 
inv.PARTITION_DATE
,inv.INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id,inv.year_id
,inv.invoice_id
,inv.cust_no
,inv.unique_cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,inv.cust_hts_id

,inv.cust_hts_desc

/*Level 2*/

,inv.branch_main_group_id

,inv.branch_main_group_desc

/*Level 3*/

,inv.branch_group_id

,inv.branch_group_desc

/*Level 4*/

,inv.branch_family_id
,inv.branch_family_desc

,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.mge_main_cat_id
,inv.mge_main_cat_desc
,inv.mge_main_cat_desc_tl
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fsd_ind
,'BC' AS currency_code_ind
,invoice_lines
,drops
,satistical_sales_margin
,inv.sell_val_nsp * XR.conversion_rate AS sell_val_nsp
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv
INNER JOIN `cf-mdw-int-msi-mid-dev.cc_dwh.DW_TARGET_CURRENCY_RATE_SNAPSHOT` AS XR ON inv.client_cd = XR.client_cd AND XR.month_id = 202209));




/*************************************************************  DROPS  **********************************************************************/



CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_MONTH_DROPS`

PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, CUST_NO, HOME_STORE_ID AS

(
-- DROPS
SELECT
 CURRENT_DATE() AS PARTITION_DATE
,inv.client_cd
,inv.cust_no
,inv.home_store_id
,inv.store_id
,cus.cust_hts_desc
,inv.month_id
,inv.year_id
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910)) 
GROUP BY 1,2,3,4,5,6,7,8
);



CREATE OR REPLACE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_MONTH_DROPS_FNF`

PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, CUST_NO, HOME_STORE_ID AS

(
-- DROPS

SELECT
 CURRENT_DATE() AS PARTITION_DATE
,inv.client_cd
,inv.cust_no
,inv.home_store_id
,inv.store_id
,cus.cust_hts_desc
,inv.fnf_cd
,inv.month_id
,inv.year_id
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910)) 
GROUP BY 1,2,3,4,5,6,7,8,9
);

-- Final code with drops, sales, etc in one materialized view built on top of lowest level data at which team calculated drops

Drop materialized view `metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops` ;

CREATE Materialized view `metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops`
AS
(
SELECT
  invline.store_id,
  invline.cust_hts_id,
  invline.cust_hts_desc,
  invline.branch_main_group_id,
  --invline.branch_main_group_desc,
  invline.month_id,
  invline.year_id,
  invline.fnf_cd,
  --invline.fnf_desc,
  invline.stratbuy_division_int_id,
  --invline.stratbuy_division_int_desc,
  invline.fsd_ind,
  invline.currency_code_ind,
  invline.client_cd,
  sum(invline.sell_val_nsp) as sales,
  sum(invline.invoice_lines) as lines,
  sum(invline.drops) as drops,
  sum(invline.satistical_sales_margin) as statistical_sales_margin,
  sum(invline.cu_prof_later_income_val) as later_income
FROM
  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` invline
GROUP BY
  1,2,3,4,5,6,7,8,9,10,11
)

-- View on top of the above MV
create or replace view cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_aggr_custl2_prodl2_sales_drops
as(
select 
a.store_id,
a.cust_hts_id,
a.cust_hts_desc,
a.branch_main_group_id,
a.branch_main_group_desc,
a.month_id,
a.year_id,
a.fnf_cd,
a.fnf_desc,
a.stratbuy_division_int_id,
a.stratbuy_division_int_desc,
a.fsd_ind,
a.currency_code_ind,
a.client_cd,
a.sales,
a.lines,
a.drops,
a.statistical_sales_margin,
a.later_income from metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops as a )


## CUSTOMER FLATTEN DIM 

In [ ]:
%%bigquery --project cf-michelange-dev-sx

/****************************************************************** CUSTOMER FLATTEN DIM ********************************************************************************************************************/

/*
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim`  
PARTITION BY PARTITION_DATE
CLUSTER BY client_cd, cust_no, cust_hts_desc, branch_group_desc

AS (
*/

SELECT DISTINCT
concat(cust_hts_id,'_',cust_hts_desc) as HTS
FROM (

SELECT 
CURRENT_DATE() AS partition_date,
cb.client_cd,
c.cust_no,
cust.unique_cust_no,
c.home_store_id,
/*Level 1*/
cust_hts.cust_hts_id,
cust_hts_desc,
/*Level 2*/
bf.branch_main_group_id,
cmb.branch_main_group_desc,
/*Level 3*/
cbg.branch_group_id,
cbg.branch_group_desc,
/*Level 4*/
cb.branch_family_id,
bf.branch_family_desc
--cust_sales_channel_cd_desc 
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer` c 
--LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer` uc ON uc.cust_no = c.cust_no AND uc.home_store_id = c.home_store_id AND uc.client_cd = c.client_cd
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_customer_unique` cust ON c.cust_no = cust.unique_cust_no AND 
c.unique_home_store_id = cust.unique_home_store_id AND c.client_cd = cust.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch` cb ON c.branch_id = cb.branch_id AND c.client_cd = cb.client_cd 
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_family` bf ON cb.branch_family_id = bf.branch_family_id AND cb.cust_assort_section_id = bf.cust_assort_section_id AND cb.client_cd = bf.client_cd                     
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_main_group` cmb ON cmb.branch_main_group_id = bf.branch_main_group_id AND cmb.client_cd = bf.client_cd 
LEFT JOIN  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_branch_group` cbg ON cbg.branch_group_id = cb.branch_id AND cbg.client_cd = cb.client_cd 
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_v_cust_assort_section` cas ON bf.cust_assort_section_id = cas.cust_assort_section_id AND bf.client_cd = cas.client_cd                                            
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_v_cust_hts` cust_hts ON (cas.cust_hts_id = cust_hts.cust_hts_id) AND cas.client_cd = cust_hts.client_cd 

/*
where 
cust_hts.cust_hts_id=1
and bf.branch_main_group_id=10
and cbg.branch_group_id='101'
-- and cb.branch_family_id=
limit 100
*/

) a
order by 1
--);


Query is running:   0%|          |

Downloading:   0%|          |

,HTS
0,None
1,1_HoReCa
2,2_Trader
3,3_SCO


## PRODUCT FLATTEN DIM

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/***** PRODUCT FLATTEN DIM ******/
/*
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.product_dim`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.product_dim`  
PARTITION BY PARTITION_DATE
CLUSTER BY client_cd, fnf_cd, stratbuy_division_int_id, stratbuy_domain_int_id AS
*/
-- (
  SELECT
CURRENT_DATE() AS partition_date
,art.client_cd
,ar.fnf_cd
,ar.fnf_desc
,mmc.assort_section_id
,mmc.assort_section_desc
,sbdvint.stratbuy_division_int_id
,sbdvint.stratbuy_division_int_desc
,sbdmint.stratbuy_domain_int_id
,sbdmint.stratbuy_domain_int_desc
,ar.mge_main_cat_id
,ar.mge_main_cat_desc
,mmc.mge_main_cat_desc_tl
,ar.mge_cat_id
,ar.mge_cat_desc
,ar.art_no
,ar.art_name
,art.var_tu_key
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_art_var_tu` AS art
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_article` ar ON art.art_no = ar.art_no and art.client_cd = ar.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cat_mge_main_cat` mmc on (ar.mge_main_cat_id = mmc.mge_main_cat_id) and ar.client_cd = mmc.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cat_mmc_stratbuy_int` mmcsbint on (ar.mge_main_cat_id = mmcsbint.mge_main_cat_id) and ar.client_cd = mmcsbint.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_stratbuy_domain_int` sbdmint on (mmcsbint.stratbuy_domain_int_id = sbdmint.stratbuy_domain_int_id) and sbdmint.client_cd = mmcsbint.client_cd
LEFT JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_stratbuy_division_int` sbdvint on (sbdmint.stratbuy_division_int_id = sbdvint.stratbuy_division_int_id) and sbdvint.client_cd = sbdmint.client_cd
--);


Query is running:   0%|          |

Downloading:   0%|          |

,partition_date,client_cd,fnf_cd,fnf_desc,assort_section_id,assort_section_desc,stratbuy_division_int_id,stratbuy_division_int_desc,stratbuy_domain_int_id,stratbuy_domain_int_desc,mge_main_cat_id,mge_main_cat_desc,mge_main_cat_desc_tl,mge_cat_id,mge_cat_desc,art_no,art_name,var_tu_key
0,2023-02-21,8910,F,FOOD,F,FOOD,80,FRESH,114,DEEP FROZEN,260,ICE CREAM,ICE CREAM,60,ICE CREAM TUBS/BULK,304039,1L SLADOLED FINESTRO BISCUITTO FF,1001
1,2023-02-21,8910,F,FOOD,F,FOOD,80,FRESH,114,DEEP FROZEN,260,ICE CREAM,ICE CREAM,60,ICE CREAM TUBS/BULK,304030,1500ML SLADOLED FRUIT TRIO F.FOOD,1001
2,2023-02-21,8910,F,FOOD,F,FOOD,80,FRESH,114,DEEP FROZEN,260,ICE CREAM,ICE CREAM,60,ICE CREAM TUBS/BULK,304003,1500ML FINE FOOD DELICIOUS TRIO,1001
3,2023-02-21,8910,?,None,?,undefined / undefiniert,-1,Unallocated,-1,Unallocated,-1,None,undefined,-1,None,-388637,unbekannt,1001
4,2023-02-21,8910,F,FOOD,F,FOOD,75,DRY,152,EDIBLE GROCERY,396,COLD SAUCES,COLD SAUCES,20,MAYONNAISE,345118,200ML MAJONEZ CLASSIC DIJAMANT #,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610883,2023-02-21,2760,F,FOOD,1,FOOD,75,DRY,153,CONFECTIONARY AND SNACKS,402,SCHOKOLADE,CHOCOLATE,20,RIEGEL/LOSE/SNACKS,846138,250g=9x SUPER DICKMANNS,1004
1610884,2023-02-21,2760,F,FOOD,1,FOOD,75,DRY,153,CONFECTIONARY AND SNACKS,402,SCHOKOLADE,CHOCOLATE,20,RIEGEL/LOSE/SNACKS,846138,250g=9x SUPER DICKMANNS,1008
1610885,2023-02-21,2760,F,FOOD,1,FOOD,75,DRY,153,CONFECTIONARY AND SNACKS,402,SCHOKOLADE,CHOCOLATE,20,RIEGEL/LOSE/SNACKS,846138,250g=9x SUPER DICKMANNS,1006
1610886,2023-02-21,2760,F,FOOD,1,FOOD,75,DRY,153,CONFECTIONARY AND SNACKS,402,SCHOKOLADE,CHOCOLATE,20,RIEGEL/LOSE/SNACKS,846138,250g=9x SUPER DICKMANNS,1007


## TEMP TABLES

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/***************************************************** TEMP TABLES ****************************************************************************************/
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`  AS
(SELECT DISTINCT cus.cust_no, cus.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON cus.cust_no = inv.cust_no AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`  AS
(SELECT ar.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP` art ON ar.art_no = art.art_no AND ar.client_cd = art.client_cd
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/***************************************************** TEMP TABLES ****************************************************************************************/
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP`  AS
(SELECT DISTINCT ar.art_no,ar.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON ar.art_no = inv.art_no AND ar.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO`  AS
(SELECT ar.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.artical_dim` ar
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO_TEMP` art ON ar.art_no = art.art_no AND ar.client_cd = art.client_cd
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP`  AS
(SELECT DISTINCT cus.cust_no, cus.client_cd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv ON cus.cust_no = inv.cust_no AND cus.client_cd = inv.client_cd AND cus.home_store_id = inv.home_store_id
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
);

DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO`  AS
(SELECT cus.*
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.customer_dim` cus
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO_TEMP` cust ON cus.cust_no = cust.cust_no AND cus.client_cd = cust.client_cd
);

## NEW MSI INVOICE TABLE

In [ ]:
/***************************************************** NEW MSI INVOICE TABLE ****************************************************************************************/
%%bigquery --project cf-michelange-dev-sx
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`
PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS
(
(SELECT
 inv.PARTITION_DATE
,INVOICE_GLOBAL_ID
,inv.client_cd
,inv.cust_no
,inv.cust_hts_desc
,inv.month_id
,inv.year_id
,inv.invoice_id
,inv.store_id
,inv.home_store_id
,inv.store_desc
,inv.mge_main_cat_desc
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.mge_main_cat_desc_tl
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.fsd_ind
,currency_code_ind
,'LC''
,inv.sell_val_nsp
,drops
,visits
,invoice_lines
,avgerage_sales_per_drop
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv

UNION ALL

(SELECT
 inv.PARTITION_DATE
,INVOICE_GLOBAL_ID
,inv.client_cd
,inv.cust_no
,inv.cust_hts_desc
,inv.month_id
,inv.year_id
,inv.invoice_id
,inv.store_id
,inv.home_store_id
,inv.store_desc
,inv.mge_main_cat_desc
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.mge_main_cat_desc_tl
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.fsd_ind
,currency_code_ind
,'EUR'
,CASE WHEN conversion_rate <> 0 AND conversion_rate <> 1 THEN inv.sell_val_nsp ELSE inv.sell_val_nsp * inv.conversion_rate END AS sell_val_nsp
,drops
,visits
,invoice_lines
,avgerage_sales_per_drop
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv
);

## NEW MSI INVOICE TEMP TABLE

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/********************** NEW MSI INVOICE TEMP TABLE **********************/
--Invoive Lines (Local Currency)
/*
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP`;
CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP`
PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS
(
*/  

SELECT CURRENT_DATE() AS PARTITION_DATE
,CONCAT(inv.client_cd,'-',inv.invoice_id) AS INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id
,inv.year_id
,inv.invoice_id
,cus.cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,cus.cust_hts_id
,cus.cust_hts_desc
/*Level 2*/
,cus.branch_main_group_id
,cus.branch_main_group_desc
/*Level 3*/
,cus.branch_group_id
,cus.branch_group_desc
/*Level 4*/
,cus.branch_family_id
,cus.branch_family_desc
,ar.fnf_cd
,ar.fnf_desc
,ar.assort_section_id
,ar.assort_section_desc
,ar.stratbuy_division_int_id
,ar.stratbuy_division_int_desc
,ar.stratbuy_domain_int_id
,ar.stratbuy_domain_int_desc
,ar.mge_main_cat_id
,ar.mge_main_cat_desc
,ar.mge_main_cat_desc_tl
,ar.mge_cat_id
,ar.mge_cat_desc
,CASE WHEN inv.fulfillment_model_cd IN (2,5,6) THEN 1 ELSE 0 END AS fsd_ind
,'LC' AS currency_code_ind
,SUM(inv.sell_val_nsp) AS sell_val_nsp
,COUNT(*) AS invoice_lines
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
,SUM(sell_val_nsp - sell_val_nnbp) AS satistical_sales_margin
,SUM(inv.sell_val_nnbp * inv.prof_later_income_pct) /* * sec_u.inv_prof_inc_det_check_ind */ AS cu_prof_later_income_val
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_ARTICLE_NO` ar ON ar.art_no = inv.art_no 
AND inv.var_tu_key = ar.var_tu_key AND ar.client_cd = inv.client_cd
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no 
AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
LEFT JOIN`metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_co_store` st ON st.store_id = inv.store_id 
AND cus.home_store_id = inv.home_store_id AND st.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910))
AND inv.date_of_day = current_date()-31
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
--)
;

Query is running:   0%|          |

Downloading: |          |

,PARTITION_DATE,INVOICE_GLOBAL_ID,client_cd,month_id,year_id,invoice_id,cust_no,home_store_id,store_id,cust_hts_id,...,mge_main_cat_desc_tl,mge_cat_id,mge_cat_desc,fsd_ind,currency_code_ind,sell_val_nsp,invoice_lines,drops,satistical_sales_margin,cu_prof_later_income_val


In [1]:
%%bigquery --project cf-michelange-dev-sx
-- select * from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` 
select * 
--max(PARTITION_DATE) as max_date 
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` 
where PARTITION_DATE = '2023-02-21'
and client_cd=8910
and month_id in (202209,202109)
LIMIT 10
-- where PARTITION_DATE = CURRENT_DATE()-5*60;


ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f7f7a1429d0>)


In [ ]:
/***************************************************** NEW MSI INVOICE TABLE ****************************************************************************************/

--Invoive Lines (Local Currency)
DROP TABLE IF EXISTS `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`;

CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`
PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, STORE_ID, CUST_NO AS

((SELECT 
inv.PARTITION_DATE
,inv.INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id,inv.year_id
,inv.invoice_id
,inv.cust_no
,inv.unique_cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,inv.cust_hts_id
,inv.cust_hts_desc
/*Level 2*/
,inv.branch_main_group_id
,inv.branch_main_group_desc
/*Level 3*/
,inv.branch_group_id
,inv.branch_group_desc
/*Level 4*/
,inv.branch_family_id
,inv.branch_family_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.mge_main_cat_id
,inv.mge_main_cat_desc
,inv.mge_main_cat_desc_tl
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fsd_ind
,'LC' AS currency_code_ind
,invoice_lines
,drops
,satistical_sales_margin
,inv.sell_val_nsp
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv)
UNION ALL
(
SELECT 
inv.PARTITION_DATE
,inv.INVOICE_GLOBAL_ID 
,inv.client_cd
,inv.month_id,inv.year_id
,inv.invoice_id
,inv.cust_no
,inv.unique_cust_no
,inv.home_store_id
,inv.store_id
/*Level 1*/
,inv.cust_hts_id
,inv.cust_hts_desc
/*Level 2*/
,inv.branch_main_group_id
,inv.branch_main_group_desc
/*Level 3*/
,inv.branch_group_id
,inv.branch_group_desc
/*Level 4*/
,inv.branch_family_id
,inv.branch_family_desc
,inv.fnf_cd
,inv.fnf_desc
,inv.assort_section_id
,inv.assort_section_desc
,inv.stratbuy_division_int_id
,inv.stratbuy_division_int_desc
,inv.stratbuy_domain_int_id
,inv.stratbuy_domain_int_desc
,inv.mge_main_cat_id
,inv.mge_main_cat_desc
,inv.mge_main_cat_desc_tl
,inv.mge_cat_id
,inv.mge_cat_desc
,inv.fsd_ind
,'BC' AS currency_code_ind
,invoice_lines
,drops
,satistical_sales_margin
,inv.sell_val_nsp * XR.conversion_rate AS sell_val_nsp
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` inv
INNER JOIN `cf-mdw-int-msi-mid-dev.cc_dwh.DW_TARGET_CURRENCY_RATE_SNAPSHOT` AS XR ON inv.client_cd = XR.client_cd AND XR.month_id = 202209
));

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select 
*
-- max(PARTITION_DATE) as max_date 
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE`
--/*
where PARTITION_DATE = '2023-02-19'
LIMIT 10
--*/

Query is running:   0%|          |

Downloading:   0%|          |

,PARTITION_DATE,INVOICE_GLOBAL_ID,client_cd,month_id,year_id,invoice_id,cust_no,home_store_id,store_id,cust_hts_id,...,mge_main_cat_desc,mge_main_cat_desc_tl,mge_cat_id,mge_cat_desc,fsd_ind,currency_code_ind,invoice_lines,drops,satistical_sales_margin,sell_val_nsp
0,2023-02-19,2760-414814278,2760,202008,2020,414814278.00000000000000000000000000000000000000,83312,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,1,1,3.807500000,14.340000000
1,2023-02-19,2760-416576151,2760,202008,2020,416576151.00000000000000000000000000000000000000,79178,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,4,1,21.194400000,68.300000000
2,2023-02-19,2760-414304356,2760,202008,2020,414304356.00000000000000000000000000000000000000,47023,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,1,1,0.809500000,10.300000000
3,2023-02-19,2760-415808902,2760,202008,2020,415808902.00000000000000000000000000000000000000,64100,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,5,1,11.918300000,50.700000000
4,2023-02-19,2760-415495326,2760,202008,2020,415495326.00000000000000000000000000000000000000,60241,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,1,1,5.560000000,16.090000000
5,2023-02-19,2760-416668025,2760,202008,2020,416668025.00000000000000000000000000000000000000,50498,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,2,1,-2.275000000,7.040000000
6,2023-02-19,2760-415376406,2760,202008,2020,415376406.00000000000000000000000000000000000000,90595,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,1,1,3.755700000,12.510000000
7,2023-02-19,2760-415499883,2760,202008,2020,415499883.00000000000000000000000000000000000000,82909,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,3,1,4.976400000,19.500000000
8,2023-02-19,2760-414434178,2760,202008,2020,414434178.00000000000000000000000000000000000000,64780,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,LC,3,1,11.265000000,39.300000000
9,2023-02-19,2760-416340311,2760,202008,2020,416340311.00000000000000000000000000000000000000,64780,13,13,3,...,SCHWEIN FR.,FRESH PORK,14,EU,0,BC,3,1,9.510000000,36.510000000


In [ ]:

/*************************************************************  DROPS  **********************************************************************/



CREATE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_MONTH_DROPS`

PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, CUST_NO, HOME_STORE_ID AS

(
-- DROPS
SELECT
 CURRENT_DATE() AS PARTITION_DATE
,inv.client_cd
,inv.cust_no
,inv.home_store_id
,inv.store_id
,cus.cust_hts_desc
,inv.month_id
,inv.year_id
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910)) 
GROUP BY 1,2,3,4,5,6,7,8
);



CREATE OR REPLACE TABLE `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_MONTH_DROPS_FNF`

PARTITION BY PARTITION_DATE
CLUSTER BY CLIENT_CD, MONTH_ID, CUST_NO, HOME_STORE_ID AS

(
-- DROPS

SELECT
 CURRENT_DATE() AS PARTITION_DATE
,inv.client_cd
,inv.cust_no
,inv.home_store_id
,inv.store_id
,cus.cust_hts_desc
,inv.fnf_cd
,inv.month_id
,inv.year_id
,COUNT (distinct CONCAT(inv.cust_no,'-',inv.home_store_id,'-',inv.date_of_day,'-',inv.store_id)) AS drops
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line` inv
INNER JOIN `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_CUSTOMER_NO` cus ON cus.cust_no = inv.cust_no AND cus.home_store_id = inv.home_store_id AND cus.client_cd = inv.client_cd
WHERE (inv.month_id BETWEEN 202008 AND 202209) AND (inv.CLIENT_CD IN (2760,8910)) 
GROUP BY 1,2,3,4,5,6,7,8,9
);

-- Final code with drops, sales, etc in one materialized view built on top of lowest level data at which team calculated drops

Drop materialized view `metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops` ;

CREATE Materialized view `metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops`
AS
(
SELECT
  invline.store_id,
  invline.cust_hts_id,
  invline.cust_hts_desc,
  invline.branch_main_group_id,
  --invline.branch_main_group_desc,
  invline.month_id,
  invline.year_id,
  invline.fnf_cd,
  --invline.fnf_desc,
  invline.stratbuy_division_int_id,
  --invline.stratbuy_division_int_desc,
  invline.fsd_ind,
  invline.currency_code_ind,
  invline.client_cd,
  sum(invline.sell_val_nsp) as sales,
  sum(invline.invoice_lines) as lines,
  sum(invline.drops) as drops,
  sum(invline.satistical_sales_margin) as statistical_sales_margin,
  sum(invline.cu_prof_later_income_val) as later_income
FROM
  `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` invline
GROUP BY
  1,2,3,4,5,6,7,8,9,10,11
)

-- View on top of the above MV
create or replace view cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_aggr_custl2_prodl2_sales_drops
as(
select 
a.store_id,
a.cust_hts_id,
a.cust_hts_desc,
a.branch_main_group_id,
a.branch_main_group_desc,
a.month_id,
a.year_id,
a.fnf_cd,
a.fnf_desc,
a.stratbuy_division_int_id,
a.stratbuy_division_int_desc,
a.fsd_ind,
a.currency_code_ind,
a.client_cd,
a.sales,
a.lines,
a.drops,
a.statistical_sales_margin,
a.later_income from metro-bi-dl-prep-mid-pp.semantic_materialised_dev.dw_mv_aggr_custl2_prodl2_sales_drops as a )

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select
client_cd,
month_id,
currency_code_ind,
sum(sell_val_nsp) as sell_val_nsp
from `cf-mdw-int-msi-dev.semantic_msi.dw_mv_msi_invoice`
where month_id between 202109 and 202208
group by 1,2,3

Executing query with job ID: acf8af5f-a93d-41de-9b52-00ba54cf196a
Query executing: 0.58s


ERROR:
 403 Access Denied: Table cf-mdw-int-msi-dev:semantic_msi.dw_mv_msi_invoice: User does not have permission to query table cf-mdw-int-msi-dev:semantic_msi.dw_mv_msi_invoice, or perhaps it does not exist in location US.

Location: US
Job ID: acf8af5f-a93d-41de-9b52-00ba54cf196a



# New Section

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'metro-bi-wb-mag-sci-s00' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()
print('Authenticated')

Authenticated


# Misc Dimensions and Functions

## Weekly 13

### ch_weekly_13

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select * from `gerccp_metro_sls.dw_cust_freq_ind_sales_ch_weekly_13`;

Executing query with job ID: 127f2abe-9ca1-4eac-8832-1a1dd44976e1
Query executing: 0.77s


ERROR:
 404 Not found: Dataset cf-michelange-dev-sx:gerccp_metro_sls was not found in location US

Location: US
Job ID: 127f2abe-9ca1-4eac-8832-1a1dd44976e1



### ind_weekly_13

In [ ]:
%%bigquery --project cf-michelange-dev-sx
select * from `gerccp_metro_sls.dw_cust_freq_ind_weekly_13`;

Executing query with job ID: b7ffabd2-b9cb-4074-a672-e13288eb40fc
Query executing: 0.45s


ERROR:
 404 Not found: Dataset cf-michelange-dev-sx:gerccp_metro_sls was not found in location US

Location: US
Job ID: b7ffabd2-b9cb-4074-a672-e13288eb40fc



### dw_rt_iso_country

In [ ]:
%%bigquery --project cf-michelange-dev-sx
/*
create table `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_rt_iso_country` 
like `metro-bi-dl-prep-ger-prod.cc_dwh.dw_rt_iso_country` 
as select * from `metro-bi-dl-prep-ger-prod.cc_dwh.dw_rt_iso_country`;

CREATE OR REPLACE VIEW `cf-mdw-int-msi-mid-dev.semantic_msi.DW_V_RT_CLIENT`
AS
*/
SELECT 
rtc.client_cd,
rtc.client_desc,
rtc.iso_numeric_cd,
rti.iso_alpha2_cd,
rtc.iso_alpha3_cd,
rti.iso_short_desc
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_rt_client` rtc 
join `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_rt_iso_country` rti 
on rtc.iso_numeric_cd = rti.iso_numeric_cd;


Query is running:   0%|          |

Downloading:   0%|          |

,client_cd,client_desc,iso_numeric_cd,iso_alpha2_cd,iso_alpha3_cd,iso_short_desc
0,8910,Metro Cash & Carry Serbia,891,RS,SRB,SERBIA
1,2760,Metro Cash & Carry Germany,276,DE,DEU,GERMANY


# Perform  - Luminita Ifram

# SAP

In [ ]:
%%bigquery --project cf-michelange-dev-sx
SELECT 
  F.sap_entity_level_0_cd,
  L0.sap_entity_level_0_desc,
  F.sap_entity_level_1_cd,
  L1.sap_entity_level_1_desc,
  F.sap_entity_level_2_cd,
  L2.sap_entity_level_2_desc,
  F.sap_entity_level_3_cd,
  L3.sap_entity_level_3_desc,
  F.sap_entity_level_4_cd,
  L4.sap_entity_level_4_desc,
  F.sap_entity_level_5_cd,
  L5.sap_entity_level_5_desc,
  F.sap_entity_level_6_cd,
  L6.sap_entity_level_6_desc,
  F.sap_entity_level_7_cd,
  L7.sap_entity_level_7_desc,
  F.sap_entity_cd,
  E.sap_entity_desc,
  F.sap_sub_account_cd,
  SA.sap_sub_account_desc,
  F.sap_scenario_cd,
  SCE.sap_scenario_desc,
  F.month_id,
  M.month_desc,
  F.sap_currency_cd,
  F.sap_currency_ind,

  SUM(F.fsd_pnl_sales)                    AS fsd_pnl_sales,
  SUM(ACCU.fsd_pnl_sales_accu)            AS sap_value_accu,
  SUM(F.fsd_pnl_total_income)             AS fsd_pnl_total_income,
  SUM(ACCU.fsd_pnl_total_income_accu)     AS fsd_pnl_total_income_accu,  
  SUM(F.fsd_pnl_transport_cost)           AS fsd_pnl_transport_cost,
  SUM(ACCU.fsd_pnl_transport_cost_accu)   AS fsd_pnl_transport_cost_accu,
  SUM(F.fsd_pnl_cost_to_serve)            AS fsd_pnl_cost_to_serve,
  SUM(ACCU.fsd_pnl_cost_to_serve_accu)    AS fsd_pnl_cost_to_serve_accu,
  SUM(F.fsd_pnl_ebitda)                   AS fsd_pnl_ebitda,
  SUM(ACCU.fsd_pnl_ebitda_accu)           AS fsd_pnl_ebitda_accu      


FROM `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_ent_acct_m` AS F
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity` AS E ON E.sap_entity_cd=F.sap_entity_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l0` AS L0 ON L0.sap_entity_level_0_cd=F.sap_entity_level_0_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l1` AS L1 ON L1.sap_entity_level_1_cd=F.sap_entity_level_1_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l2` AS L2 ON L2.sap_entity_level_2_cd=F.sap_entity_level_2_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l3` AS L3 ON L3.sap_entity_level_3_cd=F.sap_entity_level_3_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l4` AS L4 ON L4.sap_entity_level_4_cd=F.sap_entity_level_4_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l5` AS L5 ON L5.sap_entity_level_5_cd=F.sap_entity_level_5_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l6` AS L6 ON L6.sap_entity_level_6_cd=F.sap_entity_level_6_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_entity_l7` AS L7 ON L7.sap_entity_level_7_cd=F.sap_entity_level_7_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_sub_account` AS SA ON SA.sap_sub_account_cd=F.sap_sub_account_cd
INNER JOIN `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_scenario` AS SCE ON SCE.sap_scenario_cd=F.sap_scenario_cd
INNER JOIN `metro-bi-dl-ger-prod.cc_dwh.dw_time_month` AS M ON M.month_id=F.month_id

INNER JOIN 
(
  SELECT
    F.sap_entity_cd,
    F.sap_sub_account_cd,
    F.sap_scenario_cd,
    TIME_FY.month_id,
    F.sap_currency_cd,
    F.sap_currency_ind,

    SUM(F.fsd_pnl_sales)            AS fsd_pnl_sales_accu,
    SUM(F.fsd_pnl_total_income)     AS fsd_pnl_total_income_accu,
    SUM(F.fsd_pnl_transport_cost)   AS fsd_pnl_transport_cost_accu,
    SUM(F.fsd_pnl_execution_cost)   AS fsd_pnl_execution_cost_accu,
    SUM(F.fsd_pnl_cost_to_serve)    AS fsd_pnl_cost_to_serve_accu,
    SUM(F.fsd_pnl_ebitda)           AS fsd_pnl_ebitda_accu

  FROM `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_sap_ent_acct_m` AS F 
  INNER JOIN `metro-bi-dl-ger-prod.cc_dwh_views_msi.DW_V_TIMEFY_YEAR_TO_MONTH` AS TIME_FY ON TIME_FY.fy_month_id_accu_ty=F.month_id 
  GROUP BY    
    F.sap_entity_cd,
    F.sap_sub_account_cd,
    F.sap_scenario_cd,
    TIME_FY.month_id,
    F.sap_currency_cd,
    sap_currency_ind
) AS accu
  ON  accu.sap_entity_cd      = F.sap_entity_cd
  AND accu.sap_sub_account_cd = F.sap_sub_account_cd
  AND accu.sap_scenario_cd    = F.sap_scenario_cd
  AND accu.month_id           = F.month_id
  AND accu.sap_currency_cd    = F.sap_currency_cd
  AND accu.sap_currency_ind   = F.sap_currency_ind

WHERE F.month_id = 202201 -- here can select a certain month or delete the WHERE clause

GROUP BY   
  F.sap_entity_level_0_cd,
  L0.sap_entity_level_0_desc,
  F.sap_entity_level_1_cd,
  L1.sap_entity_level_1_desc,
  F.sap_entity_level_2_cd,
  L2.sap_entity_level_2_desc,
  F.sap_entity_level_3_cd,
  L3.sap_entity_level_3_desc,
  F.sap_entity_level_4_cd,
  L4.sap_entity_level_4_desc,
  F.sap_entity_level_5_cd,
  L5.sap_entity_level_5_desc,
  F.sap_entity_level_6_cd,
  L6.sap_entity_level_6_desc,
  F.sap_entity_level_7_cd,
  L7.sap_entity_level_7_desc,
  F.sap_entity_cd,
  E.sap_entity_desc,
  F.sap_sub_account_cd,
  SA.sap_sub_account_desc,
  F.sap_scenario_cd,
  SCE.sap_scenario_desc,
  F.month_id,
  M.month_desc,
  F.sap_currency_cd,
  F.sap_currency_ind


Query is running:   0%|          |

Downloading:   0%|          |

,sap_entity_level_0_cd,sap_entity_level_0_desc,sap_entity_level_1_cd,sap_entity_level_1_desc,sap_entity_level_2_cd,sap_entity_level_2_desc,sap_entity_level_3_cd,sap_entity_level_3_desc,sap_entity_level_4_cd,sap_entity_level_4_desc,...,fsd_pnl_sales,sap_value_accu,fsd_pnl_total_income,fsd_pnl_total_income_accu,fsd_pnl_transport_cost,fsd_pnl_transport_cost_accu,fsd_pnl_cost_to_serve,fsd_pnl_cost_to_serve_accu,fsd_pnl_ebitda,fsd_pnl_ebitda_accu
0,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,OC,Other Companies (OT),OC,Other Companies (OT),OC,Other Companies (OT),...,None,None,None,None,None,None,None,None,None,None
1,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,OC,Other Companies (OT),OC,Other Companies (OT),OC,Other Companies (OT),...,None,None,None,None,None,None,None,None,None,None
2,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,OC,Other Companies (OT),OC,Other Companies (OT),OC,Other Companies (OT),...,None,None,None,None,None,None,None,None,None,None
3,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,OC,Other Companies (OT),OC,Other Companies (OT),OC,Other Companies (OT),...,None,None,None,None,None,None,None,None,None,None
4,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,OC,Other Companies (OT),OC,Other Companies (OT),OC,Other Companies (OT),...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,CCWER,West,CCESTL,Spain,CCESTL,Spain,...,None,None,None,None,None,None,None,None,None,None
732,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,CCWER,West,CCESTL,Spain,CCESTL,Spain,...,None,None,None,None,None,None,None,None,None,None
733,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,CCWER,West,CCESTL,Spain,CCESTL,Spain,...,None,None,None,None,None,None,None,None,None,None
734,OS2,METRO - Operating Segments (starting 2018/revi...,WFSOS,Operating segments,CCWER,West,CCESTL,Spain,CCESTL,Spain,...,None,None,None,None,None,None,None,None,None,None


# Perform

In [ ]:
%%bigquery --project cf-michelange-dev-sx
--lookup tables
select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi;
select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_store;
select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_district;
select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_countries; 

--fact table
select * from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi_store;

-- views
select * from cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_kpi;
select * from cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_store;
select * from cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_district;
select * from cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_countries;
select * from cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_kpi_store;

I tested as below, for “FSD Store PnL Sales”:
--SELECT from original table 
select sum(selective_value) from cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi_store_month where kpi in ('180447','180448');

-- 286310309150.82849

Executing query with job ID: a35cd65e-4b49-421f-b1b3-0f2cf78b0f32
Query executing: 0.57s


ERROR:
 400 Syntax error: Expected end of input but got identifier "I" at [17:1]

Location: EU
Job ID: a35cd65e-4b49-421f-b1b3-0f2cf78b0f32



In [ ]:
%%bigquery --project cf-michelange-dev-sx
--SELECT FROM tables
select sum(selective_value), count(*) from (
select 
ft.month_id, 
c.country_name,
c.iso_alpha2_cd,
d.district_order,
d.district_cd,
d.district_name,
s.unit_id,
s.unit_name,
s.unit_type,
k.kpi_cd,
k.kpi_desc,
ft.selective_value
from 
cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi_store ft
inner join cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_kpi k 
on ft.kpi_cd= k.kpi_cd
inner join cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_store s
on ft.client_cd = s.client_cd
and ft.store_id = s.store_id
and ft.unit_id = s.unit_id
inner join cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_district d
on ft.client_cd = d.client_cd
and ft.district_cd=d.district_cd
inner join cf-mdw-int-msi-mid-dev.cc_dwh.dw_prfm_countries c
on ft.client_cd = c.client_cd
)
where kpi_cd in ('180447', '180448'); -- example for 180447 + 180448 FSD Store PnL Sales

--286310309150.83038

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,2.863103e+11,38136


In [ ]:
%%bigquery --project cf-michelange-dev-sx
--SELECT FROM views
select 
um(fsd_store_pnl_sales) as fsd_store_pnl_sales, --compare this column with previous result
sum(fsd_store_pnl_execution_cost) as fsd_store_pnl_execution_cost,
sum(fsd_store_pnl_gross_profit) as fsd_store_pnl_gross_profit,
sum(fsd_store_pnl_later_income) as fsd_store_pnl_later_income,
sum(fsd_store_pnl_total_income) as fsd_store_pnl_total_income,
sum(fsd_store_pnl_cost_to_serve) as fsd_store_pnl_cost_to_serve,
sum(fsd_store_pnl_transport_cost) as fsd_store_pnl_transport_cost,
sum(fsd_store_pnl_marketing_cost) as fsd_store_pnl_marketing_cost,
sum(fsd_store_pnl_indirect_cost) as fsd_store_pnl_indirect_cost,
sum(fsd_store_pnl_all_otr_drct_cost) as fsd_store_pnl_all_otr_drct_cost),
sum(fsd_store_pnl_depreciation) as fsd_store_pnl_depreciation),
sum(fsd_pnl_store_pnl_incm_afr_cst_to_srv) as fsd_pnl_store_pnl_incm_afr_cst_to_srv,
sum(fsd_store_pnl_cash_profit) as fsd_store_pnl_cash_profit),
sum(fsd_store_pnl_ebitda) as fsd_store_pnl_ebitda),
sum(fsd_store_pnl_ebit) as fsd_store_pnl_ebit,
count(*) from (
select 
k.month_id, 
c.country_name,
c.iso_alpha2_cd,
d.district_order,
d.district_cd,
d.district_name,
s.unit_id,
s.unit_name,
s.unit_type,
k.fsd_store_pnl_sales,
k.fsd_store_pnl_execution_cost,
k.fsd_store_pnl_gross_profit,
k.fsd_store_pnl_later_income,
k.fsd_store_pnl_total_income,
k.fsd_store_pnl_cost_to_serve,
k.fsd_store_pnl_transport_cost,
k.fsd_store_pnl_marketing_cost,
k.fsd_store_pnl_indirect_cost,
k.fsd_store_pnl_all_otr_drct_cost,
k.fsd_store_pnl_depreciation,
k.fsd_pnl_store_pnl_incm_afr_cst_to_srv,
k.fsd_store_pnl_cash_profit,
k.fsd_store_pnl_ebitda,
k.fsd_store_pnl_ebit,
k.currency_ind,
k.currency_cd,
k.conversion_rate
from 
cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_kpi_store k
inner join cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_store s
on k.client_cd = s.client_cd
and k.store_id = s.store_id
and k.unit_id = s.unit_id
inner join cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_district d
on k.client_cd = d.client_cd
and k.district_cd=d.district_cd
inner join cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_prfm_countries c
on k.client_cd = c.client_cd
) where currency_ind = 'LC';

--286310309150.8302

Executing query with job ID: b4dbc0fc-1d59-4306-95f1-86eb9891916e
Query executing: 0.51s


ERROR:
 400 Syntax error: Expected end of input but got ")" at [12:72]

Location: EU
Job ID: b4dbc0fc-1d59-4306-95f1-86eb9891916e



# Test 2022-02-23


Detail for Ex-rate

In [3]:
%%bigquery --project cf-michelange-dev-sx
select * from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` 
where client_cd=8910
and INVOICE_GLOBAL_ID='8910-4180987'
and mge_cat_id=30
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,PARTITION_DATE,INVOICE_GLOBAL_ID,client_cd,month_id,year_id,invoice_id,cust_no,home_store_id,store_id,cust_hts_id,...,mge_cat_id,mge_cat_desc,fsd_ind,currency_code_ind,sell_val_nsp,invoice_lines,buying_cust_no,buying_customers,satistical_sales_margin,cu_prof_later_income_val
0,2023-02-23,8910-4180987,8910,202103,2021,4180987.00000000000000000000000000000000000000,518619,44,44,3,...,30,SPICES,0,LC,275.820000000,1,51861900000000044,1,47.988000000,42.224104560
1,2023-02-23,8910-4180987,8910,202103,2021,4180987.00000000000000000000000000000000000000,518619,44,44,3,...,30,SPICES,0,GC,2.346784681,1,51861900000000044,1,0.408300715,0.359259233


In [12]:
%%bigquery --project cf-michelange-dev-sx
select 
mge_cat_id,
mge_cat_desc
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` 
where client_cd=8910
and mge_cat_desc like '%ret%'
group by 1,2

Query is running:   0%|          |

Downloading: |          |

,mge_cat_id,mge_cat_desc


Overall check

In [9]:
%%bigquery --project cf-michelange-dev-sx
select 
client_cd,
year_id,
currency_code_ind,
sum(sell_val_nsp) as sls,
sum(satistical_sales_margin) as sm,
sum(cu_prof_later_income_val) as li
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE_TEMP` 
where client_cd=8910
group by 1,2,3
order by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

,client_cd,year_id,currency_code_ind,sls,sm,li
0,8910,2020,GC,79121421.607173758,6932291.774418212,6289922.127946983
1,8910,2020,LC,9299221475.460000000,814759331.100900000,739260970.570714370
2,8910,2021,GC,207224476.289481258,18778728.542311357,15300373.074309223
3,8910,2021,LC,24355304303.260000000,2207083141.310100000,1798268471.212363765
4,8910,2022,GC,195644942.770854636,16831634.827634529,13696862.157514266
5,8910,2022,LC,22994349904.440000000,1978239228.741400000,1609806195.771490234


In [16]:
%%bigquery --project cf-michelange-dev-sx
SELECT 
min(date_of_day) as min_dtd,
max(date_of_day) as max_dtd
FROM `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.dw_cust_invoice_line`
-- where date_of_day='2022-09-01'
-- limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,min_dtd,max_dtd
0,2020-07-01,2022-09-30


In [18]:
%%bigquery --project cf-michelange-dev-sx
select 
client_cd,
year_id,
currency_code_ind,
sum(sell_val_nsp) as sls,
sum(satistical_sales_margin) as sm,
sum(cu_prof_later_income_val) as li
from `metro-bi-dl-prep-mid-pp.cc_dwh_bit_dev.MSI_INVOICE` 
where client_cd=8910
group by 1,2,3
order by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

,client_cd,year_id,currency_code_ind,sls,sm,li
0,8910,2020,BC,79121421.607492499,814759331.100900000,739260970.570714370
1,8910,2020,LC,9299221475.460000000,814759331.100900000,739260970.570714370
2,8910,2021,BC,207224476.290025416,2207083141.310100000,1798268471.212363765
3,8910,2021,LC,24355304303.260000000,2207083141.310100000,1798268471.212363765
4,8910,2022,BC,195644942.771160229,1978239228.741400000,1609806195.771490234
5,8910,2022,LC,22994349904.440000000,1978239228.741400000,1609806195.771490234


In [11]:
%%bigquery --project cf-michelange-dev-sx
select 
a.district_id,
b.district_desc,
a.district_store_desc,
a.store_id
from `metro-bi-dl-srb-prod.cc_dwh_dialog.dw_udef_district_store` a
left join `metro-bi-dl-srb-prod.cc_dwh_dialog.dw_udef_district` b on a.district_id=b.district_id

Query is running:   0%|          |

Downloading:   0%|          |

,district_id,district_desc,district_store_desc,store_id
0,1,BG & NS,60 DISKONT SATOR,60
1,2,Regions,42 UZICE,42
2,2,Regions,23 NIS,23
3,2,Regions,24 KRAGUJEVAC,24
4,2,Regions,44 PALIC,44
5,2,Regions,26 SUBOTICA,26
6,3,Satellite,43 POZAREVAC,43
7,3,Satellite,41 SABAC,41
8,1,BG & NS,METRO CASH&CARRY D.O.,20
9,1,BG & NS,METRO CASH&CARRY D.O.,22


In [5]:
%%bigquery --project cf-michelange-dev-sx
select * from `metro-bi-dl-srb-prod.cc_dwh_dialog.dw_udef_district_store` ;

Query is running:   0%|          |

Downloading:   0%|          |

,district_id,agg_level_id,district_store_id,district_store_desc,store_id,dwh_insert_id,extraction_id,client_cd
0,1,1,120,METRO CASH&CARRY D.O.,20,201110152151,201110152141,8910
1,1,1,122,METRO CASH&CARRY D.O.,22,201110152151,201110152141,8910
2,1,1,121,METRO CASH&CARRY D.O.,21,201110152151,201110152141,8910
3,1,1,1025,SERBIA - ALL ST,25,201110152151,201110152141,8910
4,1,1,64,60 DISKONT SATOR,60,<NA>,<NA>,8910
5,2,1,50,42 UZICE,42,<NA>,<NA>,8910
6,2,1,31,23 NIS,23,<NA>,<NA>,8910
7,2,1,32,24 KRAGUJEVAC,24,<NA>,<NA>,8910
8,2,1,52,44 PALIC,44,<NA>,<NA>,8910
9,2,1,34,26 SUBOTICA,26,<NA>,<NA>,8910


In [12]:
#@title Default title text
%%bigquery --project cf-michelange-dev-sx
select * from `metro-bi-dl-srb-prod.cc_dwh_dialog.dw_udef_district`;

Query is running:   0%|          |

Downloading:   0%|          |

,district_id,district_desc,dwh_insert_id,extraction_id,client_cd
0,1,BG & NS,201110152151,201110152141,8910
1,3,Satellite,<NA>,<NA>,8910
2,-1,dummy,<NA>,<NA>,8910
3,2,Regions,<NA>,<NA>,8910
